# VenueSignal - Yelp Business Rating Prediction
### AAI-540 Group 6

---

## Project Overview

This notebook implements a complete end-to-end MLOps pipeline for predicting Yelp business ratings with a focus on parking availability constraints. The pipeline demonstrates MLOps best practices including:

- **Data Lake Management**: S3-based data storage with proper versioning
- **Data Cataloging**: Athena tables for queryable data access
- **Feature Engineering**: Scalable feature store implementation
- **Model Development**: Baseline and advanced models with proper evaluation
- **Model Deployment**: SageMaker endpoints for inference
- **Monitoring**: Comprehensive model, data, and infrastructure monitoring

**Key Feature**: Uses AWS Account ID for bucket naming to enable each team member to run independently in their own AWS Learning Lab environment.

---

## Table of Contents

1. [Setup & Configuration](#section-1)
2. [Data Lake Setup](#section-2)
3. [Athena Tables & Data Cataloging](#section-3)
4. [Exploratory Data Analysis](#section-4)
5. [Feature Engineering & Feature Store](#section-5)
6. [Model Training](#section-6)
   - 6.1 Benchmark Models
   - 6.2 XGBoost Model
   - 6.3 Model Comparison
7. [Model Deployment](#section-7)
8. [Monitoring & Observability](#section-8)
9. [CI/CD](#section-9)

---

## 1. Setup & Configuration <a id='section-1'></a>

This section:
- Verifies Python version
- Imports all required libraries
- Retrieves AWS Account ID for unique resource naming
- Initializes AWS clients and SageMaker session
- Configures S3 buckets using Account ID pattern

In [ ]:
# Verify Python version
!python --version

### 1.1 Import Required Libraries

In [ ]:
# Standard libraries
import gdown
import os
import json
import re
import time
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import Counter
from datetime import datetime

# AWS SDK
import boto3
from botocore import UNSIGNED
from botocore.client import Config
from botocore.exceptions import ClientError


# SageMaker
import sagemaker
from sagemaker import get_execution_role
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.model_monitor import DefaultModelMonitor
sm_client = boto3.client('sagemaker')
session = sagemaker.Session()
role = get_execution_role()
region = session.boto_region_name
sagemaker_session=session

# Athena
from pyathena import connect
from pyathena.pandas.cursor import PandasCursor

# Model training and evaluation
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report
)
from sklearn.linear_model import LinearRegression

# Monitoring
from sagemaker.model_monitor import (
    DataCaptureConfig, DefaultModelMonitor, ModelQualityMonitor,
    CronExpressionGenerator, EndpointInput
)
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.s3 import S3Downloader, S3Uploader
from datetime import datetime, timedelta, timezone
from time import sleep
from threading import Thread
import io, csv

# Google Drive download
import gdown

print("All libraries imported successfully")

### 1.2 Retrieve AWS Account ID

**IMPORTANT**: This retrieves your unique AWS Account ID which will be used to create unique S3 bucket names.
This allows each team member to run this notebook independently in their own AWS Learning Lab environment.

In [ ]:
try:
    # Get AWS Account ID
    account_id = boto3.client("sts").get_caller_identity()["Account"]
    print(f"Successfully retrieved AWS Account ID: {account_id}")
except Exception as e:
    print(f"Cannot retrieve account information: {e}")
    raise



### 1.3 Initialize AWS Clients and SageMaker Session

In [ ]:
# Initialize SageMaker session
sagemaker_session = sagemaker.Session()

# Get Execution role and AWS Region
role = get_execution_role()
print("RoleArn:", role)
REGION = sagemaker_session.boto_region_name
print("Region:", REGION)


# Initialize AWS clients
s3_client = boto3.client("s3", region_name=REGION)
s3_resource = boto3.resource("s3", region_name=REGION)
athena_client = boto3.client("athena", region_name=REGION)
sagemaker_client = boto3.client("sagemaker", region_name=REGION)
cloudwatch_client = boto3.client("cloudwatch", region_name=REGION)
logs_client = boto3.client("logs", region_name=REGION)

print(f"AWS Region: {REGION}")
print(f"SageMaker Execution Role: {role}")
print(f"AWS clients initialized successfully")

# Also create cw_client alias for consistency
cw_client = cloudwatch_client


In [ ]:
project_name = "yelp-aai540-group6"

### 1.4 Configure S3 Buckets with Account ID Pattern

**IMPORTANT**: All S3 buckets are created with your Account ID to ensure uniqueness.
This pattern is used throughout the entire pipeline.

In [ ]:
# Base bucket name with Account ID
BASE_BUCKET_NAME = f"yelp-aai540-group6-{account_id}"

# S3 Prefixes (paths within buckets)
RAW_DATA_PREFIX = "yelp-dataset/json/"
PARQUET_PREFIX = "yelp-dataset/parquet/"
ATHENA_RESULTS_PREFIX = "athena/results/"
FEATURE_PREFIX = "feature-store/"
MODEL_PREFIX = "models/"
MONITORING_PREFIX = "monitoring/"

# Individual directories within the base bucket
DATA_JSON_DIR = f"{BASE_BUCKET_NAME}/{RAW_DATA_PREFIX}"  # Raw data storage
DATA_PARQUET_DIR = f"{BASE_BUCKET_NAME}/{PARQUET_PREFIX}"  # Raw data storage
ATHENA_DIR = f"{BASE_BUCKET_NAME}/{ATHENA_RESULTS_PREFIX}"  # Athena queries and results
FEATURE_DIR = f"{BASE_BUCKET_NAME}/{FEATURE_PREFIX}"  # Feature store offline
MODEL_DIR = f"{BASE_BUCKET_NAME}/{MODEL_PREFIX}"  # Model artifacts
MONITORING_DIR = f"{BASE_BUCKET_NAME}/{MONITORING_PREFIX}"  # Monitoring data

# Full S3 paths
ATHENA_RESULTS_S3 = f"s3://{ATHENA_DIR}"

# Athena Database
ATHENA_DB = "yelp"

# Store configuration
%store REGION
%store role
%store account_id
%store BASE_BUCKET_NAME
%store DATA_JSON_DIR
%store DATA_PARQUET_DIR
%store ATHENA_DIR
%store FEATURE_DIR
%store MODEL_DIR
%store MONITORING_DIR
%store ATHENA_RESULTS_S3
%store ATHENA_DB

# Display configuration
print("="*80)
print("S3 BUCKET CONFIGURATION (Account-Specific)")
print("="*80)
print(f"AWS Account ID:     {account_id}")
print(f"AWS Region:         {REGION}")
print(f"AWS Role:         {role}")
print()
print("S3 Bucket:")
print(f"  Base Bucket:      {BASE_BUCKET_NAME}")
print("S3 Bucket Directories:")
print(f"  JSON:       {DATA_JSON_DIR}")
print(f"  Parquet:    {DATA_PARQUET_DIR}")
print(f"  Athena:     {ATHENA_DIR}")
print(f"  Feature:    {FEATURE_DIR}")
print(f"  Model:      {MODEL_DIR}")
print(f"  Monitoring: {MONITORING_DIR}")
print()
print("Athena Configuration:")
print(f"  Database:         {ATHENA_DB}")
print(f"  Results Location: {ATHENA_RESULTS_S3}")
print("="*80)

### 1.5 Create S3 Buckets

This creates all required S3 buckets for the pipeline. Each bucket is unique to your AWS account.

In [ ]:
def create_bucket_if_not_exists(bucket_name, region=REGION):
    """
    Create an S3 bucket if it doesn't already exist.

    Args:
        bucket_name: Name of the bucket to create
        region: AWS region for the bucket

    Returns:
        True if bucket was created or already exists, False otherwise
    """
    try:
        # Check if bucket exists
        s3_client.head_bucket(Bucket=bucket_name)
        print(f"  Bucket already exists: {bucket_name}")
        return True
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == '404':
            # Bucket doesn't exist, create it
            try:
                if region == 'us-east-1':
                    s3_client.create_bucket(Bucket=bucket_name)
                else:
                    s3_client.create_bucket(
                        Bucket=bucket_name,
                        CreateBucketConfiguration={'LocationConstraint': region}
                    )
                print(f"  Created bucket: {bucket_name}")
                return True
            except ClientError as create_error:
                print(f"  Error creating bucket {bucket_name}: {create_error}")
                return False
        else:
            print(f"  Error checking bucket {bucket_name}: {e}")
            return False

# Create all required buckets
print("Creating S3 bucket...")

success = True
if not create_bucket_if_not_exists(BASE_BUCKET_NAME):
    success = False

if success:
    print("\n S3 bucket is ready!")
else:
    print("\n Bucket could not be created. Please check errors above.")

---

## 2. Data Lake Setup <a id='section-2'></a>

This section:
- Downloads Yelp academic dataset from Google Drive
- Uploads raw JSON files to S3 data lake
- Organizes data in a structured format

**Data Source**: Yelp Academic Dataset (5 files, ~8.5 GB total)
- Business data (150k+ businesses)
- Review data (7M+ reviews)
- User data (2M+ users)
- Check-in data
- Tip data

### 2.1 Define Google Drive File IDs

These are the file IDs for the Yelp dataset files stored in Google Drive.

In [ ]:
# Google Drive file IDs for Yelp dataset
google_drive_file_ids = {
    "yelp_academic_dataset_business.json": "1-VQQyXape4lCTa_5bA9VTlJgqKkqqR3h",
    "yelp_academic_dataset_checkin.json": "1LcnPYD4m3jp4l7EF8s8mqp3F8QUlcr9-",
    "yelp_academic_dataset_review.json": "1Q_qpG918HVo4UpT1qyGCH4c_MfgQVgWg",
    "yelp_academic_dataset_tip.json": "1vyYognzSAMenVakNyXgchwfZlVc76ZMk",
    "yelp_academic_dataset_user.json": "1wYnBYQspgfh9PnVS3GTyjT8la5PXJni9"
}
#https://drive.google.com/file/d/1Q_qpG918HVo4UpT1qyGCH4c_MfgQVgWg/view?usp=drive_link
#https://drive.google.com/file/d/1wYnBYQspgfh9PnVS3GTyjT8la5PXJni9/view?usp=sharing
#https://drive.google.com/file/d/1M8QVg2aiAwSSQO3zRJYj35PLnMKBa5L9/view?usp=drive_link-copy
#https://drive.google.com/file/d/1kz33s_tiLydRDFRf4GMxxBIvrW_lEpTC/view?usp=drive_link-copy
#https://drive.google.com/file/d/1-VQQyXape4lCTa_5bA9VTlJgqKkqqR3h/view?usp=drive_link
#https://drive.google.com/file/d/1LcnPYD4m3jp4l7EF8s8mqp3F8QUlcr9-/view?usp=drive_link
#https://drive.google.com/file/d/1eQ8nSwENhtwu7X1aNj8XgmHy5KwcMfEU/view?usp=drive_link
#https://drive.google.com/file/d/1vyYognzSAMenVakNyXgchwfZlVc76ZMk/view?usp=drive_link
#https://drive.google.com/file/d/1yLL_31R4J1Me_CEyZCYSsJrcQkzZtxKf/view?usp=drive_link


print(f"Files to download: {len(google_drive_file_ids)}")
for filename in google_drive_file_ids.keys():
    print(f"  - {filename}")

### 2.2 Download and Upload to S3

**Process**:
1. Download each file from Google Drive
2. Upload to your account-specific S3 data bucket
3. Clean up local files to save disk space

**Warning**: This will download ~8.5 GB of data. Ensure you have sufficient disk space and network bandwidth.

In [ ]:
file_to_dir = {
    "yelp_academic_dataset_business.json": "business/",
    "yelp_academic_dataset_checkin.json": "checkin/",
    "yelp_academic_dataset_review.json": "review/",
    "yelp_academic_dataset_tip.json": "tip/",
    "yelp_academic_dataset_user.json": "user/",
}

# Change to working directory
work_dir = "/home/sagemaker-user/VenueSignal"
os.makedirs(work_dir, exist_ok=True)
os.chdir(work_dir)

print(f"Working directory: {os.getcwd()}")
print(f"Target S3 bucket: {BASE_BUCKET_NAME}")
print(f"Target S3 prefix: {RAW_DATA_PREFIX}")
print()


def process_one_file(filename, file_id, s3_client, RAW_DATA_PREFIX, BASE_BUCKET_NAME):
    # Step 1: Download from Google Drive
    download_url = f"https://drive.google.com/uc?id={file_id}"
    gdown.download(download_url, filename, quiet=True)

    # Step 2: Upload to S3
    file_dir = file_to_dir[filename]
    s3_key = f"{RAW_DATA_PREFIX}{file_dir}{filename}"
    s3_client.upload_file(filename, BASE_BUCKET_NAME, s3_key)

    # Step 3: Clean up local file
    if os.path.exists(filename):
        os.remove(filename)

    return filename, s3_key


def download_and_upload_all_concurrently(
    google_drive_file_ids: dict,
    s3_client,
    RAW_DATA_PREFIX: str,
    BASE_BUCKET_NAME: str,
    max_workers: int = 5,
):
    results = {"ok": [], "failed": []}

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(
                process_one_file,
                filename,
                file_id,
                s3_client,
                RAW_DATA_PREFIX,
                BASE_BUCKET_NAME,
            ): filename
            for filename, file_id in google_drive_file_ids.items()
        }

        for fut in as_completed(futures):
            filename = futures[fut]
            try:
                fname, s3_key = fut.result()
                results["ok"].append((fname, s3_key))
                print(f"✓ {fname} -> s3://{BASE_BUCKET_NAME}/{s3_key}")
            except Exception as e:
                results["failed"].append((filename, str(e)))
                print(f"✗ {filename} failed: {e}")

    print("\nDone.")
    print(f"Successful: {len(results['ok'])}")
    print(f"Failed:     {len(results['failed'])}")

    return results


results = download_and_upload_all_concurrently(
    google_drive_file_ids=google_drive_file_ids,
    s3_client=s3_client,
    RAW_DATA_PREFIX=RAW_DATA_PREFIX,
    BASE_BUCKET_NAME=BASE_BUCKET_NAME,
    max_workers=5,
)

print(f"\n{'='*80}")
print(" All files processed successfully!")
print(f"{'='*80}")

### 2.3 Verify Data Upload

In [ ]:
# List files in S3
s3_path = f"s3://{DATA_JSON_DIR}"
print(f"Files in {s3_path}:\n")
!aws s3 ls {s3_path} --recursive --human-readable

# Create clickable link to S3 console
from IPython.display import display, HTML
s3_console_url = f"https://s3.console.aws.amazon.com/s3/buckets/{DATA_JSON_DIR}?region={REGION}&tab=overview"
display(HTML(f'<b>View in S3 Console: <a target="_blank" href="{s3_console_url}">S3 Bucket - Yelp Dataset</a></b>'))

---

## 3. Athena Tables & Data Cataloging <a id='section-3'></a>

This section:
- Creates Athena database
- Defines table schemas for JSON data
- Creates queryable tables
- Converts JSON to Parquet for better performance

**Benefits of Athena**:
- Query data in S3 using SQL
- No data movement required
- Pay only for queries run
- Integrates with SageMaker Feature Store

### 3.1 Create Athena Database

In [ ]:
def execute_athena_query(query, database=None, output_location=ATHENA_RESULTS_S3):
    """
    Execute an Athena query and wait for completion.

    Args:
        query: SQL query to execute
        database: Athena database name (optional)
        output_location: S3 location for query results

    Returns:
        pandas.DataFrame
    """
    params = {
        "QueryString": query,
        "ResultConfiguration": {"OutputLocation": output_location},
    }
    if database:
        params["QueryExecutionContext"] = {"Database": database}

    # Start query
    qx = athena_client.start_query_execution(**params)
    qid = qx["QueryExecutionId"]

    # Wait for completion
    while True:
        resp = athena_client.get_query_execution(QueryExecutionId=qid)
        state = resp["QueryExecution"]["Status"]["State"]
        if state in ("SUCCEEDED", "FAILED", "CANCELLED"):
            break
        time.sleep(1)

    if state != "SUCCEEDED":
        reason = resp["QueryExecution"]["Status"].get("StateChangeReason", "")
        raise RuntimeError(f"Athena query {state}: {reason}\n\nQuery:\n{query}")

    # Fetch results
    paginator = athena_client.get_paginator("get_query_results")
    columns = None
    data_rows = []

    for page in paginator.paginate(QueryExecutionId=qid):
        result_set = page.get("ResultSet", {})
        metadata = result_set.get("ResultSetMetadata", {})
        col_info = metadata.get("ColumnInfo", [])

        # Resolve headers from metadata once
        if columns is None and col_info:
            columns = [c.get("Name") for c in col_info]

        for row in result_set.get("Rows", []):
            values = [c.get("VarCharValue") for c in row.get("Data", [])]

            # If Athena included a header-like first row equal to column names, skip it.
            if columns and values == columns:
                continue

            # Normalize row length to match columns
            if columns:
                if len(values) < len(columns):
                    values += [None] * (len(columns) - len(values))
                elif len(values) > len(columns):
                    values = values[:len(columns)]

            data_rows.append(values)

    # No result rows (common for DDL/DML)
    if not data_rows and not columns:
        return pd.DataFrame()

    # If we have columns but no data, still return empty df with correct headers
    if columns and not data_rows:
        return pd.DataFrame(columns=columns)

    # If columns are missing for some reason, generate generic names
    if not columns:
        max_len = max((len(r) for r in data_rows), default=0)
        columns = [f"col_{i}" for i in range(max_len)]
        data_rows = [r + [None] * (max_len - len(r)) for r in data_rows]

    return pd.DataFrame(data_rows, columns=columns)


# Create Athena database
print(f"Creating Athena database: {ATHENA_DB}")
create_db_query = f"CREATE DATABASE IF NOT EXISTS {ATHENA_DB}"
try:
    execute_athena_query(create_db_query)
    print(f" Database '{ATHENA_DB}' created successfully")
except Exception as e:
    print(f" Error creating database: {e}")

### 3.2 Define File Locations

Map table names to their S3 file locations.

In [ ]:
# Define JSON files
FILES = {
    'business': 'yelp_academic_dataset_business.json',
    'review': 'yelp_academic_dataset_review.json',
    'user': 'yelp_academic_dataset_user.json',
    'checkin': 'yelp_academic_dataset_checkin.json',
    'tip': 'yelp_academic_dataset_tip.json'
}

# Create S3 object keys
OBJECT_KEYS = {
    table: f"{RAW_DATA_PREFIX}{table}/{fname}" for table, fname in FILES.items()
}

print("File mappings:")
for table, key in OBJECT_KEYS.items():
    print(f"  {table:10} -> s3://{BASE_BUCKET_NAME}/{key}")

### 3.3 Verify File Access

In [ ]:
dest_locations = {}

print("Verifying S3 file access...\n")
for table, key in OBJECT_KEYS.items():
    try:
        s3_client.head_object(Bucket=BASE_BUCKET_NAME, Key=key)
        print(f" {table:10} {key}")
        dest_locations[table] = f"s3://{DATA_JSON_DIR}{table}/"
    except ClientError:
        print(f" {table:10} {key} NOT FOUND")

print()
print("JSON file directory destinations...\n")
for t, loc in dest_locations.items():
    print(f"{t:8} -> {loc}")

### 3.4 Create Athena Tables from JSON

Create external tables in Athena that point to the JSON files in S3.

If you experience any errors while running the table creation cells, uncomment the bellow cell and run it.

In [ ]:

TABLES = ["business", "review", "user", "checkin", "tip", "business_attributes"]

for table in TABLES:
    print(f"Dropping table: {ATHENA_DB}.{table}")
    execute_athena_query(
        f"DROP TABLE IF EXISTS {ATHENA_DB}.{table};",
        database=ATHENA_DB
    )

paginator = s3_client.get_paginator("list_objects_v2")
to_delete = []
for page in paginator.paginate(Bucket=BASE_BUCKET_NAME, Prefix=ATHENA_RESULTS_PREFIX):
    for obj in page.get("Contents", []):
        to_delete.append({"Key": obj["Key"]})

if not to_delete:
    print("✅ Nothing to delete under", f"s3://{BASE_BUCKET_NAME}/{ATHENA_RESULTS_PREFIX}")
else:
    # delete in batches of 1000 (S3 limit)
    for i in range(0, len(to_delete), 1000):
        s3_client.delete_objects(
            Bucket=BASE_BUCKET_NAME,
            Delete={"Objects": to_delete[i:i+1000]}
        )
    print(f"✅ Deleted {len(to_delete)} objects under s3://{BASE_BUCKET_NAME}/{ATHENA_RESULTS_PREFIX}")

print("✅ All tables dropped.")

In [ ]:
# TABLES = ["business", "review", "user", "checkin", "tip", "business_attributes"]

# for table in TABLES:
#     print(f"Dropping table: {ATHENA_DB}.{table}")
#     execute_athena_query(
#         f"DROP TABLE IF EXISTS {ATHENA_DB}.{table};",
#         database=ATHENA_DB
#     )

# paginator = s3_client.get_paginator("list_objects_v2")
# to_delete = []
# for page in paginator.paginate(Bucket=BASE_BUCKET_NAME, Prefix=ATHENA_RESULTS_PREFIX):
#     for obj in page.get("Contents", []):
#         to_delete.append({"Key": obj["Key"]})

# if not to_delete:
#     print("✅ Nothing to delete under", f"s3://{BASE_BUCKET_NAME}/{ATHENA_RESULTS_PREFIX}")
# else:
#     # delete in batches of 1000 (S3 limit)
#     for i in range(0, len(to_delete), 1000):
#         s3_client.delete_objects(
#             Bucket=BASE_BUCKET_NAME,
#             Delete={"Objects": to_delete[i:i+1000]}
#         )
#     print(f"✅ Deleted {len(to_delete)} objects under s3://{BASE_BUCKET_NAME}/{ATHENA_RESULTS_PREFIX}")

# print("✅ All tables dropped.")

In [ ]:
business_location = dest_locations["business"]

# parquet_prefix
business_parquet_location = f"s3://{DATA_PARQUET_DIR}business"

print("Creating temporary table")
execute_athena_query(f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {ATHENA_DB}.business_temp (
  business_id string,
  name string,
  address string,
  city string,
  state string,
  postal_code string,
  latitude double,
  longitude double,
  stars double,
  review_count int,
  is_open int,
  attributes map<string,string>,
  categories string,
  hours map<string,string>
)
ROW FORMAT SERDE 'org.openx.data.jsonserde.JsonSerDe'
WITH SERDEPROPERTIES ('ignore.malformed.json'='true')
LOCATION '{business_location}'
TBLPROPERTIES ('has_encrypted_data'='false');
""", database=ATHENA_DB)

print("Creating table with parquets")
execute_athena_query(f"""
CREATE TABLE {ATHENA_DB}.business
WITH (
  format = 'PARQUET',
  external_location = '{business_parquet_location}'
) AS
SELECT
  business_id,
  name,
  address,
  city,
  state,
  postal_code,
  latitude,
  longitude,
  stars,
  review_count,
  is_open,
  attributes,
  categories,
  hours

FROM {ATHENA_DB}.business_temp;
""", database=ATHENA_DB)

print("drop temp table")
execute_athena_query(f"DROP TABLE IF EXISTS {ATHENA_DB}.business_temp;", database=ATHENA_DB)

print(f"✅ Created table {ATHENA_DB}.business")

In [ ]:
review_location = dest_locations["review"]
# parquet_prefix
review_parquet_location = f"s3://{DATA_PARQUET_DIR}review"

print("Creating temporary table")
execute_athena_query(f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {ATHENA_DB}.review_temp (
  review_id string,
  user_id string,
  business_id string,
  stars double,
  useful int,
  funny int,
  cool int,
  text string,
  date string
)
ROW FORMAT SERDE 'org.openx.data.jsonserde.JsonSerDe'
WITH SERDEPROPERTIES ('ignore.malformed.json'='true')
LOCATION '{review_location}'
TBLPROPERTIES ('has_encrypted_data'='false');
""", database=ATHENA_DB)

print("Creating table with parquets")
execute_athena_query(f"""
CREATE TABLE {ATHENA_DB}.review
WITH (
  format = 'PARQUET',
  external_location = '{review_parquet_location}',
  partitioned_by = ARRAY['year']
) AS
SELECT
  review_id,
  user_id,
  business_id,
  stars,
  useful,
  funny,
  cool,
  text,
  date,
  CAST(substr(date, 1, 4) AS integer) AS year

FROM {ATHENA_DB}.review_temp
WHERE date IS NOT NULL;
""", database=ATHENA_DB)

print("drop temp table")
execute_athena_query(f"DROP TABLE IF EXISTS {ATHENA_DB}.review_temp;", database=ATHENA_DB)

print(f"✅ Created table {ATHENA_DB}.review")

In [ ]:
user_location = dest_locations["user"]

# parquet_prefix
user_parquet_location = f"s3://{DATA_PARQUET_DIR}user"

print("Create temp table")
execute_athena_query(f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {ATHENA_DB}.user_temp (
  user_id string,
  name string,
  review_count int,
  yelping_since string,
  friends array<string>,
  useful int,
  funny int,
  cool int,
  fans int,
  elite array<string>,
  average_stars double,
  compliment_hot int,
  compliment_more int,
  compliment_profile int,
  compliment_cute int,
  compliment_list int,
  compliment_note int,
  compliment_plain int,
  compliment_cool int,
  compliment_funny int,
  compliment_writer int,
  compliment_photos int
)
ROW FORMAT SERDE 'org.openx.data.jsonserde.JsonSerDe'
WITH SERDEPROPERTIES ('ignore.malformed.json'='true')
LOCATION '{user_location}'
TBLPROPERTIES ('has_encrypted_data'='false');
""", database=ATHENA_DB)

print("Create Parquet table")
execute_athena_query(f"""
CREATE TABLE {ATHENA_DB}.user
WITH (
  format = 'PARQUET',
  external_location = '{user_parquet_location}'
) AS
SELECT
  user_id,
  name,
  review_count,
  yelping_since,
  friends,
  useful,
  funny,
  cool,
  fans,
  elite,
  average_stars,
  compliment_hot,
  compliment_more,
  compliment_profile,
  compliment_cute,
  compliment_list,
  compliment_note,
  compliment_plain,
  compliment_cool,
  compliment_funny,
  compliment_writer,
  compliment_photos
FROM {ATHENA_DB}.user_temp;
""", database=ATHENA_DB)

print("Drop temp table")
execute_athena_query(f"DROP TABLE IF EXISTS {ATHENA_DB}.user_temp;", database=ATHENA_DB)

print(f"✅ Created table {ATHENA_DB}.user")

In [ ]:
checkin_location = dest_locations["checkin"]

# parquet_prefix
checkin_parquet_location = f"s3://{DATA_PARQUET_DIR}checkin"

print("Create temp table")
execute_athena_query(f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {ATHENA_DB}.checkin_temp (
  business_id string,
  date string
)
ROW FORMAT SERDE 'org.openx.data.jsonserde.JsonSerDe'
WITH SERDEPROPERTIES ('ignore.malformed.json'='true')
LOCATION '{checkin_location}'
TBLPROPERTIES ('has_encrypted_data'='false');
""", database=ATHENA_DB)

print("Create Parquet table")
execute_athena_query(f"""
CREATE TABLE {ATHENA_DB}.checkin
WITH (
  format = 'PARQUET',
  external_location = '{checkin_parquet_location}'
) AS
SELECT
  business_id,
  date
FROM {ATHENA_DB}.checkin_temp;
""", database=ATHENA_DB)

print("Drop temp table")
execute_athena_query(f"DROP TABLE IF EXISTS {ATHENA_DB}.checkin_temp;", database=ATHENA_DB)


print(f"✅ Created table {ATHENA_DB}.checkin")

In [ ]:
tip_location = dest_locations["tip"]

# parquet_prefix
tip_parquet_location = f"s3://{DATA_PARQUET_DIR}tip"

print("Create temp table")
execute_athena_query(f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {ATHENA_DB}.tip_temp (
  user_id string,
  business_id string,
  text string,
  date string,
  compliment_count int
)
ROW FORMAT SERDE 'org.openx.data.jsonserde.JsonSerDe'
WITH SERDEPROPERTIES ('ignore.malformed.json'='true')
LOCATION '{tip_location}'
TBLPROPERTIES ('has_encrypted_data'='false');
""", database=ATHENA_DB)

print("Create Parquet table")
execute_athena_query(f"""
CREATE TABLE {ATHENA_DB}.tip
WITH (
  format = 'PARQUET',
  external_location = '{tip_parquet_location}',
  partitioned_by = ARRAY['year']
) AS
SELECT
  user_id,
  business_id,
  text,
  date,
  compliment_count,
  CAST(substr(date, 1, 4) AS integer) AS year

FROM {ATHENA_DB}.tip_temp
WHERE date IS NOT NULL;
""", database=ATHENA_DB)

print("Drop temp table")
execute_athena_query(f"DROP TABLE IF EXISTS {ATHENA_DB}.tip_temp;", database=ATHENA_DB)

print(f"✅ Created table {ATHENA_DB}.tip")

In [ ]:
business_attributes_location = f"s3://{DATA_PARQUET_DIR}business_attributes"

execute_athena_query(f"""
CREATE TABLE {ATHENA_DB}.business_attributes
WITH (
  format = 'PARQUET',
  external_location = '{business_attributes_location}'
) AS
WITH normalized AS (
  SELECT
    business_id,
    hours,

    -- Normalize u'...' and '...' wrappers on keys + values
    map_from_entries(
      transform(
        map_entries(attributes),
        e -> CAST(
          ROW(
            regexp_replace(CAST(e[1] AS varchar), '^u?''(.*)''$', '$1'),
            regexp_replace(CAST(e[2] AS varchar), '^u?''(.*)''$', '$1')
          ) AS ROW(varchar, varchar)
        )
      )
    ) AS attrs
  FROM {ATHENA_DB}.business
  WHERE attributes IS NOT NULL
),
parsed AS (
  SELECT
    business_id,
    hours,
    -- Convert literal "None" (any case) to NULL for all attribute lookups via helper expression pattern below
    attrs,

    -- Parse BusinessParking
    TRY(
      CAST(
        json_parse(
          replace(
            replace(
              replace(
                replace(
                  regexp_replace(attrs['businessparking'], 'u''(.*?)''', '"$1"'),
                  '''', '"'
                ),
                'False', 'false'
              ),
              'True', 'true'
            ),
            'None', 'null'
          )
        ) AS map(varchar, boolean)
      )
    ) AS parking_map,

    -- Parse Ambience
    TRY(
      CAST(
        json_parse(
          replace(
            replace(
              replace(
                replace(
                  -- normalize u'...' keys inside the string
                  regexp_replace(attrs['ambience'], 'u''(.*?)''', '"$1"'),
                  '''', '"'
                ),
                'False', 'false'
              ),
              'True', 'true'
            ),
            'None', 'null'
          )
        ) AS map(varchar, boolean)
      )
    ) AS ambience_map,

    -- Parse GoodForMeal
    TRY(
      CAST(
        json_parse(
          replace(
            replace(
              replace(
                replace(
                  -- normalize u'...' keys
                  regexp_replace(attrs['goodformeal'], 'u''(.*?)''', '"$1"'),
                  '''', '"'
                ),
                'False', 'false'
              ),
              'True', 'true'
            ),
            'None', 'null'
          )
        ) AS map(varchar, boolean)
      )
    ) AS goodformeal_map,

    -- Parse BestNights
    TRY(
      CAST(
        json_parse(
          replace(
            replace(
              replace(
                replace(
                  -- normalize u'...' keys
                  regexp_replace(attrs['bestnights'], 'u''(.*?)''', '"$1"'),
                  '''', '"'
                ),
                'False', 'false'
              ),
              'True', 'true'
            ),
            'None', 'null'
          )
        ) AS map(varchar, boolean)
      )
    ) AS bestnights_map,

    -- Parse HairSpecializesIn
    TRY(
      CAST(
        json_parse(
          replace(
            replace(
              replace(
                replace(
                  -- normalize u'...' keys
                  regexp_replace(attrs['hairspecializesin'], 'u''(.*?)''', '"$1"'),
                  '''', '"'
                ),
                'False', 'false'
              ),
              'True', 'true'
            ),
            'None', 'null'
          )
        ) AS map(varchar, boolean)
      )
    ) AS hairspecializesin_map,

    -- Parse DietaryRestrictions
    TRY(
      CAST(
        json_parse(
          replace(
            replace(
              replace(
                replace(
                  -- normalize u'...' keys
                  regexp_replace(attrs['dietaryrestrictions'], 'u''(.*?)''', '"$1"'),
                  '''', '"'
                ),
                'False', 'false'
              ),
              'True', 'true'
            ),
            'None', 'null'
          )
        ) AS map(varchar, boolean)
      )
    ) AS dietaryrestrictions_map
  FROM normalized
)
SELECT
    business_id,

    -- Helper pattern: NULLIF(lower(x),'none') but preserving original case when not none
    CASE WHEN attrs['acceptsinsurance'] IS NULL OR lower(attrs['acceptsinsurance']) = 'none' THEN NULL ELSE attrs['acceptsinsurance'] END AS acceptsinsurance,
    CASE WHEN attrs['agesallowed'] IS NULL OR lower(attrs['agesallowed']) = 'none' THEN NULL ELSE attrs['agesallowed'] END AS agesallowed,
    CASE WHEN attrs['alcohol'] IS NULL OR lower(attrs['alcohol']) = 'none' THEN NULL ELSE attrs['alcohol'] END AS alcohol,
    CASE WHEN attrs['bikeparking'] IS NULL OR lower(attrs['bikeparking']) = 'none' THEN NULL ELSE attrs['bikeparking'] END AS bikeparking,
    CASE WHEN attrs['businessacceptsbitcoin'] IS NULL OR lower(attrs['businessacceptsbitcoin']) = 'none' THEN NULL ELSE attrs['businessacceptsbitcoin'] END AS businessacceptsbitcoin,
    CASE WHEN attrs['businessacceptscreditcards'] IS NULL OR lower(attrs['businessacceptscreditcards']) = 'none' THEN NULL ELSE attrs['businessacceptscreditcards'] END AS businessacceptscreditcards,
    CASE WHEN attrs['byappointmentonly'] IS NULL OR lower(attrs['byappointmentonly']) = 'none' THEN NULL ELSE attrs['byappointmentonly'] END AS byappointmentonly,
    CASE WHEN attrs['byob'] IS NULL OR lower(attrs['byob']) = 'none' THEN NULL ELSE attrs['byob'] END AS byob,
    CASE WHEN attrs['byobcorkage'] IS NULL OR lower(attrs['byobcorkage']) = 'none' THEN NULL ELSE attrs['byobcorkage'] END AS byobcorkage,
    CASE WHEN attrs['caters'] IS NULL OR lower(attrs['caters']) = 'none' THEN NULL ELSE attrs['caters'] END AS caters,
    CASE WHEN attrs['coatcheck'] IS NULL OR lower(attrs['coatcheck']) = 'none' THEN NULL ELSE attrs['coatcheck'] END AS coatcheck,
    CASE WHEN attrs['corkage'] IS NULL OR lower(attrs['corkage']) = 'none' THEN NULL ELSE attrs['corkage'] END AS corkage,
    CASE WHEN attrs['dogsallowed'] IS NULL OR lower(attrs['dogsallowed']) = 'none' THEN NULL ELSE attrs['dogsallowed'] END AS dogsallowed,
    CASE WHEN attrs['drivethru'] IS NULL OR lower(attrs['drivethru']) = 'none' THEN NULL ELSE attrs['drivethru'] END AS drivethru,
    CASE WHEN attrs['goodfordancing'] IS NULL OR lower(attrs['goodfordancing']) = 'none' THEN NULL ELSE attrs['goodfordancing'] END AS goodfordancing,
    CASE WHEN attrs['goodforkids'] IS NULL OR lower(attrs['goodforkids']) = 'none' THEN NULL ELSE attrs['goodforkids'] END AS goodforkids,
    CASE WHEN attrs['happyhour'] IS NULL OR lower(attrs['happyhour']) = 'none' THEN NULL ELSE attrs['happyhour'] END AS happyhour,
    CASE WHEN attrs['hastv'] IS NULL OR lower(attrs['hastv']) = 'none' THEN NULL ELSE attrs['hastv'] END AS hastv,
    CASE WHEN attrs['music'] IS NULL OR lower(attrs['music']) = 'none' THEN NULL ELSE attrs['music'] END AS music,
    CASE WHEN attrs['noiselevel'] IS NULL OR lower(attrs['noiselevel']) = 'none' THEN NULL ELSE attrs['noiselevel'] END AS noiselevel,
    CASE WHEN attrs['open24hours'] IS NULL OR lower(attrs['open24hours']) = 'none' THEN NULL ELSE attrs['open24hours'] END AS open24hours,
    CASE WHEN attrs['outdoorseating'] IS NULL OR lower(attrs['outdoorseating']) = 'none' THEN NULL ELSE attrs['outdoorseating'] END AS outdoorseating,
    CASE WHEN attrs['restaurantsattire'] IS NULL OR lower(attrs['restaurantsattire']) = 'none' THEN NULL ELSE attrs['restaurantsattire'] END AS restaurantsattire,
    CASE WHEN attrs['restaurantscounterservice'] IS NULL OR lower(attrs['restaurantscounterservice']) = 'none' THEN NULL ELSE attrs['restaurantscounterservice'] END AS restaurantscounterservice,
    CASE WHEN attrs['restaurantsdelivery'] IS NULL OR lower(attrs['restaurantsdelivery']) = 'none' THEN NULL ELSE attrs['restaurantsdelivery'] END AS restaurantsdelivery,
    CASE WHEN attrs['restaurantsgoodforgroups'] IS NULL OR lower(attrs['restaurantsgoodforgroups']) = 'none' THEN NULL ELSE attrs['restaurantsgoodforgroups'] END AS restaurantsgoodforgroups,
    CASE WHEN attrs['restaurantspricerange2'] IS NULL OR lower(attrs['restaurantspricerange2']) = 'none' THEN NULL ELSE attrs['restaurantspricerange2'] END AS restaurantspricerange2,
    CASE WHEN attrs['restaurantsreservations'] IS NULL OR lower(attrs['restaurantsreservations']) = 'none' THEN NULL ELSE attrs['restaurantsreservations'] END AS restaurantsreservations,
    CASE WHEN attrs['restaurantstableservice'] IS NULL OR lower(attrs['restaurantstableservice']) = 'none' THEN NULL ELSE attrs['restaurantstableservice'] END AS restaurantstableservice,
    CASE WHEN attrs['restaurantstakeout'] IS NULL OR lower(attrs['restaurantstakeout']) = 'none' THEN NULL ELSE attrs['restaurantstakeout'] END AS restaurantstakeout,
    CASE WHEN attrs['smoking'] IS NULL OR lower(attrs['smoking']) = 'none' THEN NULL ELSE attrs['smoking'] END AS smoking,
    CASE WHEN attrs['wheelchairaccessible'] IS NULL OR lower(attrs['wheelchairaccessible']) = 'none' THEN NULL ELSE attrs['wheelchairaccessible'] END AS wheelchairaccessible,
    CASE WHEN attrs['wifi'] IS NULL OR lower(attrs['wifi']) = 'none' THEN NULL ELSE attrs['wifi'] END AS wifi,

    -- Parking
    parking_map['garage']    AS parking_garage,
    parking_map['street']    AS parking_street,
    parking_map['validated'] AS parking_validated,
    parking_map['lot']       AS parking_lot,
    parking_map['valet']     AS parking_valet,

    -- Ambience
    ambience_map['divey']     AS ambience_divey,
    ambience_map['hipster']  AS ambience_hipster,
    ambience_map['casual']   AS ambience_casual,
    ambience_map['touristy'] AS ambience_touristy,
    ambience_map['trendy']   AS ambience_trendy,
    ambience_map['intimate'] AS ambience_intimate,
    ambience_map['romantic'] AS ambience_romantic,
    ambience_map['classy']   AS ambience_classy,
    ambience_map['upscale']  AS ambience_upscale,

    -- GoodForMeal
    goodformeal_map['dessert']    AS good_for_dessert,
    goodformeal_map['latenight'] AS good_for_latenight,
    goodformeal_map['lunch']     AS good_for_lunch,
    goodformeal_map['dinner']    AS good_for_dinner,
    goodformeal_map['brunch']    AS good_for_brunch,
    goodformeal_map['breakfast'] AS good_for_breakfast,

    -- BestNights
    bestnights_map['monday']    AS bestnight_monday,
    bestnights_map['tuesday']   AS bestnight_tuesday,
    bestnights_map['wednesday'] AS bestnight_wednesday,
    bestnights_map['thursday']  AS bestnight_thursday,
    bestnights_map['friday']    AS bestnight_friday,
    bestnights_map['saturday']  AS bestnight_saturday,
    bestnights_map['sunday']    AS bestnight_sunday,

    -- HairSpecializesIn
    hairspecializesin_map['africanamerican'] AS hair_africanamerican,
    hairspecializesin_map['asian']           AS hair_asian,
    hairspecializesin_map['coloring']        AS hair_coloring,
    hairspecializesin_map['curly']           AS hair_curly,
    hairspecializesin_map['extensions']      AS hair_extensions,
    hairspecializesin_map['kids']            AS hair_kids,
    hairspecializesin_map['perms']           AS hair_perms,
    hairspecializesin_map['straightperms']   AS hair_straightperms,

    -- DietaryRestrictions
    dietaryrestrictions_map['dairy-free']      AS dairy_free,
    dietaryrestrictions_map['gluten-free']    AS gluten_free,
    dietaryrestrictions_map['vegan']           AS vegan,
    dietaryrestrictions_map['kosher']          AS kosher,
    dietaryrestrictions_map['halal']           AS halal,
    dietaryrestrictions_map['soy-free']        AS soy_free,
    dietaryrestrictions_map['vegetarian']      AS vegetarian,

    -- Hours
    hours['monday']    AS hours_monday,
    hours['tuesday']   AS hours_tuesday,
    hours['wednesday'] AS hours_wednesday,
    hours['thursday']  AS hours_thursday,
    hours['friday']    AS hours_friday,
    hours['saturday']  AS hours_saturday,
    hours['sunday']    AS hours_sunday,

    CARDINALITY(map_keys(hours)) AS open_days_count,
    CASE WHEN hours['saturday'] IS NOT NULL OR hours['sunday'] IS NOT NULL THEN true ELSE false END AS open_on_weekend

FROM parsed;
""", database=ATHENA_DB)

print(f"✅ Built {ATHENA_DB}.business_attributes")
print("📍 Location:", business_attributes_location)

### 3.5 Connect to Athena with PyAthena

Create a connection to query the tables using pandas.

In [ ]:
# Create PyAthena connection
conn = connect(
    s3_staging_dir=ATHENA_RESULTS_S3,
    region_name=REGION,
    cursor_class=PandasCursor
)

print(f" Connected to Athena database: {ATHENA_DB}")
print(f"   Results location: {ATHENA_RESULTS_S3}")

### 3.6 Test Athena Tables

Run sample queries to verify table creation.

In [ ]:
# Query business table
query = f"""
SELECT
    COUNT(*) as total_businesses,
    COUNT(DISTINCT city) as unique_cities,
    COUNT(DISTINCT state) as unique_states
FROM {ATHENA_DB}.business
LIMIT 10
"""

print("Testing business table...")
df = pd.read_sql(query, conn)
display(df)



# Query review table
query = f"""
SELECT
    COUNT(*) as total_reviews,
    AVG(stars) as avg_stars,
    MIN(stars) as min_stars,
    MAX(stars) as max_stars
FROM {ATHENA_DB}.review
LIMIT 10
"""

print("\nTesting review table...")
df = pd.read_sql(query, conn)
display(df)

print("\n Athena tables are working correctly!")

---

## 4. Exploratory Data Analysis <a id='section-4'></a>

This section explores the Yelp dataset to understand:
- Business distribution across cities and states
- Review patterns and rating distributions
- Parking availability and its relationship to ratings
- Data quality issues

**Focus**: Understanding how parking constraints affect business ratings

### 4.1 Load Sample Data

In [ ]:
# Load a sample of businesses with parking information
query = f"""
SELECT
    business_id,
    name,
    city,
    state,
    stars,
    review_count,
    categories
FROM {ATHENA_DB}.business
WHERE is_open = 1
    AND review_count >= 10
"""

print("Loading sample business data...")
business_df = pd.read_sql(query, conn)
print(f" Loaded {len(business_df):,} businesses")

# Show sample data
print("\nSample data:")
display(business_df.head())

# Show basic statistics
print(f"\n Data Summary:")
print(f"   Total businesses: {len(business_df):,}")
print(f"   Unique cities: {business_df['city'].nunique():,}")
print(f"   Unique states: {business_df['state'].nunique()}")
print(f"   Average rating: {business_df['stars'].mean():.2f}")
print(f"   Average reviews: {business_df['review_count'].mean():.0f}")

print("\n Note: Parking features will be extracted from review text in Section 5")
print("   This provides more accurate parking information than business attributes!")


### 4.2 Analyze Parking Features

In [ ]:
print("\n" + "="*80)
print("BUSINESS DISTRIBUTION ANALYSIS")
print("="*80)

# Top cities
print("\n  Top 10 Cities:")
top_cities = business_df['city'].value_counts().head(10)
for idx, (city, count) in enumerate(top_cities.items(), 1):
    print(f"   {idx:2d}. {city:20s} - {count:,} businesses")

# Top states
print("\n  Top 10 States:")
top_states = business_df['state'].value_counts().head(10)
for idx, (state, count) in enumerate(top_states.items(), 1):
    print(f"   {idx:2d}. {state:5s} - {count:,} businesses")

# Rating distribution
print("\n Rating Distribution:")
rating_dist = business_df['stars'].value_counts().sort_index()
for stars, count in rating_dist.items():
    bar_length = int(count / rating_dist.max() * 40)
    bar = "█" * bar_length
    print(f"   {stars:.1f} stars: {bar} ({count:,})")

# Review count statistics
print("\n Review Count Statistics:")
review_stats = business_df['review_count'].describe()
print(f"   Min:     {review_stats['min']:,.0f}")
print(f"   25th:    {review_stats['25%']:,.0f}")
print(f"   Median:  {review_stats['50%']:,.0f}")
print(f"   75th:    {review_stats['75%']:,.0f}")
print(f"   Max:     {review_stats['max']:,.0f}")
print(f"   Mean:    {review_stats['mean']:,.0f}")


### 4.3 Visualize Key Patterns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 10)

# Create subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Rating Distribution
axes[0, 0].hist(business_df['stars'], bins=20, edgecolor='black', alpha=0.7, color='steelblue')
axes[0, 0].set_title('Distribution of Business Ratings', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Stars', fontsize=11)
axes[0, 0].set_ylabel('Count', fontsize=11)
axes[0, 0].grid(True, alpha=0.3)

# 2. Review Count Distribution (Log Scale)
axes[0, 1].hist(business_df['review_count'], bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[0, 1].set_yscale('log')
axes[0, 1].set_title('Distribution of Review Counts (Log Scale)', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Number of Reviews', fontsize=11)
axes[0, 1].set_ylabel('Count (log scale)', fontsize=11)
axes[0, 1].grid(True, alpha=0.3)

# 3. Top 10 Cities
top_cities = business_df['city'].value_counts().head(10)
y_pos = range(len(top_cities))
axes[1, 0].barh(y_pos, top_cities.values, alpha=0.7, color='green')
axes[1, 0].set_yticks(y_pos)
axes[1, 0].set_yticklabels(top_cities.index)
axes[1, 0].set_title('Top 10 Cities by Business Count', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Number of Businesses', fontsize=11)
axes[1, 0].invert_yaxis()
axes[1, 0].grid(True, alpha=0.3, axis='x')

# 4. Top 10 States
top_states = business_df['state'].value_counts().head(10)
x_pos = range(len(top_states))
axes[1, 1].bar(x_pos, top_states.values, alpha=0.7, color='purple')
axes[1, 1].set_xticks(x_pos)
axes[1, 1].set_xticklabels(top_states.index, rotation=45)
axes[1, 1].set_title('Top 10 States by Business Count', fontsize=14, fontweight='bold')
axes[1, 1].set_ylabel('Number of Businesses', fontsize=11)
axes[1, 1].grid(True, alpha=0.3, axis='y')

# Adjust layout and save
plt.tight_layout()
plt.savefig('eda_business_overview.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n Visualizations created successfully!")
print("   Saved as: eda_business_overview.png")

# Additional: Category analysis if categories exist
if 'categories' in business_df.columns:
    print("\n" + "="*80)
    print("CATEGORY ANALYSIS")
    print("="*80)

    # Extract individual categories
    all_categories = []
    for cats in business_df['categories'].dropna():
        if isinstance(cats, str):
            all_categories.extend([c.strip() for c in cats.split(',')])

    from collections import Counter
    category_counts = Counter(all_categories)

    print(f"\n Total unique categories: {len(category_counts):,}")
    print("\n🔝 Top 15 Categories:")
    for idx, (cat, count) in enumerate(category_counts.most_common(15), 1):
        print(f"   {idx:2d}. {cat:30s} - {count:,} businesses")

print("\n" + "="*80)
print(" Section 4 Complete - EDA")

---

## 5. Feature Engineering & Feature Store <a id='section-5'></a>

This section:
- Engineers features from raw data
- Creates parking-related features
- Stores features in SageMaker Feature Store
- Splits data into train/test/validation sets

**Key Features**:
- Parking availability indicators
- Review aggregations
- Business characteristics
- Target variable: High rating indicator (4+ stars)

### 5.1 Load Full Dataset from Athena

In [ ]:
%%time

# Query to join business and review data
query = f"""
SELECT
    b.business_id,
    b.name,
    b.city,
    b.state,
    b.stars as business_stars,
    b.review_count as business_review_count,
    b.categories,
    r.review_id,
    r.user_id,
    r.stars as review_stars,
    r.useful,
    r.funny,
    r.cool,
    r.text as review_text,
    r.date as review_date
FROM {ATHENA_DB}.business b
INNER JOIN {ATHENA_DB}.review r
    ON b.business_id = r.business_id
WHERE b.is_open = 1
    AND b.review_count >= 10
    AND r.stars IS NOT NULL
LIMIT 100000
"""

print("⏳ Executing query...")
df = pd.read_sql(query, conn)
print(f"\n Loaded {len(df):,} reviews from {df['business_id'].nunique():,} businesses")
print(f"   Date range: {df['review_date'].min()} to {df['review_date'].max()}")
print(f"   Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Display sample
print("\n Sample data:")
display(df.head())

### 5.2 Engineer Features

In [ ]:
# Step 1: Extract Parking Information from Review Text
print("\n1️  Extracting parking information from review text...")

def extract_parking_features(text):
    """
    Extract parking-related features from review text.
    Returns a dict with parking indicators.
    """
    if pd.isna(text):
        return {
            'mentions_parking': 0,
            'parking_positive': 0,
            'parking_negative': 0,
            'parking_type_lot': 0,
            'parking_type_street': 0,
            'parking_type_garage': 0,
            'parking_type_valet': 0,
            'parking_free': 0,
            'parking_paid': 0
        }

    text_lower = text.lower()

    # Check if parking is mentioned
    parking_keywords = ['parking', 'park', 'parked']
    mentions_parking = int(any(keyword in text_lower for keyword in parking_keywords))

    # Positive parking indicators
    positive_keywords = [
        'easy parking', 'plenty of parking', 'ample parking',
        'free parking', 'good parking', 'great parking',
        'lots of parking', 'parking available', 'easy to park'
    ]
    parking_positive = int(any(keyword in text_lower for keyword in positive_keywords))

    # Negative parking indicators
    negative_keywords = [
        'no parking', 'parking nightmare', 'hard to park',
        'difficult parking', 'parking is terrible', 'parking sucks',
        'nowhere to park', 'parking is bad', 'limited parking',
        'parking is horrible', 'parking was awful'
    ]
    parking_negative = int(any(keyword in text_lower for keyword in negative_keywords))

    # Parking types
    parking_type_lot = int('parking lot' in text_lower or 'lot parking' in text_lower)
    parking_type_street = int('street parking' in text_lower or 'park on the street' in text_lower)
    parking_type_garage = int('parking garage' in text_lower or 'garage parking' in text_lower)
    parking_type_valet = int('valet' in text_lower)

    # Cost indicators
    parking_free = int('free parking' in text_lower or 'free park' in text_lower)
    parking_paid = int('paid parking' in text_lower or 'pay for parking' in text_lower or 'parking fee' in text_lower)

    return {
        'mentions_parking': mentions_parking,
        'parking_positive': parking_positive,
        'parking_negative': parking_negative,
        'parking_type_lot': parking_type_lot,
        'parking_type_street': parking_type_street,
        'parking_type_garage': parking_type_garage,
        'parking_type_valet': parking_type_valet,
        'parking_free': parking_free,
        'parking_paid': parking_paid
    }

# Apply parking extraction
parking_features = df['review_text'].apply(extract_parking_features)
parking_df = pd.DataFrame(parking_features.tolist())

# Add to main dataframe
for col in parking_df.columns:
    df[col] = parking_df[col]

print(f" Extracted parking features from {len(df):,} reviews")
print(f"\n   Parking Statistics:")
print(f"   - Reviews mentioning parking: {df['mentions_parking'].sum():,} ({df['mentions_parking'].mean()*100:.1f}%)")
print(f"   - Positive parking mentions: {df['parking_positive'].sum():,}")
print(f"   - Negative parking mentions: {df['parking_negative'].sum():,}")
print(f"   - Free parking mentions: {df['parking_free'].sum():,}")


# Step 2: Parse review date and create temporal features
print("\n2️  Creating temporal features...")

df['review_date'] = pd.to_datetime(df['review_date'])
df['review_year'] = df['review_date'].dt.year
df['review_month'] = df['review_date'].dt.month
df['review_day_of_week'] = df['review_date'].dt.dayofweek
df['review_quarter'] = df['review_date'].dt.quarter

print(" Created temporal features")


# Step 3: Create engagement score
print("\n3️  Creating engagement features...")

df['engagement_score'] = df['useful'] + df['funny'] + df['cool']
df['is_engaged'] = (df['engagement_score'] > 0).astype(int)

print(" Created engagement features")


# Step 4: Create target variable
print("\n4️  Creating target variable...")

df['is_highly_rated'] = (df['review_stars'] >= 4).astype(int)

print(f" Created target variable")
print(f"   - Highly rated (4+ stars): {df['is_highly_rated'].sum():,} ({df['is_highly_rated'].mean()*100:.1f}%)")
print(f"   - Not highly rated: {(1-df['is_highly_rated']).sum():,} ({(1-df['is_highly_rated']).mean()*100:.1f}%)")


# Step 5: Business-level aggregations
print("\n5️  Creating business-level aggregates...")

business_agg = df.groupby('business_id').agg({
    'review_stars': ['mean', 'std', 'count', 'min', 'max'],
    'engagement_score': ['mean', 'sum'],
    'is_highly_rated': 'mean',
    'mentions_parking': ['sum', 'mean'],
    'parking_positive': 'sum',
    'parking_negative': 'sum',
    'parking_type_lot': 'sum',
    'parking_type_street': 'sum',
    'parking_type_garage': 'sum',
    'parking_type_valet': 'sum',
    'parking_free': 'sum'
}).reset_index()

# Flatten column names
business_agg.columns = [
    'business_id',
    'avg_review_stars', 'std_review_stars', 'total_reviews', 'min_review_stars', 'max_review_stars',
    'avg_engagement', 'total_engagement',
    'pct_highly_rated',
    'total_parking_mentions', 'pct_reviews_mention_parking',
    'total_positive_parking', 'total_negative_parking',
    'total_lot_mentions', 'total_street_mentions', 'total_garage_mentions', 'total_valet_mentions',
    'total_free_parking_mentions'
]

# Create derived parking features
business_agg['has_parking_data'] = (business_agg['total_parking_mentions'] > 0).astype(int)
business_agg['parking_sentiment'] = (
    (business_agg['total_positive_parking'] - business_agg['total_negative_parking']) /
    (business_agg['total_parking_mentions'] + 1)  # +1 to avoid division by zero
)

# Determine dominant parking type
business_agg['primary_parking_type'] = business_agg[
    ['total_lot_mentions', 'total_street_mentions', 'total_garage_mentions', 'total_valet_mentions']
].idxmax(axis=1).str.replace('total_', '').str.replace('_mentions', '')

print(f" Created business-level aggregates")
print(f"   - Unique businesses: {len(business_agg):,}")
print(f"   - Businesses with parking data: {business_agg['has_parking_data'].sum():,} ({business_agg['has_parking_data'].mean()*100:.1f}%)")

# Display sample
print("\n Sample business aggregates:")
display(business_agg.head())


# Step 6: Merge aggregates back to reviews
print("\n6️  Merging business aggregates back to review data...")

df = df.merge(business_agg, on='business_id', how='left', suffixes=('', '_agg'))

print(f" Merged aggregates - DataFrame now has {len(df.columns)} columns")


# Step 7: Handle categories
print("\n7️  Processing business categories...")

# Check if business is a restaurant
df['is_restaurant'] = df['categories'].fillna('').str.contains('Restaurant|Food', case=False, na=False).astype(int)

# Extract price range from categories (if present)
def extract_price_range(categories):
    if pd.isna(categories):
        return 2  # Default to medium price
    # Look for price indicators in categories
    if any(word in categories.lower() for word in ['$$$', 'expensive', 'upscale', 'fine dining']):
        return 4
    elif any(word in categories.lower() for word in ['$$', 'moderate']):
        return 3
    elif any(word in categories.lower() for word in ['$', 'cheap', 'budget', 'fast food']):
        return 1
    else:
        return 2  # Default

df['price_range_numeric'] = df['categories'].apply(extract_price_range)

print(f" Processed categories")
print(f"   - Restaurants: {df['is_restaurant'].sum():,} ({df['is_restaurant'].mean()*100:.1f}%)")


# Step 8: Create enhanced parking score
print("\n8️  Creating enhanced parking score...")

# Weighted parking score combining multiple factors
df['enhanced_parking_score'] = (
    df['parking_positive'] * 2 +           # Positive mention worth 2 points
    df['parking_negative'] * -2 +          # Negative mention -2 points
    df['parking_free'] * 1 +               # Free parking worth 1 point
    df['parking_type_lot'] * 0.5 +         # Lot parking worth 0.5
    df['parking_type_garage'] * 0.5 +      # Garage parking worth 0.5
    df['parking_type_valet'] * 0.3 +       # Valet worth 0.3
    df['parking_type_street'] * 0.2        # Street parking worth 0.2
)

# Normalize to 0-10 scale
df['enhanced_parking_score'] = (
    (df['enhanced_parking_score'] - df['enhanced_parking_score'].min()) /
    (df['enhanced_parking_score'].max() - df['enhanced_parking_score'].min()) * 10
)

print(f" Created enhanced parking score")
print(f"   - Mean score: {df['enhanced_parking_score'].mean():.2f}")
print(f"   - Median score: {df['enhanced_parking_score'].median():.2f}")


# Display final feature summary
print("\n" + "="*80)
print("FEATURE ENGINEERING COMPLETE")
print("="*80)
print(f"\n Final Dataset Summary:")
print(f"   Total records: {len(df):,}")
print(f"   Total features: {len(df.columns)}")
print(f"   Unique businesses: {df['business_id'].nunique():,}")
print(f"   Unique users: {df['user_id'].nunique():,}")
print(f"\n Target Variable Distribution:")
print(f"   Highly rated (4+ stars): {df['is_highly_rated'].sum():,} ({df['is_highly_rated'].mean()*100:.1f}%)")
print(f"   Not highly rated: {(1-df['is_highly_rated']).sum():,} ({(1-df['is_highly_rated']).mean()*100:.1f}%)")
print(f"\n Parking Feature Coverage:")
print(f"   Reviews with parking mentions: {df['mentions_parking'].sum():,} ({df['mentions_parking'].mean()*100:.1f}%)")
print(f"   Businesses with parking data: {business_agg['has_parking_data'].sum():,}")

### 5.3 Prepare Data for Feature Store

In [ ]:
# Select features for Feature Store
feature_columns = [
    'review_id',  # Primary key
    'business_id',
    'user_id',
    # Parking features (extracted from review text)
    'mentions_parking',
    'parking_positive',
    'parking_negative',
    'parking_type_lot',
    'parking_type_street',
    'parking_type_garage',
    'parking_type_valet',
    'parking_free',
    'parking_paid',
    'enhanced_parking_score',
    # Business features (from aggregates)
    'business_stars',
    'business_review_count',
    'avg_review_stars',
    'std_review_stars',
    'total_reviews',
    'avg_engagement',
    'pct_highly_rated',
    'has_parking_data',
    'parking_sentiment',
    # Review features
    'review_stars',
    'useful',
    'funny',
    'cool',
    'engagement_score',
    'is_engaged',
    'review_year',
    'review_month',
    'review_quarter',
    # Business type
    'is_restaurant',
    'price_range_numeric',
    # Target
    'is_highly_rated'
]

# Create feature store dataframe
print("\n1️  Selecting features...")
fs_df = df[feature_columns].copy()

# Add event time (required by Feature Store)
print("\n2️  Adding event time...")
fs_df['event_time'] = pd.Timestamp.now().isoformat()

# Remove any remaining nulls
print("\n3️  Handling missing values...")
initial_count = len(fs_df)
fs_df = fs_df.dropna()
print(f"   Dropped {initial_count - len(fs_df):,} rows with missing values")
print(f"   Remaining: {len(fs_df):,} rows")

# Add data split column for train/test/validation
print("\n4️  Creating train/test/validation splits...")
np.random.seed(42)
fs_df['split'] = np.random.choice(
    ['train', 'validation', 'test', 'production'],
    size=len(fs_df),
    p=[0.4, 0.1, 0.1, 0.4]  # 40% train, 10% val, 10% test, 40% production
)

print("\n Data split distribution:")
print(fs_df['split'].value_counts().sort_index())
print(f"\n   Train:      {len(fs_df[fs_df['split']=='train']):,} rows")
print(f"   Validation: {len(fs_df[fs_df['split']=='validation']):,} rows")
print(f"   Test:       {len(fs_df[fs_df['split']=='test']):,} rows")
print(f"   Production: {len(fs_df[fs_df['split']=='production']):,} rows")

# Display sample
print("\n Feature Store DataFrame sample:")
display(fs_df.head())

print("\n" + "="*80)
print(" SECTION 5 COMPLETE")
print("="*80)
print(f"\nFeature Store DataFrame:")
print(f"  - Total records: {len(fs_df):,}")
print(f"  - Total features: {len(feature_columns)}")
print(f"  - Memory usage: {fs_df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"\n Ready to ingest into SageMaker Feature Store")

### 5.4 Create SageMaker Feature Store

Store the engineered features in SageMaker Feature Store for:
- Versioned feature access
- Online and offline feature serving
- Feature reuse across models

In [ ]:
# Feature store configuration using Account ID
feature_group_name = f"venuesignal-features-{account_id}"
feature_store_bucket = f"s3://{FEATURE_DIR}"

print(f"Feature Group Name: {feature_group_name}")
print(f"Offline Store: {feature_store_bucket}")

In [ ]:
# Create feature group
feature_group = FeatureGroup(
    name=feature_group_name,
    sagemaker_session=sagemaker_session
)

# Load feature definitions from dataframe
feature_group.load_feature_definitions(data_frame=fs_df)

print(f"\n Feature group configured with {len(fs_df.columns)} features")

In [ ]:
# Create the feature group (if it doesn't exist)
try:
    feature_group.create(
        s3_uri=feature_store_bucket,
        record_identifier_name="review_id",
        event_time_feature_name="event_time",
        role_arn=role,
        enable_online_store=False  # Only offline store for this project
    )
    print(f" Created feature group: {feature_group_name}")
    print("   Waiting for creation to complete (this may take a few minutes)...")

    # Wait for feature group to be created
    import time
    while True:
        status = feature_group.describe()['FeatureGroupStatus']
        if status == 'Created':
            print(" Feature group is ready!")
            break
        elif status == 'CreateFailed':
            print(" Feature group creation failed")
            break
        print(f"   Status: {status}...")
        time.sleep(30)

except Exception as e:
    if 'ResourceInUse' in str(e):
        print(f" Feature group '{feature_group_name}' already exists")
    else:
        print(f" Error creating feature group: {e}")

In [ ]:
# Ingest features into feature store
from datetime import datetime

print("Fixing event_time format...")
current_time = datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
fs_df['event_time'] = current_time  # Use same timestamp for all rows
fs_df['event_time'] = fs_df['event_time'].astype(str)  # Ensure string type
print(f" Fixed event_time: {current_time}")

# Now ingest in small batches
print(f"\nIngesting {len(fs_df):,} records...")
batch_size = 1000
successful = 0

for i in range(0, len(fs_df), batch_size):
    batch = fs_df.iloc[i:i+batch_size].copy()
    batch['event_time'] = current_time  # Ensure format

    try:
        feature_group.ingest(data_frame=batch, max_workers=2, wait=True)
        successful += len(batch)
        print(f"   Batch {i//batch_size + 1}: {successful:,} total records", end='\r')
    except Exception as e:
        if 'already' in str(e).lower():
            print(f"    Batch {i//batch_size + 1}: Already ingested")
        else:
            print(f"   Batch {i//batch_size + 1}: {str(e)[:80]}")

print(f"\n\n Ingestion complete! {successful:,} records ingested")

### 5.5 Export Features for Training

Export features from Feature Store to S3 for model training.

In [ ]:
# Step 1: Split the data
print("\n1️  Splitting data...")
train_df = fs_df[fs_df['split'] == 'train'].drop(columns=['event_time', 'split'])
validation_df = fs_df[fs_df['split'] == 'validation'].drop(columns=['event_time', 'split'])
test_df = fs_df[fs_df['split'] == 'test'].drop(columns=['event_time', 'split'])
production_df = fs_df[fs_df['split'] == 'production'].drop(columns=['event_time', 'split'])

print(f"    Train:      {len(train_df):,} records")
print(f"    Validation: {len(validation_df):,} records")
print(f"    Test:       {len(test_df):,} records")
print(f"    Production: {len(production_df):,} records")

# Step 2: Save locally (SIMPLE path - no complex directories!)
print("\n2️  Saving to local files...")

# Just use /tmp directly - simple and always works!
train_df.to_csv('/tmp/train.csv', index=False)
print("    /tmp/train.csv")

validation_df.to_csv('/tmp/validation.csv', index=False)
print("    /tmp/validation.csv")

test_df.to_csv('/tmp/test.csv', index=False)
print("    /tmp/test.csv")

production_df.to_csv('/tmp/production.csv', index=False)
print("    /tmp/production.csv")

# Step 3: Upload to S3
print("\n3️  Uploading to S3...")

# Define S3 paths
train_data_path = f"s3://{FEATURE_DIR}training-data/train.csv"
validation_data_path = f"s3://{FEATURE_DIR}training-data/validation.csv"
test_data_path = f"s3://{FEATURE_DIR}training-data/test.csv"
production_data_path = f"s3://{FEATURE_DIR}training-data/production.csv"

# Upload each file
uploads = [
    ('/tmp/train.csv', train_data_path, 'train.csv'),
    ('/tmp/validation.csv', validation_data_path, 'validation.csv'),
    ('/tmp/test.csv', test_data_path, 'test.csv'),
    ('/tmp/production.csv', production_data_path, 'production.csv')
]

for local_file, s3_uri, display_name in uploads:
    bucket = s3_uri.split('/')[2]
    key = '/'.join(s3_uri.split('/')[3:])

    try:
        s3_client.upload_file(local_file, bucket, key)
        print(f"    {display_name} → {s3_uri}")
    except Exception as e:
        print(f"    {display_name} failed: {e}")

# Step 4: Store variables
print("\n4️  Storing variables...")
%store train_data_path
%store validation_data_path
%store test_data_path
%store production_data_path

print("    Variables stored")

# Step 5: Summary
print("\n" + "="*80)
print(" SECTION 5.7 COMPLETE - DATA EXPORTED!")
print("="*80)

print(f"\n S3 Locations:")
print(f"   Train:      {train_data_path}")
print(f"   Validation: {validation_data_path}")
print(f"   Test:       {test_data_path}")
print(f"   Production: {production_data_path}")

print(f"  Local Files (temporary):")
for filename in ['train.csv', 'validation.csv', 'test.csv', 'production.csv']:
    filepath = f'/tmp/{filename}'
    if os.path.exists(filepath):
        size_mb = os.path.getsize(filepath) / (1024 * 1024)
        print(f"   {filepath:30s} - {size_mb:6.2f} MB")

print("\n" + "="*80)
print(" SECTION 5 COMPLETE!")
print("="*80)
print("\n Summary:")
print(f"    Features engineered from review text")
print(f"    {len(fs_df):,} total records processed")
print(f"    Training data split and exported")
print(f"    All data in S3 and ready for model training")

---

## 6. Model Training <a id='section-6'></a>

This section trains and evaluates multiple models:

1. **Baseline Model #1**: Simple heuristic (business average rating)
2. **Baseline Model #2**: Linear regression with key features
3. **XGBoost Model**: Gradient boosted trees for classification

**Goal**: Predict whether a review will be highly rated (4+ stars) based on business characteristics, especially parking availability.

In [ ]:
# Define S3 paths
train_data_path = f"s3://{FEATURE_DIR}training-data/train.csv"
validation_data_path = f"s3://{FEATURE_DIR}training-data/validation.csv"
test_data_path = f"s3://{FEATURE_DIR}training-data/test.csv"
production_data_path = f"s3://{FEATURE_DIR}training-data/production.csv"

### 6.1 Load Training Data

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    mean_squared_error, mean_absolute_error, r2_score
)
import numpy as np
import pandas as pd

# Load training data
print("\n1️  Loading training data...")
train_df = pd.read_csv(train_data_path)
validation_df = pd.read_csv(validation_data_path)
test_df = pd.read_csv(test_data_path)

print(f"   Training:   {len(train_df):,} records")
print(f"   Validation: {len(validation_df):,} records")
print(f"   Test:       {len(test_df):,} records")

# Separate features and targets
print("\n2️  Preparing features and targets...")

# Classification target (binary)
y_train_class = train_df['is_highly_rated']
y_val_class = validation_df['is_highly_rated']
y_test_class = test_df['is_highly_rated']

# Regression target (actual stars)
y_train_stars = train_df['review_stars']
y_val_stars = validation_df['review_stars']
y_test_stars = test_df['review_stars']

print(f"\n   Classification target distribution:")
print(f"   - Highly rated (1): {y_train_class.sum():,} ({y_train_class.mean()*100:.1f}%)")
print(f"   - Not highly rated (0): {(1-y_train_class).sum():,} ({(1-y_train_class.mean())*100:.1f}%)")

print(f"\n   Star rating distribution:")
print(f"   - Mean: {y_train_stars.mean():.2f}")
print(f"   - Std:  {y_train_stars.std():.2f}")
print(f"   - Range: {y_train_stars.min():.0f} - {y_train_stars.max():.0f}")

def evaluate_model_comprehensive(y_true_class, y_pred_class, y_true_stars, y_pred_stars,
                                   y_pred_proba=None, dataset_name="Dataset"):
    """
    Comprehensive evaluation with both classification and regression metrics.

    Parameters:
    - y_true_class: True binary labels (0/1)
    - y_pred_class: Predicted binary labels (0/1)
    - y_true_stars: True star ratings (1-5)
    - y_pred_stars: Predicted star ratings (1-5)
    - y_pred_proba: Predicted probabilities (optional, for ROC-AUC)
    - dataset_name: Name for display
    """
    results = {}

    # Classification Metrics
    results['accuracy'] = accuracy_score(y_true_class, y_pred_class)
    results['precision'] = precision_score(y_true_class, y_pred_class, zero_division=0)
    results['recall'] = recall_score(y_true_class, y_pred_class, zero_division=0)
    results['f1'] = f1_score(y_true_class, y_pred_class, zero_division=0)

    if y_pred_proba is not None:
        results['roc_auc'] = roc_auc_score(y_true_class, y_pred_proba)
    else:
        results['roc_auc'] = None

    # Regression Metrics
    results['mse'] = mean_squared_error(y_true_stars, y_pred_stars)
    results['rmse'] = np.sqrt(results['mse'])
    results['mae'] = mean_absolute_error(y_true_stars, y_pred_stars)
    results['r2'] = r2_score(y_true_stars, y_pred_stars)

    # Custom Metrics: Within X stars
    abs_error = np.abs(y_true_stars - y_pred_stars)
    results['within_0.5_stars'] = (abs_error <= 0.5).mean()
    results['within_1.0_stars'] = (abs_error <= 1.0).mean()

    return results


def print_results(results, model_name, dataset_name):
    """Pretty print evaluation results."""
    print(f"\n{'='*80}")
    print(f"{model_name} - {dataset_name}")
    print(f"{'='*80}")

    print(f"\n Classification Metrics (Binary: Highly Rated vs Not):")
    print(f"   Accuracy:  {results['accuracy']:.4f}")
    print(f"   Precision: {results['precision']:.4f}")
    print(f"   Recall:    {results['recall']:.4f}")
    print(f"   F1-Score:  {results['f1']:.4f}")
    if results['roc_auc'] is not None:
        print(f"   ROC-AUC:   {results['roc_auc']:.4f}")

    print(f"\n Regression Metrics (Star Rating Prediction):")
    print(f"   MSE:       {results['mse']:.4f}")
    print(f"   RMSE:      {results['rmse']:.4f}")
    print(f"   MAE:       {results['mae']:.4f}")
    print(f"   R²:        {results['r2']:.4f}")

    print(f"\n Accuracy Metrics (Star Prediction):")
    print(f"   Within 0.5 stars: {results['within_0.5_stars']*100:.2f}%")
    print(f"   Within 1.0 stars: {results['within_1.0_stars']*100:.2f}%")



### 6.2 Baseline Model #1: Simple Heuristic

In [ ]:
print("\n" + "="*80)
print("BASELINE MODEL #1: Simple Heuristic")
print("="*80)
print("Approach: Predict highly_rated if avg_review_stars >= 4.0")

# Classification predictions
baseline1_pred_class_train = (train_df['avg_review_stars'] >= 4.0).astype(int)
baseline1_pred_class_val = (validation_df['avg_review_stars'] >= 4.0).astype(int)
baseline1_pred_class_test = (test_df['avg_review_stars'] >= 4.0).astype(int)

# Star predictions (use average directly)
baseline1_pred_stars_train = train_df['avg_review_stars']
baseline1_pred_stars_val = validation_df['avg_review_stars']
baseline1_pred_stars_test = test_df['avg_review_stars']

# Evaluate
baseline1_results_train = evaluate_model_comprehensive(
    y_train_class, baseline1_pred_class_train,
    y_train_stars, baseline1_pred_stars_train,
    y_pred_proba=None, dataset_name="Training"
)

baseline1_results_val = evaluate_model_comprehensive(
    y_val_class, baseline1_pred_class_val,
    y_val_stars, baseline1_pred_stars_val,
    y_pred_proba=None, dataset_name="Validation"
)

baseline1_results_test = evaluate_model_comprehensive(
    y_test_class, baseline1_pred_class_test,
    y_test_stars, baseline1_pred_stars_test,
    y_pred_proba=None, dataset_name="Test"
)

print_results(baseline1_results_train, "Baseline #1: Heuristic", "Training Set")
print_results(baseline1_results_val, "Baseline #1: Heuristic", "Validation Set")
print_results(baseline1_results_test, "Baseline #1: Heuristic", "Test Set")

### 6.3 Baseline Model #2: Linear Regression

In [ ]:

# Select features
baseline2_features = [
    'avg_review_stars',
    'enhanced_parking_score',
    'business_review_count'
]

print(f"Features: {', '.join(baseline2_features)}")

# Prepare feature matrices
X_train = train_df[baseline2_features].fillna(0)
X_val = validation_df[baseline2_features].fillna(0)
X_test = test_df[baseline2_features].fillna(0)

print(f"\n⏳ Training logistic regression...")
baseline2_model = LogisticRegression(random_state=42, max_iter=1000)
baseline2_model.fit(X_train, y_train_class)
print(" Model trained!")

# Classification predictions
baseline2_pred_class_train = baseline2_model.predict(X_train)
baseline2_pred_class_val = baseline2_model.predict(X_val)
baseline2_pred_class_test = baseline2_model.predict(X_test)

# Probabilities
baseline2_prob_train = baseline2_model.predict_proba(X_train)[:, 1]
baseline2_prob_val = baseline2_model.predict_proba(X_val)[:, 1]
baseline2_prob_test = baseline2_model.predict_proba(X_test)[:, 1]

# Convert probabilities to star predictions (1-5 scale)
# Probability 0-1 maps to stars 1-5
def prob_to_stars(prob):
    """Convert probability to star rating (1-5 scale)."""
    return 1 + (prob * 4)  # Maps 0->1, 0.5->3, 1->5

baseline2_pred_stars_train = prob_to_stars(baseline2_prob_train)
baseline2_pred_stars_val = prob_to_stars(baseline2_prob_val)
baseline2_pred_stars_test = prob_to_stars(baseline2_prob_test)

# Evaluate
baseline2_results_train = evaluate_model_comprehensive(
    y_train_class, baseline2_pred_class_train,
    y_train_stars, baseline2_pred_stars_train,
    y_pred_proba=baseline2_prob_train, dataset_name="Training"
)

baseline2_results_val = evaluate_model_comprehensive(
    y_val_class, baseline2_pred_class_val,
    y_val_stars, baseline2_pred_stars_val,
    y_pred_proba=baseline2_prob_val, dataset_name="Validation"
)

baseline2_results_test = evaluate_model_comprehensive(
    y_test_class, baseline2_pred_class_test,
    y_test_stars, baseline2_pred_stars_test,
    y_pred_proba=baseline2_prob_test, dataset_name="Test"
)

print_results(baseline2_results_train, "Baseline #2: Logistic Regression", "Training Set")
print_results(baseline2_results_val, "Baseline #2: Logistic Regression", "Validation Set")
print_results(baseline2_results_test, "Baseline #2: Logistic Regression", "Test Set")

# Feature importance
print(f"\n Feature Importance (Coefficients):")
for feature, coef in zip(baseline2_features, baseline2_model.coef_[0]):
    print(f"   {feature:30s}: {coef:8.4f}")




In [ ]:

# =============================================================================
# COMPREHENSIVE MODEL COMPARISON
# =============================================================================

print("\n" + "="*80)
print("MODEL COMPARISON - VALIDATION SET")
print("="*80)

# Create comparison dataframe
comparison_df = pd.DataFrame({
    'Metric': [
        'Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC',
        'MSE', 'RMSE', 'MAE', 'R²',
        'Within 0.5★', 'Within 1.0★'
    ],
    'Baseline #1 (Heuristic)': [
        baseline1_results_val['accuracy'],
        baseline1_results_val['precision'],
        baseline1_results_val['recall'],
        baseline1_results_val['f1'],
        baseline1_results_val['roc_auc'] if baseline1_results_val['roc_auc'] else 0,
        baseline1_results_val['mse'],
        baseline1_results_val['rmse'],
        baseline1_results_val['mae'],
        baseline1_results_val['r2'],
        baseline1_results_val['within_0.5_stars'],
        baseline1_results_val['within_1.0_stars']
    ],
    'Baseline #2 (LogReg)': [
        baseline2_results_val['accuracy'],
        baseline2_results_val['precision'],
        baseline2_results_val['recall'],
        baseline2_results_val['f1'],
        baseline2_results_val['roc_auc'],
        baseline2_results_val['mse'],
        baseline2_results_val['rmse'],
        baseline2_results_val['mae'],
        baseline2_results_val['r2'],
        baseline2_results_val['within_0.5_stars'],
        baseline2_results_val['within_1.0_stars']
    ]
})

print("\n Validation Set Performance:")
print(comparison_df.to_string(index=False))

# Highlight best scores
print("\n Best Scores (Validation Set):")
print("\nClassification Metrics:")
print(f"   Best Accuracy:  {max(baseline1_results_val['accuracy'], baseline2_results_val['accuracy']):.4f}")
print(f"   Best F1-Score:  {max(baseline1_results_val['f1'], baseline2_results_val['f1']):.4f}")

print("\nRegression Metrics:")
print(f"   Best RMSE:      {min(baseline1_results_val['rmse'], baseline2_results_val['rmse']):.4f}")
print(f"   Best R²:        {max(baseline1_results_val['r2'], baseline2_results_val['r2']):.4f}")

print("\nPrediction Accuracy:")
print(f"   Best Within 0.5★: {max(baseline1_results_val['within_0.5_stars'], baseline2_results_val['within_0.5_stars'])*100:.2f}%")
print(f"   Best Within 1.0★: {max(baseline1_results_val['within_1.0_stars'], baseline2_results_val['within_1.0_stars'])*100:.2f}%")


# =============================================================================
# STORE RESULTS
# =============================================================================

# Store for later comparison with XGBoost
baseline_results = {
    'baseline1': {
        'train': baseline1_results_train,
        'val': baseline1_results_val,
        'test': baseline1_results_test
    },
    'baseline2': {
        'train': baseline2_results_train,
        'val': baseline2_results_val,
        'test': baseline2_results_test
    }
}

%store baseline_results
%store baseline2_features

### 6.4 XGBoost Model Training

In [ ]:

import sagemaker
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
import boto3
import time

# Configuration
print("\n1️  Configuring XGBoost training...")

# Select features for XGBoost (more features than baseline)
xgb_features = [
    # Business features
    'avg_review_stars',
    'std_review_stars',
    'business_review_count',
    'pct_highly_rated',
    # Parking features
    'enhanced_parking_score',
    'parking_positive',
    'parking_negative',
    'parking_sentiment',
    'has_parking_data',
    # Review engagement
    'avg_engagement',
    # Business attributes
    'is_restaurant',
    'price_range_numeric'
]

print(f"   Selected {len(xgb_features)} features for XGBoost")
print(f"\n   Features:")
for i, feat in enumerate(xgb_features, 1):
    print(f"   {i:2d}. {feat}")

# Prepare data for XGBoost (target must be first column, no header)
print("\n2️  Preparing data for XGBoost format...")

def prepare_xgb_data(df, features, target='is_highly_rated'):
    """Prepare data in XGBoost format: target first, no header."""
    # Select features and fill missing values
    X = df[features].fillna(0)
    y = df[target]

    # Combine with target first
    xgb_data = pd.concat([y, X], axis=1)

    return xgb_data

# Prepare datasets
train_xgb = prepare_xgb_data(train_df, xgb_features)
val_xgb = prepare_xgb_data(validation_df, xgb_features)

print(f"   Training data shape: {train_xgb.shape}")
print(f"   Validation data shape: {val_xgb.shape}")

# Save locally
print("\n3️  Saving XGBoost-formatted data...")
train_xgb.to_csv('/tmp/train_xgb.csv', index=False, header=False)
val_xgb.to_csv('/tmp/val_xgb.csv', index=False, header=False)
print("    Data saved in XGBoost format")

# Upload to S3
print("\n4️  Uploading to S3...")
xgb_train_path = f"s3://{MODEL_DIR}xgboost-training/train.csv"
xgb_val_path = f"s3://{MODEL_DIR}xgboost-training/validation.csv"
xgb_output_path = f"s3://{MODEL_DIR}xgboost-output"

bucket = BASE_BUCKET_NAME
s3_client.upload_file('/tmp/train_xgb.csv', bucket, f'{MODEL_PREFIX}xgboost-training/train.csv')
s3_client.upload_file('/tmp/val_xgb.csv', bucket, f'{MODEL_PREFIX}xgboost-training/validation.csv')

print(f"    Training data: {xgb_train_path}")
print(f"    Validation data: {xgb_val_path}")
print(f"    Output path: {xgb_output_path}")

# Get XGBoost container
print("\n5️  Getting XGBoost container...")
from sagemaker.image_uris import retrieve

container = retrieve('xgboost', REGION, version='1.5-1')
print(f"   Container: {container}")

In [ ]:
# Create estimator
print("\n6️  Creating XGBoost estimator...")

xgb_estimator = sagemaker.estimator.Estimator(
    container,
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    output_path=xgb_output_path,
    sagemaker_session=sagemaker_session,
    base_job_name='venuesignal-xgboost'
)

# Set hyperparameters
xgb_estimator.set_hyperparameters(
    objective='binary:logistic',
    num_round=100,
    max_depth=6,
    eta=0.3,
    gamma=0,
    min_child_weight=1,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='auc',
    early_stopping_rounds=10
)

print("    Estimator configured")
print(f"\n   Hyperparameters:")
print(f"   - Objective: binary:logistic")
print(f"   - Num rounds: 100")
print(f"   - Max depth: 6")
print(f"   - Learning rate (eta): 0.3")
print(f"   - Early stopping: 10 rounds")

# Create training input channels
print("\n7️  Creating training input channels...")

train_input = TrainingInput(xgb_train_path, content_type='text/csv')
val_input = TrainingInput(xgb_val_path, content_type='text/csv')

# Train model
print("\n8️  Starting XGBoost training...")
print(f"   Training on {len(train_xgb):,} records")
print(f"   Validating on {len(val_xgb):,} records")

xgb_estimator.fit({
    'train': train_input,
    'validation': val_input
})

print("\n XGBoost training complete!")
print(f"   Model artifacts: {xgb_estimator.model_data}")

# Store model data path
xgb_model_data = xgb_estimator.model_data
%store xgb_model_data
%store xgb_features

In [ ]:
# Section 6.3: XGBoost Evaluation

print("\n" + "="*80)
print("SECTION 6.3: XGBOOST MODEL EVALUATION")
print("="*80)

# For evaluation, we need to make predictions locally
# Download the model and make predictions

print("\n Making predictions with trained XGBoost model...")
print("   (Note: In production, this would use SageMaker endpoint)")

# Alternative: Use local XGBoost with same hyperparameters
import xgboost as xgb

print("\n1️  Training local XGBoost for evaluation...")

# Prepare data for local XGBoost
X_train_xgb = train_df[xgb_features].fillna(0)
X_val_xgb = validation_df[xgb_features].fillna(0)
X_test_xgb = test_df[xgb_features].fillna(0)

# Train local model with same hyperparameters
xgb_local = xgb.XGBClassifier(
    objective='binary:logistic',
    n_estimators=100,
    max_depth=6,
    learning_rate=0.3,
    gamma=0,
    min_child_weight=1,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='auc',
    early_stopping_rounds=10,
    random_state=42
)

xgb_local.fit(
    X_train_xgb, y_train_class,
    eval_set=[(X_val_xgb, y_val_class)],
    verbose=False
)

print(" Local XGBoost model trained")

# Make predictions
print("\n2️  Making predictions...")

# Classification predictions
xgb_pred_class_train = xgb_local.predict(X_train_xgb)
xgb_pred_class_val = xgb_local.predict(X_val_xgb)
xgb_pred_class_test = xgb_local.predict(X_test_xgb)

# Probabilities
xgb_prob_train = xgb_local.predict_proba(X_train_xgb)[:, 1]
xgb_prob_val = xgb_local.predict_proba(X_val_xgb)[:, 1]
xgb_prob_test = xgb_local.predict_proba(X_test_xgb)[:, 1]

# Convert to star predictions
xgb_pred_stars_train = prob_to_stars(xgb_prob_train)
xgb_pred_stars_val = prob_to_stars(xgb_prob_val)
xgb_pred_stars_test = prob_to_stars(xgb_prob_test)

print(" Predictions complete")

# Comprehensive evaluation
print("\n3️  Evaluating XGBoost model...")

xgb_results_train = evaluate_model_comprehensive(
    y_train_class, xgb_pred_class_train,
    y_train_stars, xgb_pred_stars_train,
    y_pred_proba=xgb_prob_train, dataset_name="Training"
)

xgb_results_val = evaluate_model_comprehensive(
    y_val_class, xgb_pred_class_val,
    y_val_stars, xgb_pred_stars_val,
    y_pred_proba=xgb_prob_val, dataset_name="Validation"
)

xgb_results_test = evaluate_model_comprehensive(
    y_test_class, xgb_pred_class_test,
    y_test_stars, xgb_pred_stars_test,
    y_pred_proba=xgb_prob_test, dataset_name="Test"
)

print_results(xgb_results_train, "XGBoost Model", "Training Set")
print_results(xgb_results_val, "XGBoost Model", "Validation Set")
print_results(xgb_results_test, "XGBoost Model", "Test Set")

# Feature importance
print("\n" + "="*80)
print("XGBOOST FEATURE IMPORTANCE")
print("="*80)

importance_dict = dict(zip(xgb_features, xgb_local.feature_importances_))
importance_sorted = sorted(importance_dict.items(), key=lambda x: x[1], reverse=True)

print("\n Top 10 Most Important Features:")
for i, (feature, importance) in enumerate(importance_sorted[:10], 1):
    bar_length = int(importance * 50)
    bar = "█" * bar_length
    print(f"   {i:2d}. {feature:30s} {bar} {importance:.4f}")


---

## 7. Model Deployment <a id='section-7'></a>

This section:
- Registers the XGBoost model in SageMaker Model Registry
- Creates a SageMaker endpoint for real-time inference
- Tests the deployed model

**Deployment Strategy**: Real-time endpoint for individual predictions

In [ ]:
# Standard libraries
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from time import sleep, gmtime, strftime, time

import warnings
warnings.filterwarnings('ignore')

#sklearn
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from sklearn.metrics import f1_score, precision_recall_curve, auc, precision_score, recall_score

# Sagemaker imports
import boto3
import sagemaker

# Initialize clients
sm = boto3.client('sagemaker', region_name=REGION)
sagemaker_runtime = boto3.client('sagemaker-runtime', region_name=REGION)

### Section 7.1 Prepare Model Metadata

In [ ]:
#@title Section 7.1 Prepare Model Metadata
# Get training job details
model_name = xgb_estimator.latest_training_job.name
print(f"\n📋 Model Information:")
print(f"   Training job name: {model_name}")

# Get model artifacts
info = sm.describe_training_job(TrainingJobName=model_name)
model_data = info['ModelArtifacts']['S3ModelArtifacts']
print(f"   Model artifacts: {model_data}")

# Get XGBoost container image
from sagemaker.image_uris import retrieve
image = retrieve('xgboost', REGION, version='1.7-1')
print(f"   Container image: {image}")

# Datacapture URI
data_capture_prefix = f"{MODEL_PREFIX}datacapture"
s3_capture_upload_path = f"s3://{BASE_BUCKET_NAME}/{data_capture_prefix}"

print(f"   S3 Datacapture URI: {s3_capture_upload_path}")

### Section 7.2: Create SageMaker Model

In [ ]:
#@title Section 7.2: Create SageMaker Model
print(f"\n Creating SageMaker model: {model_name}")

# Define primary container
primary_container = {
    'Image': image,
    'ModelDataUrl': model_data
}

try:
    # Create model in SageMaker Model Registry
    create_model_response = sm.create_model(
        ModelName=model_name,
        ExecutionRoleArn=role,
        PrimaryContainer=primary_container
    )

    print(f" Model created successfully!")
    print(f"   Model ARN: {create_model_response['ModelArn']}")
    model_created = True

except Exception as e:
    error_msg = str(e)
    if 'already exists' in error_msg.lower():
        print(f"  Model already exists, will use existing model")
        model_created = True
    else:
        print(f" Error creating model: {error_msg[:200]}")
        model_created = False

### Section 7.3: Create the custom inference script

In [ ]:
import os
os.makedirs("/tmp/venuesignal_model", exist_ok=True)

inference_script = '''
import os
import numpy as np
import xgboost as xgb

def model_fn(model_dir):
    model_file = os.path.join(model_dir, "xgboost-model")
    model = xgb.Booster()
    model.load_model(model_file)
    return model

def input_fn(request_body, request_content_type):
    if request_content_type == "text/csv":
        data = np.array([
            [float(x) for x in row.split(",")]
            for row in request_body.strip().split("\\n")
            if row.strip()
        ])
        return xgb.DMatrix(data)
    raise ValueError(f"Unsupported content type: {request_content_type}")

def predict_fn(input_data, model):
    return model.predict(input_data)

def output_fn(predictions, accept):
    """
    Return TWO columns per row: probability,predicted_class
    - _c0 = probability  (float)
    - _c1 = predicted class (0 or 1)
    This prevents the Model Quality Monitor Spark job from
    treating the single probability column as the join ID,
    which causes endpointOutput_0 to not exist.
    """
    lines = []
    for p in predictions:
        prob = float(p)
        pred_class = 1 if prob >= 0.5 else 0
        lines.append(f"{prob},{pred_class}")
    return "\\n".join(lines), "text/csv"
'''

script_path = "/tmp/venuesignal_model/inference.py"
with open(script_path, "w") as f:
    f.write(inference_script)

print(f"✓ Inference script written: {script_path}")
print("  output_fn now returns: probability,predicted_class (2 columns)")
print("  _c0 = probability, _c1 = predicted class")

In [ ]:
# variant_name will be set after deployment in Cell 85
# hardcoding here is a placeholder only
variant_name = 'venuesignal-xgboost'
print(f"Placeholder variant name: {variant_name}")
print("Note: actual variant name will be confirmed after deployment")

### Section 7.4: Deploy Model to Real-Time Endpoint

In [ ]:
# Delete old endpoint first
try:
    sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
    print(f"✓ Deleting old endpoint: {endpoint_name}")
    waiter = sagemaker_client.get_waiter("endpoint_deleted")
    waiter.wait(EndpointName=endpoint_name)
    print("✓ Endpoint deleted")
except Exception as e:
    print(f"Note: {e}")

from sagemaker.xgboost import XGBoostModel

script_s3_uri = f"s3://{BASE_BUCKET_NAME}/{MODEL_PREFIX}code/"
S3Uploader.upload("/tmp/venuesignal_model/inference.py", script_s3_uri)
print(f"\n✓ Inference script uploaded to: {script_s3_uri}")

print("\nDeploying model with custom inference script...")
print("This takes ~5-10 minutes...")

xgb_model = XGBoostModel(
    model_data=xgb_model_data,
    role=role,
    entry_point="inference.py",
    source_dir="/tmp/venuesignal_model",
    framework_version="1.7-1",
    sagemaker_session=sagemaker_session,
)

endpoint_name = (
    f"venuesignal-endpoint-{datetime.now(timezone.utc):%Y%m%d-%H%M%S}"
)

predictor = xgb_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name=endpoint_name,
    data_capture_config=sagemaker.model_monitor.DataCaptureConfig(
        enable_capture=True,
        sampling_percentage=100,
        destination_s3_uri=s3_capture_upload_path,
        capture_options=["Input", "Output"],
        csv_content_types=["text/csv"],
    ),
)

print(f"✓ Endpoint deployed: {endpoint_name}")
%store endpoint_name

print("\nSending initial traffic...")
runtime = boto3.client("sagemaker-runtime", region_name=REGION)
for i in range(20):
    payload = (
        production_df[xgb_features]
        .iloc[i % len(production_df)]
        .fillna(0)
        .to_csv(header=None, index=False)
        .strip()
    )
    runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="text/csv",
        Body=payload,
        InferenceId=str(i),
    )
    sleep(1)
print("✓ 20 requests sent — endpoint ready")

endpoint_deployed = True
# Override variant_name with actual deployed value
ep_desc = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
variant_name = ep_desc["ProductionVariants"][0]["VariantName"]
print(f"✓ Actual variant name: {variant_name}")
%store endpoint_name
%store variant_name

### Section 7.5: Test Endpoint

In [ ]:
#@title Section 7.5: Test Endpoint
if endpoint_deployed:
    print("\n" + "="*80)
    print("SECTION 7.5: TEST ENDPOINT")
    print("="*80)

    print("\n Testing endpoint with sample predictions...")

    # Prepare test sample (features only, no header)
    test_sample = test_df[xgb_features].head(10).fillna(0)
    test_csv = test_sample.to_csv(header=None, index=False).strip('\n').split('\n')

    try:
        # Test with first sample
        invoke_endpoint_response = sagemaker_runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='text/csv',
            Body=test_csv[0]
        )

        prediction = invoke_endpoint_response['Body'].read().decode('utf-8')
        print(f" Endpoint responding successfully!")
        print(f"   Sample prediction: {prediction}")

        # Test with multiple samples
        print(f"\n Testing with {len(test_csv)} samples...")

        body = ""
        for row in test_csv:
            body += row + "\n"

        response = sagemaker_runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='text/csv',
            Body=body
        )

        predictions_str = response['Body'].read().decode('utf-8')
        predictions = [float(val) for val in predictions_str.strip().split("\n") if val.strip()]

        print(f" Received {len(predictions)} predictions")
        print(f"\n   Sample predictions:")
        for i, pred in enumerate(predictions[:5]):
            print(f"   Sample {i+1}: {pred:.4f} (probability)")

        # Store endpoint info
        %store endpoint_name

        print(f"\n Endpoint deployed and tested successfully!")

    except Exception as e:
        print(f" Error invoking endpoint: {e}")

### Section 7.6: Full Test Set Evaluation

In [ ]:
#@title Section 7.6: Full Test Set Evaluation

print("\n" + "="*80)
print("SECTION 7.6: FULL TEST SET EVALUATION")
print("="*80)

# Load test data
X_test_eval = test_df[xgb_features].fillna(0)
y_test_eval = test_df['is_highly_rated']
y_test_stars = test_df['review_stars']

if endpoint_deployed:
    print("\n Generating predictions using deployed endpoint...")

    try:
        # Prepare test data
        test_csv_full = X_test_eval.to_csv(header=None, index=False).strip('\n').split('\n')

        # Create request body
        body = ""
        for row in test_csv_full:
            body += row + "\n"

        # Invoke endpoint
        response = sagemaker_runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='text/csv',
            Body=body
        )

        # Parse predictions
        predictions_str = response['Body'].read().decode('utf-8')
        predictions_proba = np.array([float(val) for val in predictions_str.split().split("\n") if val.strip()])
        predictions_raw = predictions_str.strip().split("\n")
        predictions_class = [float(val) for val in predictions_raw if val.strip()]
        predictions_stars = prob_to_stars(predictions_proba)


        print(f" Generated {len(predictions_proba):,} predictions from endpoint")
        prediction_source = "SageMaker Endpoint"

    except Exception as e:
        print(f"  Endpoint invocation failed: {e}")
        print("   Falling back to local predictions...")

        predictions_class = xgb_local.predict(X_test_eval)
        predictions_proba = xgb_local.predict_proba(X_test_eval)[:, 1]
        predictions_stars = prob_to_stars(predictions_proba)
        prediction_source = "Local Model"
else:
    print("\n Generating predictions using local model...")

    predictions_class = xgb_local.predict(X_test_eval)
    predictions_proba = xgb_local.predict_proba(X_test_eval)[:, 1]
    predictions_stars = prob_to_stars(predictions_proba)

    print(f" Generated {len(predictions_proba):,} predictions locally")
    prediction_source = "Local Model"

# Evaluate
print(f"\n Test Set Performance (using {prediction_source}):")

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error

accuracy = accuracy_score(y_test_eval, predictions_class)
precision = precision_score(y_test_eval, predictions_class)
recall = recall_score(y_test_eval, predictions_class)
f1 = f1_score(y_test_eval, predictions_class)
rmse = np.sqrt(mean_squared_error(y_test_stars, predictions_stars))
mae = np.mean(np.abs(y_test_stars - predictions_stars))
within_05 = (np.abs(y_test_stars - predictions_stars) <= 0.5).mean()
within_10 = (np.abs(y_test_stars - predictions_stars) <= 1.0).mean()

print(f"\n   Classification Metrics:")
print(f"   Accuracy:  {accuracy:.4f}")
print(f"   Precision: {precision:.4f}")
print(f"   Recall:    {recall:.4f}")
print(f"   F1-Score:  {f1:.4f}")

print(f"\n   Regression Metrics:")
print(f"   RMSE:      {rmse:.4f} stars")
print(f"   MAE:       {mae:.4f} stars")

print(f"\n   Prediction Accuracy:")
print(f"   Within 0.5 star: {within_05*100:.2f}%")
print(f"   Within 1.0 star: {within_10*100:.2f}%")

# Save predictions
print(f"\n Saving predictions...")

predictions_df = pd.DataFrame({
    'business_id': test_df['business_id'],
    'review_id': test_df['review_id'],
    'actual_stars': y_test_stars,
    'is_highly_rated': y_test_class,
    'predicted_stars': predictions_stars,
    'predicted_class': predictions_class,
    'probability': predictions_proba,
    'error': np.abs(y_test_stars - predictions_stars),
    'within_1_star': np.abs(y_test_stars - predictions_stars) <= 1.0
})

# Save locally and to S3
predictions_local = '/tmp/test_predictions.csv'
predictions_df.to_csv(predictions_local, index=False)

predictions_s3 = f"s3://{MODEL_DIR}predictions/test_predictions.csv"
s3_client.upload_file(predictions_local, BASE_BUCKET_NAME, f'{MODEL_PREFIX}predictions/test_predictions.csv')

print(f"    Saved to: {predictions_s3}")

# Display sample
print(f"\n Sample Predictions:")
display(predictions_df.head(10))


### Section 7.7: Create Model Package Group (Model Registry)

In [ ]:
model_package_group_name = f"venuesignal-model-group-{account_id}"
model_description = "VenueSignal model package group: predicts business ratings based on parking features"

print(f"\n Creating model package group...")
print(f"   Name: {model_package_group_name}")

In [ ]:
#@title Section 7.7: Create Model Package Group (Model Registry)

print("\n" + "="*80)
print("SECTION 7.7: CREATE MODEL PACKAGE GROUP")
print("="*80)

try:
    model_package_group_input_dict = {
        'ModelPackageGroupName': model_package_group_name,
        'ModelPackageGroupDescription': model_description
    }

    create_model_package_group_response = sm.create_model_package_group(
        **model_package_group_input_dict
    )

    print(f" Model package group created!")
    print(f"   ARN: {create_model_package_group_response['ModelPackageGroupArn']}")

    model_package_group_created = True

except Exception as e:
    if 'already exists' in str(e).lower():
        print(f"  Model package group already exists")
        model_package_group_created = True
    else:
        print(f" Error: {e}")
        model_package_group_created = False

### Section 7.8: Register Model Version

In [ ]:
#@title Section 7.8: Register Model Version

if model_package_group_created:
    print("\n" + "="*80)
    print("SECTION 7.8: REGISTER MODEL VERSION")
    print("="*80)

    print(f"\n Registering model version to package group...")

    try:
        # Define inference specification
        modelpackage_inference_specification = {
            'InferenceSpecification': {
                'Containers': [
                    {
                        'Image': image,
                        'ModelDataUrl': info['ModelArtifacts']['S3ModelArtifacts'],
                    }
                ],
                'SupportedContentTypes': ['text/csv'],
                'SupportedResponseMIMETypes': ['text/csv'],
                'SupportedTransformInstanceTypes': ['ml.m5.xlarge'],
            }
        }

        # Create model package input
        create_model_package_input_dict = {
            'ModelPackageGroupName': model_package_group_name,
            'ModelPackageDescription': model_description,
            'ModelApprovalStatus': 'Approved'
        }
        create_model_package_input_dict.update(modelpackage_inference_specification)

        # Register model
        create_model_package_response = sm.create_model_package(
            **create_model_package_input_dict
        )

        model_package_arn = create_model_package_response['ModelPackageArn']
        print(f" Model version registered!")
        print(f"   Model Package ARN: {model_package_arn}")

        # Store for later use
        %store model_package_group_name
        %store model_package_arn

    except Exception as e:
        print(f" Error registering model: {e}")

### Section 7.9: Deployment Summary

In [ ]:
print("\n" + "="*80)
print("DEPLOYMENT SUMMARY")
print("="*80)

print(f"\n Deployment Status:")

print(f"\n1. SageMaker Model:")
if model_created:
    print(f"   Created: {model_name}")
    print(f"   Location: {model_data}")
else:
    print(f"    Not created")

print(f"\n2. Endpoint Configuration:")
print(f"   Created automatically by XGBoostModel.deploy()")
print(f"   Instance type: ml.m5.large")

print(f"\n3. Endpoint:")
if endpoint_deployed:
    print(f"   Created: {endpoint_name}")
    print(f"   Variant: {variant_name}")
    print(f"   Status: InService")
else:
    print(f"    Not deployed")

print(f"\n4. Data Capture:")
print(f"   Enabled: True")
print(f"   Path: {s3_capture_upload_path}")
print(f"   Content type: text/csv (no BASE64)")

print(f"\n5. Model Registry:")
if 'model_package_group_name' in dir():
    print(f"   Group: {model_package_group_name}")
else:
    print(f"   Not registered")

print("\n" + "="*80)
print(" SECTION 7 COMPLETE - MODEL DEPLOYMENT")
print("="*80)

## 8. Monitoring & Observability <a id='section-8'></a>

This section implements comprehensive production monitoring across three pillars:

1. **Model Quality Monitoring** (Section 8.2): Tracks prediction accuracy and classification drift against a validated baseline
2. **Data Quality Monitoring** (Section 8.3): Detects feature distribution shifts in incoming data
3. **Infrastructure Monitoring** (Section 8.4): Monitors endpoint health, latency, and resource utilization
4. **CloudWatch Dashboard** (Section 8.5): Centralized visualization of all monitoring signals
5. **Monitoring Reports** (Section 8.6): Automated report generation and violation review

All monitors are baselined, scheduled to run **hourly**, and publish metrics to CloudWatch.


### 8.1 Configure Monitoring

In [ ]:
# ── Imports ────────────────────────────────────────────────────────────────
import copy
import json
import random
import io
import csv
import time
import uuid
import numpy as np
import pandas as pd
import boto3
import sagemaker

from threading import Thread
from datetime import datetime, timedelta, timezone
from time import sleep

from sagemaker import get_execution_role, image_uris, Session
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.model_monitor import (
    DefaultModelMonitor,
    ModelQualityMonitor,
    CronExpressionGenerator,
    DataCaptureConfig,
    EndpointInput,
)
from sagemaker.model_monitor.dataset_format import DatasetFormat

# ── Core references (already defined in earlier sections) ─────────────────
# Confirm key variables are available
print("Confirming monitoring configuration...")
print(f"  Endpoint     : {endpoint_name}")
print(f"  Variant      : {variant_name}")
print(f"  Bucket       : {BASE_BUCKET_NAME}")
print(f"  Monitoring   : s3://{MONITORING_DIR}")
print(f"  Region       : {REGION}")
print(f"  Role         : {role[:50]}...")

# ── S3 paths for all monitoring artefacts ──────────────────────────────────
monitoring_output_path      = f"s3://{MONITORING_DIR}monitoring-output"
baseline_results_path       = f"s3://{MONITORING_DIR}baseline-results"
monitoring_reports_path     = f"s3://{MONITORING_DIR}reports"
reports_uri                 = monitoring_reports_path

# Model Quality paths
mq_baseline_data_uri    = f"s3://{BASE_BUCKET_NAME}/{MONITORING_PREFIX}baselining/data"
mq_baseline_results_uri = f"s3://{BASE_BUCKET_NAME}/{MONITORING_PREFIX}baselining/results"
mq_results_uri          = f"s3://{BASE_BUCKET_NAME}/{MONITORING_PREFIX}model-quality-results"
ground_truth_upload_path = f"s3://{MONITORING_DIR}ground_truth_data"

# Data Quality paths
dq_baseline_uri  = f"s3://{MONITORING_DIR}data-quality-baseline"
dq_results_uri   = f"s3://{MONITORING_DIR}data-quality-results"

# ── Retrieve the model monitor image ──────────────────────────────────────
monitor_image_uri = image_uris.retrieve(framework="model-monitor", region=REGION)

print("\n✓ Monitoring paths configured")
print(f"  MQ baseline data   : {mq_baseline_data_uri}")
print(f"  MQ baseline results: {mq_baseline_results_uri}")
print(f"  MQ results         : {mq_results_uri}")
print(f"  Ground truth       : {ground_truth_upload_path}")
print(f"  DQ baseline        : {dq_baseline_uri}")
print(f"  DQ results         : {dq_results_uri}")
print(f"  Reports            : {monitoring_reports_path}")
print(f"  Monitor image      : {monitor_image_uri}")

%store monitoring_output_path
%store monitoring_reports_path
%store ground_truth_upload_path
%store mq_results_uri
%store dq_results_uri
%store reports_uri


### 8.2 Model Quality Monitoring

Model Quality Monitoring continuously evaluates classification performance
(accuracy, precision, recall, F1, AUC) of the live endpoint against a
baseline derived from held-out validation predictions.

**Process:**
1. Upload a baseline dataset of validation predictions
2. Run a baselining job to compute statistics and suggest constraints
3. Stream live traffic through the endpoint (data capture)
4. Upload synthetic ground-truth labels
5. Schedule an hourly monitoring job that merges captured inferences with
   ground truth and flags constraint violations


#### 8.2.1 Upload Baseline Dataset

In [ ]:
# predictions_local was written in Section 7.6 (test_predictions.csv)
# It contains: review_id, probability, predicted_class, is_highly_rated
print(f"Uploading baseline dataset from: {predictions_local}")
baseline_dataset_uri = S3Uploader.upload(predictions_local, mq_baseline_data_uri)
print(f"✓ Baseline dataset uploaded: {baseline_dataset_uri}")


#### 8.2.2 Create Model Quality Monitor & Baseline Job

In [ ]:
# Instantiate the ModelQualityMonitor
xgboost_model_quality_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=sagemaker_session,
)
print("✓ ModelQualityMonitor created")


In [ ]:
%%time
# Run the baseline suggestion job.
# Problem type is BinaryClassification; column names match test_predictions.csv
mq_baseline_job_name = (
    f"venuesignal-mq-baseline-{account_id}-"
    f"{datetime.now(timezone.utc):%Y%m%d-%H%M%S}"
)
print(f"Creating model quality baseline: {mq_baseline_job_name}")
print("This will take approximately 10-15 minutes…")

xgboost_model_quality_monitor.suggest_baseline(
    job_name=mq_baseline_job_name,
    baseline_dataset=baseline_dataset_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=mq_baseline_results_uri,
    problem_type="BinaryClassification",
    inference_attribute="predicted_class",
    probability_attribute="probability",
    ground_truth_attribute="is_highly_rated",
    wait=True,
    logs=False,
)
print(f"\n✓ Baseline job complete: {mq_baseline_job_name}")
%store mq_baseline_job_name


#### 8.2.3 Review Baseline Results

In [ ]:
mq_baseline_job = xgboost_model_quality_monitor.latest_baselining_job

# ── Statistics ─────────────────────────────────────────────────────────────
print("=" * 70)
print("MODEL QUALITY BASELINE — STATISTICS")
print("=" * 70)

try:
    binary_metrics = mq_baseline_job.baseline_statistics().body_dict[
        "binary_classification_metrics"
    ]
    import pandas as pd
    print(pd.json_normalize(binary_metrics).T.to_string())
except Exception as e:
    print(f"Warning: could not retrieve statistics — {e}")

# ── Constraints ─────────────────────────────────────────────────────────────
print("\n" + "=" * 70)
print("MODEL QUALITY BASELINE — SUGGESTED CONSTRAINTS")
print("=" * 70)
try:
    constraints = mq_baseline_job.suggested_constraints().body_dict[
        "binary_classification_constraints"
    ]
    print(pd.DataFrame(constraints).T.to_string())
except Exception as e:
    print(f"Warning: could not retrieve constraints — {e}")


#### 8.2.4 Generate Live Traffic for Data Capture

In [ ]:
# Start a background thread that continuously invokes the endpoint so that
# SageMaker captures input/output pairs.  The thread runs indefinitely;
# restart the kernel to stop it.

def _invoke_endpoint_loop(ep_name, feature_df):
    runtime = boto3.client("sagemaker-runtime", region_name=REGION)
    ids = list(range(500))  # must match ground truth IDs exactly (0-499)
    while True:
        try:
            for i in ids:
                row = feature_df.iloc[i % len(feature_df)]
                buf = io.StringIO()
                csv.writer(buf).writerow(row.fillna(0).tolist())
                runtime.invoke_endpoint(
                    EndpointName=ep_name,
                    ContentType="text/csv",
                    Body=buf.getvalue().strip(),
                    InferenceId=str(i),  # 0-499 matches ground truth
                )
                sleep(2)
        except Exception:
            pass  # retry on transient errors

traffic_thread = Thread(
    target=_invoke_endpoint_loop,
    args=(endpoint_name, production_df[xgb_features]),
    daemon=True,
)
traffic_thread.start()
print("✓ Endpoint traffic thread started (daemon — stops with kernel)")

#### 8.2.5 Verify Captured Data

In [ ]:
print("Waiting for capture files to appear in S3…", end="")
capture_files = []
for _ in range(120):
    capture_files = sorted(
        S3Downloader.list(f"{s3_capture_upload_path}/{endpoint_name}")
    )
    if capture_files:
        sample_lines = S3Downloader.read_file(capture_files[-1]).split("\n")
        sample_record = json.loads(sample_lines[0])
        if "inferenceId" in sample_record.get("eventMetadata", {}):
            break
    print(".", end="", flush=True)
    sleep(1)
print()

if capture_files:
    print(f"✓ Found {len(capture_files)} capture file(s)")
    print("\nLatest capture files:")
    for f in capture_files[-3:]:
        print(f"  {f}")
    print("\nSample capture record (first event):")
    print(json.dumps(sample_record, indent=2)[:800])
else:
    print("⚠ No capture files found yet — ensure data capture is enabled on the endpoint")


#### 8.2.6 Generate Synthetic Ground Truth

In [ ]:
# In production, ground truth arrives asynchronously (actual outcomes).
# Here we simulate it with random labels (70 % positive) for demonstration.

def _ground_truth_record(inference_id):
    random.seed(inference_id)
    return {
        "groundTruthData": {
            "data": "1" if random.random() < 0.7 else "0",
            "encoding": "CSV",
        },
        "eventMetadata": {"eventId": str(inference_id)},
        "eventVersion": "0",
    }

def _upload_ground_truth(records, upload_time):
    body = "\n".join(json.dumps(r) for r in records)
    target = f"{ground_truth_upload_path}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"
    S3Uploader.upload_string_as_file_body(body, target)
    print(f"  Uploaded {len(records)} ground-truth records → {target}")

def _ground_truth_loop(n=500):
    while True:
        records = [_ground_truth_record(i) for i in range(n)]
        _upload_ground_truth(records, datetime.utcnow())
        sleep(3600)  # re-upload once per hour

gt_thread = Thread(target=_ground_truth_loop, daemon=True)
gt_thread.start()

# Upload one batch immediately so the first monitoring job has data
_upload_ground_truth([_ground_truth_record(i) for i in range(500)], datetime.utcnow())
print("\n✓ Ground-truth thread started; initial batch uploaded")


#### 8.2.7 Create Hourly Model Quality Monitoring Schedule

In [ ]:
# Delete old schedule
try:
    sagemaker_client.delete_monitoring_schedule(
        MonitoringScheduleName=mq_schedule_name
    )
    print(f"✓ Deleted old schedule: {mq_schedule_name}")
    time.sleep(15)
except Exception as e:
    print(f"Note: {e}")

# Now output has 2 columns: _c0=probability, _c1=predicted_class
mq_endpoint_input = EndpointInput(
    endpoint_name=endpoint_name,
    probability_attribute="0",     # _c0 = probability value
    inference_attribute="1",       # _c1 = predicted class (0 or 1)
    probability_threshold_attribute=0.5,
    destination="/opt/ml/processing/input_data",
)

mq_schedule_name = (
    f"venuesignal-mq-schedule-{account_id}-"
    f"{datetime.now(timezone.utc):%Y%m%d-%H%M%S}"
)
print(f"Creating model quality schedule: {mq_schedule_name}")

try:
    xgboost_model_quality_monitor.create_monitoring_schedule(
        monitor_schedule_name=mq_schedule_name,
        endpoint_input=mq_endpoint_input,
        output_s3_uri=mq_results_uri,
        problem_type="BinaryClassification",
        ground_truth_input=ground_truth_upload_path,
        constraints=mq_baseline_job.suggested_constraints(),
        schedule_cron_expression=CronExpressionGenerator.hourly(),
        enable_cloudwatch_metrics=True,
    )
    print(f"✓ Model quality schedule created: {mq_schedule_name}")
    print(f"  probability_attribute : '0' → endpointOutput_0 (probability)")
    print(f"  inference_attribute   : '1' → endpointOutput_1 (predicted class)")
    %store mq_schedule_name
except Exception as e:
    print(f"✗ Error: {e}")
    raise

#### 8.2.8 Verify Model Quality Schedule

In [ ]:
try:
    desc = sagemaker_client.describe_monitoring_schedule(
        MonitoringScheduleName=mq_schedule_name
    )
    status = desc["MonitoringScheduleStatus"]
    print(f"Schedule : {mq_schedule_name}")
    print(f"Status   : {status}")
    if status == "Scheduled":
        print("✓ Model quality schedule is active and running hourly")
    elif "FailureReason" in desc:
        print(f"✗ Failure: {desc['FailureReason']}")
except Exception as e:
    print(f"✗ Could not describe schedule: {e}")


---

### 8.3 Data Quality Monitoring

Data Quality Monitoring detects feature-level drift between the baseline
training distribution and incoming inference data.  It monitors statistics
(mean, standard deviation, missing-value rates, etc.) for every feature
and raises violations when values exceed the baselined thresholds.


#### 8.3.1 Verify Data Capture is Enabled on the Endpoint

In [ ]:
# Data capture must be enabled on the endpoint config; it was configured
# in Section 7.3.  This cell verifies the configuration.
try:
    ep_desc = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    config_name = ep_desc["EndpointConfigName"]
    config_desc = sagemaker_client.describe_endpoint_config(
        EndpointConfigName=config_name
    )
    if "DataCaptureConfig" in config_desc:
        dc = config_desc["DataCaptureConfig"]
        print(f"✓ Data capture is enabled on endpoint: {endpoint_name}")
        print(f"  Capture destination : {dc.get('DestinationS3Uri', 'N/A')}")
        print(f"  Capture percentage  : {dc.get('InitialSamplingPercentage', 100)}%")
        data_capture_enabled = True
    else:
        print("⚠ Data capture is NOT enabled on this endpoint configuration.")
        print("  Ensure Section 7.3 was executed with DataCaptureConfig.")
        data_capture_enabled = False
except Exception as e:
    print(f"Error checking endpoint config: {e}")
    data_capture_enabled = False


#### 8.3.2 Configure Data Quality Paths

In [ ]:
# training_data_uri was defined in Section 6 and stored via %store
# It points to the training CSV used to fit the XGBoost model.
print(f"Training data URI : {train_data_path}")
print(f"DQ baseline URI   : {dq_baseline_uri}")
print(f"DQ results URI    : {dq_results_uri}")

# Verify the training CSV is accessible
try:
    bucket = train_data_path.split('/')[2]
    key    = '/'.join(train_data_path.split('/')[3:])
    s3_client.head_object(Bucket=bucket, Key=key)
    print("✓ Training data verified in S3")
except Exception as e:
    print(f"⚠ Training data not found — {e}")
    print("  Update train_data_path if the file has moved.")

%store dq_baseline_uri
%store dq_results_uri


#### 8.3.3 Create Data Quality Monitor & Baseline Job

In [ ]:
data_quality_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=sagemaker_session,  # use the module-level session object
)
print("✓ DefaultModelMonitor created")


In [ ]:
%%time
# Rebuild baseline using ONLY the xgb_features columns —
# these are the exact columns the endpoint receives at inference time

import pandas as pd
from datetime import datetime, timezone

# Load training data and keep only the feature columns
train_features_only = pd.read_csv(train_data_path)

# Keep only the columns the endpoint actually sees
missing = [f for f in xgb_features if f not in train_features_only.columns]
if missing:
    print(f"⚠ Missing features in training CSV: {missing}")
else:
    train_features_only = train_features_only[xgb_features]
    print(f"✓ Keeping {len(xgb_features)} feature columns (matching endpoint input)")

# Save the features-only CSV to a new S3 path
features_baseline_local = "/tmp/train_features_only.csv"
train_features_only.to_csv(features_baseline_local, index=False)

features_baseline_s3 = (
    f"s3://{BASE_BUCKET_NAME}/{MONITORING_PREFIX}dq-baseline-input/train_features_only.csv"
)
s3_client.upload_file(
    features_baseline_local,
    BASE_BUCKET_NAME,
    f"{MONITORING_PREFIX}dq-baseline-input/train_features_only.csv",
)
print(f"✓ Features-only baseline data uploaded: {features_baseline_s3}")
print(f"  Columns  : {len(xgb_features)}")
print(f"  Rows     : {len(train_features_only):,}")

# Delete old baseline results so there's no confusion
try:
    old_files = S3Downloader.list(dq_baseline_uri)
    for f in old_files:
        key = f.replace(f"s3://{BASE_BUCKET_NAME}/", "")
        s3_client.delete_object(Bucket=BASE_BUCKET_NAME, Key=key)
    print(f"✓ Cleared old baseline from: {dq_baseline_uri}")
except Exception as e:
    print(f"Note: {e}")

# Delete and recreate the data quality monitor to clear cached baseline
data_quality_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=sagemaker_session,
)


In [ ]:
%%time
dq_baseline_job_name = (
    f"venuesignal-dq-baseline-{account_id}-"
    f"{datetime.now(timezone.utc):%Y%m%d-%H%M%S}"
)
print(f"Creating data quality baseline: {dq_baseline_job_name}")
print("This will take approximately 15-20 minutes…")

data_quality_monitor.suggest_baseline(
    job_name=dq_baseline_job_name,
    baseline_dataset=features_baseline_s3,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=dq_baseline_uri,
    wait=True,
    logs=False,
)
print(f"\n✓ Data quality baseline job complete: {dq_baseline_job_name}")
%store dq_baseline_job_name


#### 8.3.4 Review Data Quality Baseline

In [ ]:
print("=" * 70)
print("DATA QUALITY BASELINE REVIEW")
print("=" * 70)

dq_baseline_job = data_quality_monitor.latest_baselining_job
if dq_baseline_job is None:
    print("✗ No baseline job found — run Section 8.3.3 first")
else:
    print(f"✓ Baseline job : {dq_baseline_job.job_name}")

    stats_file       = f"{dq_baseline_uri}/statistics.json"
    constraints_file = f"{dq_baseline_uri}/constraints.json"

    try:
        local_stats = S3Downloader.download(stats_file, "/tmp/dq_stats")
        with open(local_stats[0]) as fh:
            dq_stats = json.load(fh)

        def _fmt(val, decimals=4):
            """Format a numeric value; return 'N/A' if missing or non-numeric."""
            try:
                return f"{float(val):.{decimals}f}"
            except (TypeError, ValueError):
                return "N/A"

        features = dq_stats.get("features", [])
        print(f"\nTotal features analysed: {len(features)}")
        print("\nFirst 8 features:")
        for feat in features[:8]:
            name  = feat["name"]
            ftype = feat.get("inferred_type", "unknown")
            if "numerical_statistics" in feat:
                ns = feat["numerical_statistics"]
                print(
                    f"  {name:<35} [{ftype}]  "
                    f"mean={_fmt(ns.get('mean'))}  "
                    f"std={_fmt(ns.get('stdDev'))}"
                )
            else:
                ss = feat.get("string_statistics", {})
                print(
                    f"  {name:<35} [{ftype}]  "
                    f"distinct={ss.get('distinct_count', 'N/A')}"
                )
    except Exception as e:
        print(f"⚠ Could not download statistics file: {e}")

#### 8.3.5 Create Hourly Data Quality Monitoring Schedule

In [ ]:
# EndpointInput for DefaultModelMonitor
dq_endpoint_input = EndpointInput(
    endpoint_name=endpoint_name,
    destination="/opt/ml/processing/input_data",
    s3_input_mode="File",
    s3_data_distribution_type="FullyReplicated",
)

dq_schedule_prefix = "venuesignal"
dq_schedule_name = (
    f"{dq_schedule_prefix}-dq-schedule-"
    f"{datetime.now(timezone.utc):%Y%m%d-%H%M%S}"
)
print(f"Creating data quality schedule: {dq_schedule_name}")

try:
    data_quality_monitor.create_monitoring_schedule(
        monitor_schedule_name=dq_schedule_name,
        endpoint_input=dq_endpoint_input,
        output_s3_uri=dq_results_uri,
        statistics=data_quality_monitor.baseline_statistics(),
        constraints=data_quality_monitor.suggested_constraints(),
        schedule_cron_expression=CronExpressionGenerator.hourly(),
        enable_cloudwatch_metrics=True,
    )
    print(f"✓ Data quality schedule created: {dq_schedule_name}")
    print(f"  Monitoring : INPUT features (13 columns)")
    print(f"  Baseline   : {dq_baseline_uri}")
    print(f"  Results    : {dq_results_uri}")
    %store dq_schedule_name
except Exception as e:
    print(f"✗ Error: {e}")
    raise

#### 8.3.6 Verify Data Quality Schedule

In [ ]:
try:
    desc = sagemaker_client.describe_monitoring_schedule(
        MonitoringScheduleName=dq_schedule_name
    )
    status = desc["MonitoringScheduleStatus"]
    print(f"Schedule : {dq_schedule_name}")
    print(f"Status   : {status}")
    if status == "Scheduled":
        print("✓ Data quality schedule is active and running hourly")
    elif "FailureReason" in desc:
        print(f"✗ Failure: {desc['FailureReason']}")
except Exception as e:
    print(f"✗ Could not describe schedule: {e}")


---

### 8.4 Infrastructure Monitoring

Infrastructure monitoring uses CloudWatch to track endpoint-level hardware
and performance metrics (CPU, memory, disk, latency, error rates) and runs
integration tests to validate end-to-end system health.


#### 8.4.1 Query SageMaker Endpoint Metrics from CloudWatch

In [ ]:
def get_cw_metric(metric_name, namespace="AWS/SageMaker", statistic="Average", hours=1):
    """Return the most recent datapoint for an endpoint metric."""
    cw  = cloudwatch_client
    end = datetime.now(timezone.utc)
    start = end - timedelta(hours=hours)
    dims = [
        {"Name": "EndpointName", "Value": endpoint_name},
        {"Name": "VariantName",  "Value": variant_name},
    ]
    try:
        resp = cw.get_metric_statistics(
            Namespace=namespace,
            MetricName=metric_name,
            Dimensions=dims,
            StartTime=start,
            EndTime=end,
            Period=300,
            Statistics=[statistic],
        )
        pts = resp.get("Datapoints", [])
        if pts:
            return sorted(pts, key=lambda x: x["Timestamp"])[-1][statistic]
    except Exception:
        pass
    return None

print("Querying endpoint metrics for the last hour…")
print("=" * 60)
latency      = get_cw_metric("ModelLatency")
invocations  = get_cw_metric("Invocations",          statistic="Sum")
errors_4xx   = get_cw_metric("Invocation4XXErrors",   statistic="Sum")
errors_5xx   = get_cw_metric("Invocation5XXErrors",   statistic="Sum")
overhead     = get_cw_metric("OverheadLatency")

cpu    = get_cw_metric("CPUUtilization",    namespace="/aws/sagemaker/Endpoints")
memory = get_cw_metric("MemoryUtilization", namespace="/aws/sagemaker/Endpoints")
disk   = get_cw_metric("DiskUtilization",   namespace="/aws/sagemaker/Endpoints")

print(f"  Avg Model Latency   : {latency/1000:.2f} ms"  if latency     else "  Avg Model Latency   : N/A (no data yet)")
print(f"  Total Invocations   : {invocations:.0f}"      if invocations else "  Total Invocations   : N/A")
print(f"  4XX Errors          : {errors_4xx:.0f}"       if errors_4xx  else "  4XX Errors          : N/A")
print(f"  5XX Errors          : {errors_5xx:.0f}"       if errors_5xx  else "  5XX Errors          : N/A")
print(f"  Overhead Latency    : {overhead/1000:.2f} ms" if overhead    else "  Overhead Latency    : N/A")
print(f"  CPU Utilization     : {cpu:.2f}%"             if cpu         else "  CPU Utilization     : N/A")
print(f"  Memory Utilization  : {memory:.2f}%"          if memory      else "  Memory Utilization  : N/A")
print(f"  Disk Utilization    : {disk:.2f}%"            if disk        else "  Disk Utilization    : N/A")


#### 8.4.2 Endpoint Health Checks

In [ ]:
def endpoint_health_check(ep_name, payload):
    """Single health check; returns latency and HTTP status."""
    runtime = boto3.client("sagemaker-runtime", region_name=REGION)
    try:
        t0 = time.time()
        resp = runtime.invoke_endpoint(
            EndpointName=ep_name,
            ContentType="text/csv",
            Body=payload,
            InferenceId=str(uuid.uuid4()),
        )
        return {
            "status": "healthy",
            "latency_ms": (time.time() - t0) * 1000,
            "http_status": resp["ResponseMetadata"]["HTTPStatusCode"],
            "timestamp": datetime.utcnow().isoformat(),
        }
    except Exception as exc:
        return {
            "status": "unhealthy",
            "error": str(exc),
            "timestamp": datetime.utcnow().isoformat(),
        }

def run_health_check_batch(ep_name, payload, n=10):
    """Run n health checks and return a summary dict."""
    results = []
    print(f"Running {n} health checks on {ep_name}…")
    for i in range(n):
        results.append(endpoint_health_check(ep_name, payload))
        if (i + 1) % 5 == 0:
            print(f"  Completed {i+1}/{n}")
        sleep(1)

    healthy = [r for r in results if r["status"] == "healthy"]
    success_rate = len(healthy) / len(results)
    avg_latency  = np.mean([r["latency_ms"] for r in healthy]) if healthy else None

    print("=" * 60)
    print(f"✓ Health Check Summary")
    print(f"  Success rate : {success_rate:.1%}")
    print(f"  Avg latency  : {avg_latency:.2f} ms" if avg_latency else "  Avg latency  : N/A")
    print(f"  Failures     : {len(results) - len(healthy)}")
    print("=" * 60)
    return {
        "total": len(results),
        "successful": len(healthy),
        "failed": len(results) - len(healthy),
        "success_rate": success_rate,
        "avg_latency_ms": avg_latency,
    }

# Use a single production row as health-check payload
hc_payload = production_df[xgb_features].iloc[0].fillna(0).to_csv(header=None, index=False).strip()
health_summary = run_health_check_batch(endpoint_name, hc_payload)


#### 8.4.3 Integration Tests

In [ ]:
INTEGRATION_TEST_CASES = [
    {
        "name": "High-rated business with ample parking",
        "input": production_df[xgb_features].iloc[0].fillna(0)
                   .to_csv(header=None, index=False).strip(),
        "expected_range": (0.5, 1.0),
    },
    {
        "name": "Low parking availability",
        "input": production_df[xgb_features].iloc[50].fillna(0)
                   .to_csv(header=None, index=False).strip(),
        "expected_range": (0.0, 1.0),
    },
    {
        "name": "Average business profile",
        "input": production_df[xgb_features].iloc[100].fillna(0)
                   .to_csv(header=None, index=False).strip(),
        "expected_range": (0.0, 1.0),
    },
]

def run_integration_tests(ep_name, test_cases):
    """Run integration tests and return pass-rate summary."""
    runtime = boto3.client("sagemaker-runtime", region_name=REGION)
    results = []
    print("=" * 60)
    print("Running integration tests…")
    print("=" * 60)
    for tc in test_cases:
        try:
            resp = runtime.invoke_endpoint(
                EndpointName=ep_name,
                ContentType="text/csv",
                Body=tc["input"],
            )
            # XGBoost may return multiple newline-separated scores;
            # take the first value only
            raw = resp["Body"].read().decode().strip()
            probability = float(raw.split("\n")[0].strip())
            lo, hi = tc["expected_range"]
            passed = lo <= probability <= hi
            print(f"{'✓ PASS' if passed else '✗ FAIL'}  {tc['name']}")
            print(f"       probability={probability:.4f}  expected=[{lo}, {hi}]")
            results.append({"name": tc["name"], "passed": passed, "value": probability})
        except Exception as exc:
            print(f"✗ ERROR  {tc['name']}: {exc}")
            results.append({"name": tc["name"], "passed": False, "error": str(exc)})

    passed_n = sum(1 for r in results if r["passed"])
    rate = passed_n / len(results) if results else 0
    print("=" * 60)
    print(f"Integration results: {passed_n}/{len(results)} passed ({rate:.1%})")
    print("=" * 60)
    return {"pass_rate": rate, "passed": passed_n,
            "failed": len(results) - passed_n, "total": len(results), "results": results}

test_results = run_integration_tests(endpoint_name, INTEGRATION_TEST_CASES)

#### 8.4.4 Integration Quality Gate

In [ ]:
INTEGRATION_THRESHOLDS = {
    "min_pass_rate":      0.95,
    "max_latency_ms":    1000,
    "min_availability":   0.99,
}

violations = []
if test_results["pass_rate"] < INTEGRATION_THRESHOLDS["min_pass_rate"]:
    violations.append(
        f"Integration pass rate {test_results['pass_rate']:.1%} "
        f"< threshold {INTEGRATION_THRESHOLDS['min_pass_rate']:.1%}"
    )
if health_summary["success_rate"] < INTEGRATION_THRESHOLDS["min_availability"]:
    violations.append(
        f"Endpoint availability {health_summary['success_rate']:.1%} "
        f"< threshold {INTEGRATION_THRESHOLDS['min_availability']:.1%}"
    )
if health_summary["avg_latency_ms"] and health_summary["avg_latency_ms"] > INTEGRATION_THRESHOLDS["max_latency_ms"]:
    violations.append(
        f"Avg latency {health_summary['avg_latency_ms']:.2f} ms "
        f"> threshold {INTEGRATION_THRESHOLDS['max_latency_ms']} ms"
    )

gate_passed = len(violations) == 0
print("=" * 60)
if gate_passed:
    print("✓ INTEGRATION QUALITY GATE: PASSED")
else:
    print("✗ INTEGRATION QUALITY GATE: FAILED")
    for i, v in enumerate(violations, 1):
        print(f"  {i}. {v}")
print("=" * 60)


#### 8.4.5 Publish Custom Integration Metrics to CloudWatch

In [ ]:
metric_data = [
    {
        "MetricName": "IntegrationTestPassRate",
        "Value":      test_results["pass_rate"] * 100,
        "Unit":       "Percent",
        "Dimensions": [{"Name": "EndpointName", "Value": endpoint_name}],
    },
    {
        "MetricName": "HealthCheckSuccessRate",
        "Value":      health_summary["success_rate"] * 100,
        "Unit":       "Percent",
        "Dimensions": [{"Name": "EndpointName", "Value": endpoint_name}],
    },
]
if health_summary.get("avg_latency_ms"):
    metric_data.append({
        "MetricName": "HealthCheckLatencyMs",
        "Value":      health_summary["avg_latency_ms"],
        "Unit":       "Milliseconds",
        "Dimensions": [{"Name": "EndpointName", "Value": endpoint_name}],
    })

try:
    cloudwatch_client.put_metric_data(
        Namespace="VenueSignal/Integration",
        MetricData=metric_data,
    )
    print("✓ Custom metrics published to CloudWatch (VenueSignal/Integration)")
    for m in metric_data:
        print(f"  {m['MetricName']}: {m['Value']:.2f} {m['Unit']}")
except Exception as e:
    print(f"✗ Failed to publish metrics: {e}")


#### 8.4.6 Create CloudWatch Alarms

In [ ]:
def create_alarm(alarm_name, metric_name, namespace, threshold,
                 comparison, statistic, period, evaluation_periods,
                 alarm_desc, dimensions, treat_missing="notBreaching"):
    """Helper: create or update a CloudWatch alarm."""
    try:
        cloudwatch_client.put_metric_alarm(
            AlarmName=alarm_name,
            AlarmDescription=alarm_desc,
            ActionsEnabled=True,
            MetricName=metric_name,
            Namespace=namespace,
            Statistic=statistic,
            Dimensions=dimensions,
            Period=period,
            EvaluationPeriods=evaluation_periods,
            DatapointsToAlarm=evaluation_periods,
            Threshold=threshold,
            ComparisonOperator=comparison,
            TreatMissingData=treat_missing,
        )
        print(f"✓ Alarm created/updated: {alarm_name}")
        return alarm_name
    except Exception as exc:
        print(f"✗ Failed to create alarm {alarm_name}: {exc}")
        return None

ep_dims = [{"Name": "EndpointName", "Value": endpoint_name}]
ep_variant_dims = ep_dims + [{"Name": "VariantName", "Value": variant_name}]

created_alarms = []

# 1. High 4XX error rate
created_alarms.append(create_alarm(
    alarm_name=f"{endpoint_name}-High4XXErrors",
    metric_name="Invocation4XXErrors",
    namespace="AWS/SageMaker",
    threshold=10,
    comparison="GreaterThanThreshold",
    statistic="Sum",
    period=300,
    evaluation_periods=2,
    alarm_desc="Alert: more than 10 client errors in 5 minutes",
    dimensions=ep_variant_dims,
))

# 2. High 5XX error rate
created_alarms.append(create_alarm(
    alarm_name=f"{endpoint_name}-High5XXErrors",
    metric_name="Invocation5XXErrors",
    namespace="AWS/SageMaker",
    threshold=5,
    comparison="GreaterThanThreshold",
    statistic="Sum",
    period=300,
    evaluation_periods=2,
    alarm_desc="Alert: more than 5 server errors in 5 minutes",
    dimensions=ep_variant_dims,
))

# 3. High model latency
created_alarms.append(create_alarm(
    alarm_name=f"{endpoint_name}-HighModelLatency",
    metric_name="ModelLatency",
    namespace="AWS/SageMaker",
    threshold=1_000_000,   # microseconds (SageMaker reports in µs)
    comparison="GreaterThanThreshold",
    statistic="Average",
    period=300,
    evaluation_periods=2,
    alarm_desc="Alert: average model latency exceeds 1 second",
    dimensions=ep_variant_dims,
))

# 4. High CPU utilisation
created_alarms.append(create_alarm(
    alarm_name=f"{endpoint_name}-HighCPU",
    metric_name="CPUUtilization",
    namespace="/aws/sagemaker/Endpoints",
    threshold=80,
    comparison="GreaterThanThreshold",
    statistic="Average",
    period=300,
    evaluation_periods=3,
    alarm_desc="Alert: endpoint CPU utilisation above 80 %",
    dimensions=ep_dims,
))

# 5. High memory utilisation
created_alarms.append(create_alarm(
    alarm_name=f"{endpoint_name}-HighMemory",
    metric_name="MemoryUtilization",
    namespace="/aws/sagemaker/Endpoints",
    threshold=85,
    comparison="GreaterThanThreshold",
    statistic="Average",
    period=300,
    evaluation_periods=3,
    alarm_desc="Alert: endpoint memory utilisation above 85 %",
    dimensions=ep_dims,
))

# 6. Integration test pass-rate drop (custom metric)
created_alarms.append(create_alarm(
    alarm_name=f"{endpoint_name}-IntegrationTestFailure",
    metric_name="IntegrationTestPassRate",
    namespace="VenueSignal/Integration",
    threshold=95,
    comparison="LessThanThreshold",
    statistic="Average",
    period=600,
    evaluation_periods=1,
    alarm_desc="Alert: integration test pass rate below 95 %",
    dimensions=ep_dims,
    treat_missing="breaching",
))

# 7. Model quality: recall drift (populated once MQ schedule runs)
mq_alarm_dims = [
    {"Name": "Endpoint",           "Value": endpoint_name},
    {"Name": "MonitoringSchedule", "Value": mq_schedule_name},
]
created_alarms.append(create_alarm(
    alarm_name="VenueSignal-ModelQuality-RecallDrift",
    metric_name="recall",
    namespace="aws/sagemaker/Endpoints/model-metrics",
    threshold=0.70,
    comparison="LessThanOrEqualToThreshold",
    statistic="Average",
    period=600,
    evaluation_periods=1,
    alarm_desc="Alert: model recall dropped below 0.70 baseline",
    dimensions=mq_alarm_dims,
    treat_missing="breaching",
))

created_alarms = [a for a in created_alarms if a]
print(f"\n✓ {len(created_alarms)} CloudWatch alarms configured")


---

### 8.5 CloudWatch Dashboard

A single comprehensive dashboard that surfaces all four monitoring pillars:
endpoint performance, resource utilisation, model quality, and data quality.


#### 8.5.1 Create Comprehensive Monitoring Dashboard

In [ ]:
dashboard_name = f"{project_name}-monitoring-dashboard"

def _metric_widget(title, metrics, x, y, w=12, h=6, stat="Average",
                   period=300, region=REGION, y_min=0, y_label=None):
    props = {
        "title": title,
        "metrics": metrics,
        "period": period,
        "stat": stat,
        "region": region,
        "view": "timeSeries",
        "yAxis": {"left": {"min": y_min}},
    }
    if y_label:
        props["yAxis"]["left"]["label"] = y_label
    return {"type": "metric", "x": x, "y": y, "width": w, "height": h,
            "properties": props}

dashboard_body = {
    "widgets": [
        # ── Row 0: Section title ──────────────────────────────────────────
        {
            "type": "text",
            "x": 0, "y": 0, "width": 24, "height": 2,
            "properties": {
                "markdown": (
                    "# VenueSignal — ML Monitoring Dashboard\n"
                    f"**Endpoint:** `{endpoint_name}` | "
                    f"**Region:** `{REGION}` | "
                    "Metrics refresh every 5 min"
                )
            },
        },
        # ── Row 1: Endpoint invocations & errors ──────────────────────────
        _metric_widget(
            title="Endpoint Invocations",
            metrics=[
                ["AWS/SageMaker", "Invocations",
                 "EndpointName", endpoint_name, "VariantName", variant_name,
                 {"stat": "Sum", "label": "Invocations (Sum)"}],
            ],
            stat="Sum", x=0, y=2,
        ),
        _metric_widget(
            title="Invocation Errors (4XX / 5XX)",
            metrics=[
                ["AWS/SageMaker", "Invocation4XXErrors",
                 "EndpointName", endpoint_name, "VariantName", variant_name,
                 {"stat": "Sum", "color": "#ff7f0e", "label": "4XX Client Errors"}],
                ["AWS/SageMaker", "Invocation5XXErrors",
                 "EndpointName", endpoint_name, "VariantName", variant_name,
                 {"stat": "Sum", "color": "#d62728", "label": "5XX Server Errors"}],
            ],
            stat="Sum", x=12, y=2,
        ),
        # ── Row 2: Latency ────────────────────────────────────────────────
        _metric_widget(
            title="Model Latency (µs) — Avg & p99",
            metrics=[
                ["AWS/SageMaker", "ModelLatency",
                 "EndpointName", endpoint_name, "VariantName", variant_name,
                 {"stat": "Average", "label": "Avg Latency"}],
                ["...", {"stat": "p99", "label": "p99 Latency", "color": "#e377c2"}],
            ],
            x=0, y=8, y_label="Microseconds",
        ),
        _metric_widget(
            title="Overhead Latency (µs)",
            metrics=[
                ["AWS/SageMaker", "OverheadLatency",
                 "EndpointName", endpoint_name, "VariantName", variant_name,
                 {"stat": "Average", "label": "Avg Overhead"}],
            ],
            x=12, y=8, y_label="Microseconds",
        ),
        # ── Row 3: Resource utilisation ────────────────────────────────────
        _metric_widget(
            title="CPU Utilization (%)",
            metrics=[
                ["/aws/sagemaker/Endpoints", "CPUUtilization",
                 "EndpointName", endpoint_name,
                 {"stat": "Average", "label": "CPU %"}],
            ],
            x=0, y=14, y_min=0, y_label="Percent",
        ),
        _metric_widget(
            title="Memory Utilization (%)",
            metrics=[
                ["/aws/sagemaker/Endpoints", "MemoryUtilization",
                 "EndpointName", endpoint_name,
                 {"stat": "Average", "label": "Memory %"}],
            ],
            x=12, y=14, y_min=0, y_label="Percent",
        ),
        _metric_widget(
            title="Disk Utilization (%)",
            metrics=[
                ["/aws/sagemaker/Endpoints", "DiskUtilization",
                 "EndpointName", endpoint_name,
                 {"stat": "Average", "label": "Disk %"}],
            ],
            x=0, y=20, w=12, y_min=0, y_label="Percent",
        ),
        # ── Row 4: Model quality metrics (populated by MQ schedule) ───────
        _metric_widget(
            title="Model Quality — Recall (vs baseline)",
            metrics=[
                ["aws/sagemaker/Endpoints/model-metrics", "recall",
                 "Endpoint", endpoint_name,
                 "MonitoringSchedule", mq_schedule_name,
                 {"stat": "Average", "label": "Recall"}],
            ],
            x=12, y=20, y_min=0, y_label="Score",
        ),
        # ── Row 5: Integration / custom metrics ───────────────────────────
        _metric_widget(
            title="Integration Test Pass Rate (%)",
            metrics=[
                ["VenueSignal/Integration", "IntegrationTestPassRate",
                 "EndpointName", endpoint_name,
                 {"stat": "Average", "label": "Pass Rate %"}],
            ],
            stat="Average", x=0, y=26, y_min=0,
        ),
        _metric_widget(
            title="Health Check Latency (ms)",
            metrics=[
                ["VenueSignal/Integration", "HealthCheckLatencyMs",
                 "EndpointName", endpoint_name,
                 {"stat": "Average", "label": "Latency ms"}],
            ],
            x=12, y=26, y_min=0, y_label="Milliseconds",
        ),
    ]
}

try:
    cloudwatch_client.put_dashboard(
        DashboardName=dashboard_name,
        DashboardBody=json.dumps(dashboard_body),
    )
    print(f"✓ CloudWatch dashboard created/updated: {dashboard_name}")
    print(f"\nView dashboard at:")
    print(
        f"  https://console.aws.amazon.com/cloudwatch/home"
        f"?region={REGION}#dashboards:name={dashboard_name}"
    )
except Exception as e:
    print(f"✗ Error creating dashboard: {e}")


---

### 8.6 Model Performance Tracking

Compare training-time performance across all models and save a persistent
summary to S3 for the monitoring record.


In [ ]:
print("=" * 70)
print("MODEL PERFORMANCE TRACKING — Validation Set")
print("=" * 70)

# ── Retrieve stored results from Section 6 ─────────────────────────────────
%store -r baseline_results
%store -r xgb_results_val
%store -r xgb_results_test

b1 = baseline_results['baseline1']['val']
b2 = baseline_results['baseline2']['val']
xv = xgb_results_val

import pandas as pd

summary = pd.DataFrame([
    {"Model": "Baseline #1 (Heuristic)",         "Accuracy": f"{b1['accuracy']:.4f}",
     "Precision": f"{b1['precision']:.4f}",       "Recall": f"{b1['recall']:.4f}",
     "F1": f"{b1['f1']:.4f}",                    "RMSE": f"{b1['rmse']:.4f}",
     "Within 1★": f"{b1['within_1.0_stars']*100:.2f}%"},
    {"Model": "Baseline #2 (Logistic Reg.)",      "Accuracy": f"{b2['accuracy']:.4f}",
     "Precision": f"{b2['precision']:.4f}",       "Recall": f"{b2['recall']:.4f}",
     "F1": f"{b2['f1']:.4f}",                    "RMSE": f"{b2['rmse']:.4f}",
     "Within 1★": f"{b2['within_1.0_stars']*100:.2f}%"},
    {"Model": "XGBoost (Deployed)",               "Accuracy": f"{xv['accuracy']:.4f}",
     "Precision": f"{xv['precision']:.4f}",       "Recall": f"{xv['recall']:.4f}",
     "F1": f"{xv['f1']:.4f}",                    "RMSE": f"{xv['rmse']:.4f}",
     "Within 1★": f"{xv['within_1.0_stars']*100:.2f}%"},
])
display(summary)

# ── XGBoost improvements ───────────────────────────────────────────────────
def pct_change(new, old):
    return (new - old) / old * 100

print("\nXGBoost improvements over Baseline #1:")
print(f"  Accuracy : {pct_change(xv['accuracy'], b1['accuracy']):+.2f}%")
print(f"  F1-Score : {pct_change(xv['f1'],       b1['f1']):+.2f}%")
print(f"  RMSE     : {pct_change(b1['rmse'],     xv['rmse']):+.2f}%  (lower is better)")

print("\nXGBoost improvements over Baseline #2:")
print(f"  Accuracy : {pct_change(xv['accuracy'], b2['accuracy']):+.2f}%")
print(f"  F1-Score : {pct_change(xv['f1'],       b2['f1']):+.2f}%")
print(f"  RMSE     : {pct_change(b2['rmse'],     xv['rmse']):+.2f}%  (lower is better)")

# Save to S3 as monitoring artefact
summary_local = "/tmp/model_performance_summary.csv"
summary.to_csv(summary_local, index=False)
s3_client.upload_file(
    summary_local,
    BASE_BUCKET_NAME,
    f"{MONITORING_PREFIX}performance_summary.csv",
)
print(f"\n✓ Performance summary saved to s3://{MONITORING_DIR}performance_summary.csv")


---

### 8.7 Generate Monitoring Reports

Generate comprehensive status reports for all monitoring schedules and
upload them to S3.


#### 8.7.1 List All Monitoring Schedules

In [ ]:
schedules = sagemaker_client.list_monitoring_schedules(
    EndpointName=endpoint_name,
    MaxResults=100,
)
print("=== Active Monitoring Schedules ===")
for s in schedules["MonitoringScheduleSummaries"]:
    print(f"\nSchedule : {s['MonitoringScheduleName']}")
    print(f"  Type    : {s.get('MonitoringType', 'DataQuality')}")
    print(f"  Status  : {s['MonitoringScheduleStatus']}")
    print(f"  Created : {s['CreationTime']}")


#### 8.7.2 Check Latest Monitoring Executions

In [ ]:
def get_latest_execution(schedule_name):
    try:
        execs = sagemaker_client.list_monitoring_executions(
            MonitoringScheduleName=schedule_name,
            MaxResults=1,
            SortBy="CreationTime",
            SortOrder="Descending",
        )
        summaries = execs.get("MonitoringExecutionSummaries", [])
        return summaries[0] if summaries else None
    except Exception:
        return None

print("=== Latest Monitoring Executions ===")
for s in schedules["MonitoringScheduleSummaries"]:
    name = s["MonitoringScheduleName"]
    execution = get_latest_execution(name)
    print(f"\n{name}:")
    if execution:
        print(f"  Status   : {execution['MonitoringExecutionStatus']}")
        print(f"  Scheduled: {execution.get('ScheduledTime', 'N/A')}")
        if "ProcessingJobArn" in execution:
            print(f"  Job      : {execution['ProcessingJobArn'].split('/')[-1]}")
        if "FailureReason" in execution:
            print(f"  Failure  : {execution['FailureReason']}")
    else:
        print("  No executions yet — first run at the top of the next hour")


#### 8.7.3 Model Quality Report

In [ ]:
print("=== Model Quality Monitoring Report ===")
try:
    mq_files = S3Downloader.list(mq_results_uri)
    if mq_files:
        print(f"Found {len(mq_files)} result file(s) in {mq_results_uri}")
        for fpath in sorted(mq_files, reverse=True):
            if "constraint_violations" in fpath:
                local = S3Downloader.download(fpath, "/tmp/mq_report")
                with open(local[0]) as fh:
                    v = json.load(fh)
                viol = v.get("violations", [])
                if viol:
                    print(f"\n⚠ {len(viol)} constraint violation(s) detected:")
                    for item in viol:
                        print(f"  - {item.get('metric_name', 'N/A')}: "
                              f"{item.get('description', item)}")
                else:
                    print("\n✓ No model quality violations detected")
                break
    else:
        print("No model quality results yet — schedule must run at least once")
        print("First execution occurs at the top of the next hour.")
except Exception as e:
    print(f"Cannot retrieve model quality results: {e}")
    print("This is expected if the monitoring schedule has not yet run.")


#### 8.7.4 Data Quality Report

In [ ]:
print("=== Data Quality Monitoring Report ===")
try:
    dq_files = S3Downloader.list(dq_results_uri)
    if dq_files:
        print(f"Found {len(dq_files)} result file(s) in {dq_results_uri}")
        for fpath in sorted(dq_files, reverse=True):
            if "constraint_violations" in fpath:
                local = S3Downloader.download(fpath, "/tmp/dq_report")
                with open(local[0]) as fh:
                    v = json.load(fh)
                viol = v.get("violations", [])
                if viol:
                    print(f"\n⚠ {len(viol)} data quality violation(s) detected:")
                    for item in viol:
                        feat = item.get("feature_name", "N/A")
                        desc = item.get("description", item)
                        print(f"  - Feature '{feat}': {desc}")
                else:
                    print("\n✓ No data quality violations detected")
                break
    else:
        print("No data quality results yet — schedule must run at least once")
        print("First execution occurs at the top of the next hour.")
except Exception as e:
    print(f"Cannot retrieve data quality results: {e}")
    print("This is expected if the monitoring schedule has not yet run.")


#### 8.7.5 Comprehensive Monitoring Summary Report

In [ ]:
# Retrieve CloudWatch alarms prefixed with the project name
try:
    alarms_resp = cloudwatch_client.describe_alarms(
        AlarmNamePrefix=project_name,
        MaxRecords=100,
    )
    alarm_list = alarms_resp.get("MetricAlarms", [])
except Exception as e:
    print(f"Warning: Could not retrieve alarms: {e}")
    alarm_list = []

# Query the latest infrastructure metrics
latency_latest  = get_cw_metric("ModelLatency")
invoc_latest    = get_cw_metric("Invocations",        statistic="Sum")
err4xx_latest   = get_cw_metric("Invocation4XXErrors", statistic="Sum")
cpu_latest      = get_cw_metric("CPUUtilization",     namespace="/aws/sagemaker/Endpoints")
mem_latest      = get_cw_metric("MemoryUtilization",  namespace="/aws/sagemaker/Endpoints")
disk_latest     = get_cw_metric("DiskUtilization",    namespace="/aws/sagemaker/Endpoints")

report = {
    "report_timestamp":       datetime.now(timezone.utc).isoformat(),
    "endpoint_name":          endpoint_name,
    "monitoring_schedules": [
        {
            "name":   s["MonitoringScheduleName"],
            "type":   s.get("MonitoringType", "DataQuality"),
            "status": s["MonitoringScheduleStatus"],
        }
        for s in schedules["MonitoringScheduleSummaries"]
    ],
    "infrastructure_metrics": {
        "latency_us":           latency_latest,
        "invocations":          invoc_latest,
        "errors_4xx":           err4xx_latest,
        "cpu_utilization_pct":  cpu_latest,
        "memory_utilization_pct": mem_latest,
        "disk_utilization_pct": disk_latest,
    },
    "cloudwatch_alarms": [
        {
            "name":      a["AlarmName"],
            "state":     a["StateValue"],
            "metric":    a.get("MetricName", "Expression"),
            "threshold": a.get("Threshold"),
        }
        for a in alarm_list
    ],
    "integration_tests": {
        "pass_rate":    test_results["pass_rate"],
        "passed":       test_results["passed"],
        "failed":       test_results["failed"],
    },
    "health_check": {
        "success_rate": health_summary["success_rate"],
        "avg_latency_ms": health_summary["avg_latency_ms"],
    },
}

# Persist report locally and upload to S3
report_filename = f"monitoring_report_{datetime.now(timezone.utc):%Y%m%d_%H%M%S}.json"
with open(report_filename, "w") as fh:
    json.dump(report, fh, indent=2, default=str)

report_s3_uri = S3Uploader.upload(report_filename, reports_uri)
print(f"✓ Monitoring report saved to: {report_s3_uri}")
print("\n=== Report Summary ===")
print(json.dumps(report, indent=2, default=str))


---

### 8.8 Examine Execution Results

The cells below should be run **after** the monitoring schedules have
completed their first hourly execution (usually within ~20 minutes of the
top of the next hour).  They download constraint-violation reports and
analyse the CloudWatch metrics emitted by the model quality job.


#### 8.8.1 Wait for a Model Quality Execution to Complete

In [ ]:
# Poll for the first successful execution — waits up to 90 minutes
deadline = time.time() + 90 * 60
print(f"Polling for model quality execution (schedule: {mq_schedule_name})…")
print("Note: first execution fires at the top of the next hour + up to 20 min.")

latest_execution = None
while time.time() < deadline:
    try:
        desc = sagemaker_client.describe_monitoring_schedule(
            MonitoringScheduleName=mq_schedule_name
        )
        print(f"  Schedule status: {desc['MonitoringScheduleStatus']}")
        last_exec = desc.get("LastMonitoringExecutionSummary")
        if last_exec:
            print(f"  Last execution : {last_exec['MonitoringExecutionStatus']}")
            if last_exec["MonitoringExecutionStatus"] in ("Completed", "CompletedWithViolations"):
                print("✓ Execution completed!")
                # Retrieve the execution object for deeper inspection
                execs = xgboost_model_quality_monitor.list_executions()
                if execs:
                    latest_execution = execs[-1]
                break
            elif last_exec["MonitoringExecutionStatus"] == "Failed":
                print(f"✗ Execution failed: {last_exec.get('FailureReason', 'unknown')}")
                break
    except Exception as e:
        print(f"  Polling error: {e}")
    sleep(30)

if time.time() >= deadline:
    print("⏱ Timeout: no completed execution within 90 minutes.")
    print("  Re-run this cell later or check the SageMaker console.")


#### 8.8.2 Review Constraint Violations

In [ ]:
if latest_execution is not None:
    try:
        violations = latest_execution.constraint_violations().body_dict.get(
            "violations", []
        )
        if violations:
            import pandas as pd
            pd.options.display.max_colwidth = None
            print(f"⚠ {len(violations)} constraint violation(s):")
            display(pd.json_normalize(violations).head(20))
        else:
            print("✓ No constraint violations in this execution")
    except Exception as e:
        print(f"Could not retrieve violations: {e}")
else:
    print("No completed execution available yet — run Section 8.8.1 first")


#### 8.8.3 Analyse Model Quality CloudWatch Metrics

In [ ]:
# List all model-quality metrics emitted by the monitor for this schedule
cw_namespace = "aws/sagemaker/Endpoints/model-metrics"
cw_dims = [
    {"Name": "Endpoint",           "Value": endpoint_name},
    {"Name": "MonitoringSchedule", "Value": mq_schedule_name},
]

paginator = cloudwatch_client.get_paginator("list_metrics")
mq_metric_names = []
for page in paginator.paginate(Dimensions=cw_dims, Namespace=cw_namespace):
    for metric in page.get("Metrics", []):
        mq_metric_names.append(metric["MetricName"])

if mq_metric_names:
    print(f"Model quality metrics in CloudWatch ({len(mq_metric_names)} found):")
    for name in mq_metric_names:
        print(f"  {name}")
else:
    print("No model quality metrics in CloudWatch yet.")
    print("They appear after the first successful monitoring execution.")


In [ ]:
import boto3
from datetime import datetime, timezone, timedelta

cw = boto3.client("cloudwatch", region_name=REGION)

# Check 1: What metrics exist for this endpoint?
print(f"Checking CloudWatch metrics for: {endpoint_name}\n")
response = cw.list_metrics(
    Namespace="AWS/SageMaker",
    Dimensions=[{"Name": "EndpointName", "Value": endpoint_name}]
)
if response["Metrics"]:
    print(f"✓ Found {len(response['Metrics'])} metric(s):")
    for m in response["Metrics"]:
        dims = {d["Name"]: d["Value"] for d in m["Dimensions"]}
        print(f"  {m['MetricName']:30} VariantName={dims.get('VariantName','N/A')}")
else:
    print("✗ NO metrics found in AWS/SageMaker namespace for this endpoint")
    print("  The traffic thread is not running or endpoint has never been invoked")

# Check 2: Is the traffic thread alive?
import threading
print(f"\nRunning threads:")
for t in threading.enumerate():
    print(f"  {t.name} (daemon={t.daemon}, alive={t.is_alive()})")

# Check 3: Send 5 test requests right now and verify
print(f"\nSending 5 test requests to verify endpoint is reachable...")
runtime = boto3.client("sagemaker-runtime", region_name=REGION)
for i in range(5):
    try:
        payload = (
            production_df[xgb_features]
            .iloc[i]
            .fillna(0)
            .to_csv(header=None, index=False)
            .strip()
        )
        resp = runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType="text/csv",
            Body=payload,
            InferenceId=str(i),
        )
        result = resp["Body"].read().decode().strip()
        print(f"  Request {i}: ✓  prediction={result[:20]}")
    except Exception as e:
        print(f"  Request {i}: ✗  {e}")

#### 8.9 Stop Monitors/Review

In [ ]:
#import threading

# List all running threads
#for t in threading.enumerate():
#    print(t.name, t.daemon)

In [ ]:
#xgboost_model_quality_monitor.stop_monitoring_schedule()
#data_quality_monitor.stop_monitoring_schedule()
#print("✓ Both monitoring schedules stopped")

In [ ]:
#Review Logs/Errors
capture_files = sorted(S3Downloader.list(f"{s3_capture_upload_path}/{endpoint_name}"))
print(f"Found {len(capture_files)} capture files in new path")

if capture_files:
    # Read the most recent file
    raw = S3Downloader.read_file(capture_files[-1])
    first_record = json.loads(raw.split("\n")[0])
    print("\nCapture record structure:")
    print(json.dumps(first_record, indent=2))

    # Show input and output data
    print("\n--- Input data (what was sent to endpoint) ---")
    print(first_record.get("captureData", {}).get("endpointInput", {}).get("data", "N/A"))
    print("\n--- Output data (what endpoint returned) ---")
    print(first_record.get("captureData", {}).get("endpointOutput", {}).get("data", "N/A"))
    print("\n--- Encoding ---")
    print("Input encoding: ", first_record.get("captureData", {}).get("endpointInput", {}).get("encoding", "N/A"))
    print("Output encoding:", first_record.get("captureData", {}).get("endpointOutput", {}).get("encoding", "N/A"))
    print("\n--- InferenceId ---")
    print(first_record.get("eventMetadata", {}).get("inferenceId", "N/A"))
else:
    print("No capture files found yet — send some traffic first")

In [ ]:
# Review Cloudwatch logs
logs_client = boto3.client("logs", region_name=REGION)

try:
    # SageMaker model monitor logs go to this log group
    log_group = f"/aws/sagemaker/ProcessingJobs"

    # Find the most recent model quality processing job
    desc = sagemaker_client.describe_monitoring_schedule(
        MonitoringScheduleName=mq_schedule_name
    )
    last_exec = desc.get("LastMonitoringExecutionSummary", {})
    job_arn = last_exec.get("ProcessingJobArn", "")
    job_name = job_arn.split("/")[-1] if job_arn else None

    if job_name:
        print(f"Checking logs for job: {job_name}")
        streams = logs_client.describe_log_streams(
        logGroupName=log_group,
        logStreamNamePrefix=job_name,
    )
        if streams["logStreams"]:
            stream_name = streams["logStreams"][0]["logStreamName"]
            events = logs_client.get_log_events(
                logGroupName=log_group,
                logStreamName=stream_name,
                limit=50,
            )
            print(f"\nLast 50 log lines from: {stream_name}\n")
            for e in events["events"]:
                print(e["message"])
        else:
            print("No log streams found for this job")
    else:
        print("No processing job found in last execution")
except Exception as e:
    print(f"Error fetching logs: {e}")

## 9. CI/CD Pipeline <a id='section-9'></a>

This section implements a **SageMaker Pipeline** to automate the full model lifecycle:

| Step | Name | Description |
|------|------|-------------|
| 1 | Data Validation | Checks incoming data quality — fails pipeline if data is unusable |
| 2 | Model Training | Retrains XGBoost on the latest feature data |
| 3 | Model Evaluation | Computes F1, ROC-AUC, and MAE against the test set |
| 4 | Quality Gate | Conditionally proceeds only if F1 ≥ threshold |
| 5a | Model Registration | Registers passing model to SageMaker Model Registry |
| 5b | Fail Step | Halts and reports failure if quality gate is not met |

**Two execution modes are demonstrated:**
- **Success run** — model meets quality threshold, registered to Model Registry
- **Failure run** — threshold raised above model performance, pipeline halts at gate

**S3 Naming Convention:** All CI/CD artifacts continue to follow the existing `yelp-aai540-group6-{account_id}` pattern.

### 9.0 — Cleanup / Reset Section 9

Run this cell **before re-running section 9** if a previous attempt left partial resources behind.

**What this deletes:**
| Resource | Action |
|----------|--------|
| SageMaker Pipeline | Stopped + deleted |
| All pipeline executions | Stopped if still running |
| Model Package Group + all versions | All versions deleted, then group deleted |
| S3 `cicd/` prefix | All objects cleared |

**What this does NOT delete:**
- Endpoint (Section 7)
- Feature store (Section 5)
- Trained model artifacts in `models/` (Section 6)
- Monitoring schedules (Section 8)
- All other S3 data

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
#  SECTION 9 CLEANUP — run before re-running section 9 from the top
# ════════════════════════════════════════════════════════════════════════════
import boto3
import time

PIPELINE_NAME            = "venuesignal-cicd-pipeline"
MODEL_PACKAGE_GROUP_NAME = "VenueSignalModelPackageGroup"
CICD_PREFIX              = "cicd/"

s3_client  = boto3.client('s3',        region_name=REGION)
sm_client  = boto3.client('sagemaker', region_name=REGION)

errors = []


# ── 1. Stop any running pipeline executions ───────────────────────────────────
print("[1/4] Stopping any running pipeline executions...")
try:
    executions = sm_client.list_pipeline_executions(
        PipelineName=PIPELINE_NAME
    )['PipelineExecutionSummaries']
    running = [e for e in executions
               if e['PipelineExecutionStatus'] == 'Executing']
    if running:
        for e in running:
            sm_client.stop_pipeline_execution(
                PipelineExecutionArn=e['PipelineExecutionArn']
            )
            print(f"  Stopped: {e.get('PipelineExecutionDisplayName', e['PipelineExecutionArn'])}")
        print(f"  Waiting 15s for executions to stop...")
        time.sleep(15)
    else:
        print("  No running executions found")
except sm_client.exceptions.ResourceNotFound:
    print("  Pipeline does not exist yet — skipping")
except Exception as e:
    print(f"  ⚠  {e}")
    errors.append(f"Stop executions: {e}")


# ── 2. Delete the pipeline ────────────────────────────────────────────────────
print("\n[2/4] Deleting pipeline...")
try:
    sm_client.delete_pipeline(PipelineName=PIPELINE_NAME)
    print(f"  ✓ Deleted pipeline: {PIPELINE_NAME}")
except sm_client.exceptions.ResourceNotFound:
    print(f"  Pipeline '{PIPELINE_NAME}' not found — skipping")
except Exception as e:
    print(f"  ⚠  {e}")
    errors.append(f"Delete pipeline: {e}")


# ── 3. Delete all model package versions then the group ───────────────────────
print("\n[3/4] Deleting model package group and all versions...")
try:
    # Must delete all versions before the group
    paginator = sm_client.get_paginator('list_model_packages')
    pages = paginator.paginate(ModelPackageGroupName=MODEL_PACKAGE_GROUP_NAME)
    version_count = 0
    for page in pages:
        for pkg in page['ModelPackageSummaryList']:
            sm_client.delete_model_package(
                ModelPackageName=pkg['ModelPackageArn']
            )
            version_count += 1
            print(f"  Deleted version {pkg['ModelPackageVersion']}: {pkg['ModelApprovalStatus']}")

    if version_count == 0:
        print("  No model versions found")

    # Now delete the group itself
    sm_client.delete_model_package_group(
        ModelPackageGroupName=MODEL_PACKAGE_GROUP_NAME
    )
    print(f"  ✓ Deleted model package group: {MODEL_PACKAGE_GROUP_NAME}")

except sm_client.exceptions.ResourceNotFound:
    print(f"  Model package group '{MODEL_PACKAGE_GROUP_NAME}' not found — skipping")
except Exception as e:
    print(f"  ⚠  {e}")
    errors.append(f"Delete model packages: {e}")


# ── 4. Clear S3 cicd/ prefix ─────────────────────────────────────────────────
print(f"\n[4/4] Clearing S3 prefix: s3://{BASE_BUCKET_NAME}/{CICD_PREFIX}")
try:
    paginator = s3_client.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=BASE_BUCKET_NAME, Prefix=CICD_PREFIX)
    obj_count = 0
    for page in pages:
        objects = page.get('Contents', [])
        if objects:
            s3_client.delete_objects(
                Bucket=BASE_BUCKET_NAME,
                Delete={'Objects': [{'Key': o['Key']} for o in objects]}
            )
            obj_count += len(objects)
    if obj_count:
        print(f"  ✓ Deleted {obj_count} S3 object(s)")
    else:
        print("  No S3 objects found under cicd/ — skipping")
except Exception as e:
    print(f"  ⚠  {e}")
    errors.append(f"Clear S3: {e}")


# ── 5. Clear stale kernel variables ──────────────────────────────────────────
print("\n[5/4] Clearing stale kernel variables...")
cicd_vars = [
    'pipeline', 'f1_threshold_param', 'sklearn_processor', 'xgb_estimator',
    'xgb_container', 'step_validation', 'step_training', 'step_evaluation',
    'step_register', 'step_fail', 'step_condition', 'evaluation_report',
    'success_execution', 'failure_execution',
    'success_execution_arn', 'failure_execution_arn', 'approved_model_arn',
    'model_package_group_name', 'model_package_arn'
]
cleared = []
for var in cicd_vars:
    if var in globals():
        del globals()[var]
        cleared.append(var)
if cleared:
    print(f"  ✓ Cleared {len(cleared)} kernel variable(s)")
else:
    print("  No CI/CD kernel variables found")


# ── Summary ───────────────────────────────────────────────────────────────────
print()
print("=" * 60)
if not errors:
    print("CLEANUP COMPLETE — Section 9 is ready for a clean rerun")
    print("=" * 60)
    print()
    print("  Proceed from section 9.2 (Model Package Group creation).")
    print("  Section 9.1 imports/config do not need to be rerun.")
else:
    print("CLEANUP COMPLETE WITH WARNINGS")
    print("=" * 60)
    print("  Non-fatal errors (likely resources that didn't exist):")
    for err in errors:
        print(f"    - {err}")
    print()
    print("  Safe to proceed with rerun from section 9.2.")

### 9.1 CI/CD Imports and Configuration

In [ ]:
# CI/CD specific imports — builds on Section 1 imports already loaded
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.parameters import ParameterFloat, ParameterString
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.model import ModelPackage
from sagemaker.inputs import TrainingInput

print("CI/CD imports loaded successfully")

In [ ]:
# ── Restore stored variables from earlier sections ──────────────────────────
%store -r account_id
%store -r REGION
%store -r role
%store -r BASE_BUCKET_NAME
%store -r MODEL_PREFIX
%store -r MONITORING_PREFIX
%store -r FEATURE_PREFIX
%store -r xgb_model_data
%store -r xgb_features
%store -r endpoint_name

# ── CI/CD-specific S3 paths (same bucket, new prefix) ──────────────────────
CICD_PREFIX          = "cicd/"
CICD_SCRIPTS_PREFIX  = f"{CICD_PREFIX}scripts"
CICD_EVAL_PREFIX     = f"{CICD_PREFIX}evaluation"
CICD_LOGS_PREFIX     = f"{CICD_PREFIX}logs"

CICD_SCRIPTS_S3      = f"s3://{BASE_BUCKET_NAME}/{CICD_SCRIPTS_PREFIX}"
CICD_EVAL_S3         = f"s3://{BASE_BUCKET_NAME}/{CICD_EVAL_PREFIX}"
XGB_TRAIN_S3         = f"s3://{BASE_BUCKET_NAME}/models/xgboost-training/train.csv"
XGB_VAL_S3           = f"s3://{BASE_BUCKET_NAME}/models/xgboost-training/validation.csv"
XGB_TEST_S3          = f"s3://{BASE_BUCKET_NAME}/models/xgboost-training/test.csv"
XGB_OUTPUT_S3        = f"s3://{BASE_BUCKET_NAME}/{MODEL_PREFIX}cicd-runs/"

# ── Pipeline identity ───────────────────────────────────────────────────────
PIPELINE_NAME            = "venuesignal-cicd-pipeline"
MODEL_PACKAGE_GROUP_NAME = "VenueSignalModelPackageGroup"

# ── Quality gate threshold (pipeline parameter — easy to override at runtime)
DEFAULT_F1_THRESHOLD     = 0.70   # Change to trigger failure demo

print("=" * 70)
print("CI/CD CONFIGURATION")
print("=" * 70)
print(f"  Bucket:              {BASE_BUCKET_NAME}")
print(f"  CI/CD prefix:        {CICD_PREFIX}")
print(f"  Scripts S3:          {CICD_SCRIPTS_S3}")
print(f"  Evaluation S3:       {CICD_EVAL_S3}")
print(f"  Pipeline name:       {PIPELINE_NAME}")
print(f"  Model Package Group: {MODEL_PACKAGE_GROUP_NAME}")
print(f"  F1 gate threshold:   {DEFAULT_F1_THRESHOLD}")
print("=" * 70)

### 9.2 Create Model Package Group in Model Registry

The Model Registry tracks all approved versions of VenueSignal. This only needs to run once.

In [ ]:
sm_client = boto3.client('sagemaker', region_name=REGION)

try:
    sm_client.create_model_package_group(
        ModelPackageGroupName=MODEL_PACKAGE_GROUP_NAME,
        ModelPackageGroupDescription=(
            "VenueSignal XGBoost models for Yelp business rating prediction. "
            "AAI-540 Group 6. Only F1 >= threshold models are registered."
        )
    )
    print(f"✓ Created Model Package Group: {MODEL_PACKAGE_GROUP_NAME}")
except sm_client.exceptions.ConflictException:
    print(f"✓ Model Package Group already exists: {MODEL_PACKAGE_GROUP_NAME}")

# Verify
group_info = sm_client.describe_model_package_group(
    ModelPackageGroupName=MODEL_PACKAGE_GROUP_NAME
)
print(f"  Status: {group_info['ModelPackageGroupStatus']}")
print(f"  ARN:    {group_info['ModelPackageGroupArn']}")

### 9.3 Write Pipeline Step Scripts

Each ProcessingStep runs a self-contained Python script. We write them locally and upload to S3 so SageMaker can execute them on managed infrastructure.

#### 9.3.1 Data Validation Script

In [ ]:
import os
os.makedirs("/tmp/cicd_scripts", exist_ok=True)

In [ ]:
%%writefile /tmp/cicd_scripts/data_validation.py
"""
VenueSignal CI/CD - Step 1: Data Validation
============================================
Validates incoming feature data before retraining.
Raises an exception (failing the pipeline) if data quality is unacceptable.

Checks performed:
  1. File exists and is non-empty
  2. Required feature columns are present
  3. Missing value rate does not exceed threshold
  4. Target column distribution is not severely imbalanced
  5. Record count meets minimum for reliable training
"""
import argparse
import json
import os
import sys
import pandas as pd
import numpy as np

# ── Thresholds ────────────────────────────────────────────────────────────────
MAX_MISSING_RATE   = 0.20   # Fail if > 20% of any column is null
MIN_RECORDS        = 1000   # Fail if fewer than 1,000 training records
MIN_CLASS_RATE     = 0.05   # Fail if minority class < 5%
REQUIRED_FEATURES  = [
    'avg_review_stars',
    'enhanced_parking_score',
    'business_review_count',
    'is_highly_rated'
]


def validate(input_path: str, output_path: str):
    print("=" * 60)
    print("VenueSignal Data Validation")
    print("=" * 60)

    # ── 1. Load data ──────────────────────────────────────────────────────────
    csv_files = [f for f in os.listdir(input_path) if f.endswith('.csv')]
    if not csv_files:
        raise FileNotFoundError(f"No CSV files found in {input_path}")

    df = pd.read_csv(os.path.join(input_path, csv_files[0]))
    print(f"\n[CHECK 1] File loaded: {csv_files[0]}")
    print(f"          Shape: {df.shape}")

    errors = []

    # ── 2. Minimum record count ───────────────────────────────────────────────
    if len(df) < MIN_RECORDS:
        errors.append(f"Record count {len(df):,} is below minimum {MIN_RECORDS:,}")
        print(f"[CHECK 2] ❌ FAIL — {errors[-1]}")
    else:
        print(f"[CHECK 2] ✓  Record count: {len(df):,} (min: {MIN_RECORDS:,})")

    # ── 3. Required columns ───────────────────────────────────────────────────
    missing_cols = [c for c in REQUIRED_FEATURES if c not in df.columns]
    if missing_cols:
        errors.append(f"Missing required columns: {missing_cols}")
        print(f"[CHECK 3] ❌ FAIL — {errors[-1]}")
    else:
        print(f"[CHECK 3] ✓  All required columns present")

    # ── 4. Missing value rate ─────────────────────────────────────────────────
    missing_rates = df.isnull().mean()
    bad_cols = missing_rates[missing_rates > MAX_MISSING_RATE]
    if len(bad_cols) > 0:
        errors.append(f"High null rate columns: {bad_cols.to_dict()}")
        print(f"[CHECK 4] ❌ FAIL — {errors[-1]}")
    else:
        print(f"[CHECK 4] ✓  Missing rates within threshold (max: {MAX_MISSING_RATE:.0%})")

    # ── 5. Class balance (only if target column exists) ───────────────────────
    if 'is_highly_rated' in df.columns:
        class_dist = df['is_highly_rated'].value_counts(normalize=True)
        minority_rate = class_dist.min()
        if minority_rate < MIN_CLASS_RATE:
            errors.append(f"Severe class imbalance — minority class rate: {minority_rate:.2%}")
            print(f"[CHECK 5] ❌ FAIL — {errors[-1]}")
        else:
            print(f"[CHECK 5] ✓  Class balance OK — minority rate: {minority_rate:.2%}")
            print(f"             Distribution: {class_dist.to_dict()}")

    # ── Write validation report ───────────────────────────────────────────────
    os.makedirs(output_path, exist_ok=True)
    report = {
        "status": "PASS" if not errors else "FAIL",
        "record_count": int(len(df)),
        "column_count": int(len(df.columns)),
        "errors": errors,
        "missing_rates": missing_rates.to_dict()
    }
    report_path = os.path.join(output_path, "validation_report.json")
    with open(report_path, 'w') as f:
        json.dump(report, f, indent=2)

    print(f"\nValidation report written to: {report_path}")

    # ── Fail pipeline if errors found ────────────────────────────────────────
    if errors:
        print(f"\n{'='*60}")
        print("DATA VALIDATION FAILED — pipeline will not proceed")
        print(f"{'='*60}")
        for e in errors:
            print(f"  ✗ {e}")
        sys.exit(1)

    print(f"\n{'='*60}")
    print("DATA VALIDATION PASSED — proceeding to training")
    print(f"{'='*60}")


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--input-path',  default='/opt/ml/processing/input/data')
    parser.add_argument('--output-path', default='/opt/ml/processing/output')
    args = parser.parse_args()
    validate(args.input_path, args.output_path)


#### 9.3.2 Model Evaluation Script

In [ ]:
%%writefile /tmp/cicd_scripts/evaluate.py
"""
VenueSignal CI/CD - Step 3: Model Evaluation
=============================================
Loads the trained XGBoost model and evaluates against the test set.
Writes a metrics JSON that the ConditionStep reads to decide
whether the model is promoted to the Model Registry.

Metrics written:
  classification: accuracy, precision, recall, f1, roc_auc
  regression:     mae, rmse
  business:       within_0.5_stars, within_1.0_stars
"""
import argparse
import json
import os
import tarfile
import numpy as np
import pandas as pd
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, mean_absolute_error, mean_squared_error
)
import xgboost as xgb


def prob_to_stars(prob):
    """Convert probability to star rating (1–5 scale)."""
    return 1 + (prob * 4)


def evaluate(model_path: str, test_path: str, output_path: str):
    print("=" * 60)
    print("VenueSignal Model Evaluation")
    print("=" * 60)

    # ── 1. Load model from tarball ────────────────────────────────────────────
    model_tar = os.path.join(model_path, 'model.tar.gz')
    with tarfile.open(model_tar) as t:
        t.extractall(path=model_path)

    # XGBoost SageMaker saves as xgboost-model
    model_file = os.path.join(model_path, 'xgboost-model')
    booster = xgb.Booster()
    booster.load_model(model_file)
    print(f"\n[1] Model loaded from: {model_file}")

    # ── 2. Load test data ─────────────────────────────────────────────────────
    csv_files = [f for f in os.listdir(test_path) if f.endswith('.csv')]
    test_df = pd.read_csv(os.path.join(test_path, csv_files[0]))
    print(f"[2] Test data loaded: {test_df.shape}")

    # ── 3. Separate features and targets ─────────────────────────────────────
    target_col   = 'is_highly_rated'
    stars_col    = 'review_stars'
    feature_cols = [c for c in test_df.columns if c not in [target_col, stars_col]]

    X_test = test_df[feature_cols].fillna(0)
    y_true_class = test_df[target_col].values
    y_true_stars = test_df[stars_col].values if stars_col in test_df.columns else None

    # ── 4. Predict ────────────────────────────────────────────────────────────
    dtest = xgb.DMatrix(X_test)
    y_prob = booster.predict(dtest)
    y_pred = (y_prob >= 0.5).astype(int)
    print(f"[3] Predictions generated for {len(y_pred):,} records")

    # ── 5. Compute metrics ────────────────────────────────────────────────────
    accuracy  = float(accuracy_score(y_true_class, y_pred))
    precision = float(precision_score(y_true_class, y_pred, zero_division=0))
    recall    = float(recall_score(y_true_class, y_pred, zero_division=0))
    f1        = float(f1_score(y_true_class, y_pred, zero_division=0))
    roc_auc   = float(roc_auc_score(y_true_class, y_prob))

    y_pred_stars = prob_to_stars(y_prob)
    mae  = float(mean_absolute_error(y_true_stars, y_pred_stars)) if y_true_stars is not None else None
    rmse = float(np.sqrt(mean_squared_error(y_true_stars, y_pred_stars))) if y_true_stars is not None else None

    within_0_5 = float(np.mean(np.abs(y_true_stars - y_pred_stars) <= 0.5)) if y_true_stars is not None else None
    within_1_0 = float(np.mean(np.abs(y_true_stars - y_pred_stars) <= 1.0)) if y_true_stars is not None else None

    # ── 6. Build metrics report ───────────────────────────────────────────────
    # NOTE: The top-level 'f1' key is what the ConditionStep reads via JsonGet.
    metrics = {
        "f1": f1,                        # <── ConditionStep reads this key
        "classification": {
            "accuracy":  accuracy,
            "precision": precision,
            "recall":    recall,
            "f1":        f1,
            "roc_auc":   roc_auc
        },
        "regression": {
            "mae":  mae,
            "rmse": rmse
        },
        "business": {
            "within_0.5_stars": within_0_5,
            "within_1.0_stars": within_1_0
        },
        "test_record_count": int(len(test_df))
    }

    # ── 7. Write report ───────────────────────────────────────────────────────
    os.makedirs(output_path, exist_ok=True)
    report_path = os.path.join(output_path, "evaluation_report.json")
    with open(report_path, 'w') as f:
        json.dump(metrics, f, indent=2)

    print(f"\n{'='*60}")
    print("EVALUATION RESULTS")
    print(f"{'='*60}")
    print(f"  Accuracy:        {accuracy:.4f}")
    print(f"  Precision:       {precision:.4f}")
    print(f"  Recall:          {recall:.4f}")
    print(f"  F1 Score:        {f1:.4f}  ← quality gate checks this")
    print(f"  ROC-AUC:         {roc_auc:.4f}")
    if mae is not None:
        print(f"  MAE (stars):     {mae:.4f}")
        print(f"  RMSE (stars):    {rmse:.4f}")
        print(f"  Within 0.5★:     {within_0_5:.2%}")
        print(f"  Within 1.0★:     {within_1_0:.2%}")
    print(f"\nReport written to: {report_path}")


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--model-path', default='/opt/ml/processing/input/model')
    parser.add_argument('--test-path',  default='/opt/ml/processing/input/test')
    parser.add_argument('--output-path',default='/opt/ml/processing/output')
    args = parser.parse_args()
    evaluate(args.model_path, args.test_path, args.output_path)


In [ ]:
# Upload scripts to S3 so SageMaker Processing Jobs can access them
from sagemaker.s3 import S3Uploader

validation_script_s3 = S3Uploader.upload(
    '/tmp/cicd_scripts/data_validation.py',
    CICD_SCRIPTS_S3
)
evaluation_script_s3 = S3Uploader.upload(
    '/tmp/cicd_scripts/evaluate.py',
    CICD_SCRIPTS_S3
)

print(f"✓ Validation script: {validation_script_s3}")
print(f"✓ Evaluation script: {evaluation_script_s3}")

### 9.4 Define the SageMaker Pipeline

The pipeline uses a **`ParameterFloat`** for the F1 threshold so you can override it at runtime without editing the pipeline definition. This is how we trigger the failure demo cleanly.

In [ ]:
import sagemaker
from sagemaker import image_uris

# ── Pipeline parameter: override threshold at execution time ─────────────────
f1_threshold_param = ParameterFloat(
    name="F1Threshold",
    default_value=DEFAULT_F1_THRESHOLD
)

# ── Processors ───────────────────────────────────────────────────────────────
# SKLearnProcessor runs the validation and evaluation scripts on managed infra
sklearn_processor = SKLearnProcessor(
    framework_version='1.0-1',
    role=role,
    instance_type='ml.m5.large',
    instance_count=1,
    sagemaker_session=sagemaker_session,
    base_job_name='venuesignal-cicd'
)

# ── XGBoost container (same as Section 6) ────────────────────────────────────
xgb_container = image_uris.retrieve(
    framework='xgboost',
    region=REGION,
    version='1.7-1'
)

print(f"✓ SKLearnProcessor configured")
print(f"✓ XGBoost container: {xgb_container}")

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
#  STEP 1 — DATA VALIDATION
# ════════════════════════════════════════════════════════════════════════════
step_validation = ProcessingStep(
    name="VenueSignal-DataValidation",
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(
            source=XGB_TRAIN_S3,
            destination='/opt/ml/processing/input/data'
        )
    ],
    outputs=[
        ProcessingOutput(
            source='/opt/ml/processing/output',
            destination=f"{CICD_EVAL_S3}validation/"
        )
    ],
    code='/tmp/cicd_scripts/data_validation.py'
)
print("✓ Step 1 defined: VenueSignal-DataValidation")


# ════════════════════════════════════════════════════════════════════════════
#  STEP 2 — MODEL TRAINING
# ════════════════════════════════════════════════════════════════════════════
xgb_estimator = sagemaker.estimator.Estimator(
    image_uri=xgb_container,
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    output_path=XGB_OUTPUT_S3,
    sagemaker_session=sagemaker_session,
    base_job_name='venuesignal-cicd-train'
)
xgb_estimator.set_hyperparameters(
    objective='binary:logistic',
    num_round=100,
    max_depth=6,
    eta=0.3,
    gamma=0,
    min_child_weight=1,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='auc',
    early_stopping_rounds=10
)

step_training = TrainingStep(
    name="VenueSignal-ModelTraining",
    estimator=xgb_estimator,
    inputs={
        'train':      TrainingInput(XGB_TRAIN_S3, content_type='text/csv'),
        'validation': TrainingInput(XGB_VAL_S3,   content_type='text/csv')
    },
    depends_on=[step_validation]   # Only runs if validation passes
)
print("✓ Step 2 defined: VenueSignal-ModelTraining")


# ════════════════════════════════════════════════════════════════════════════
#  STEP 3 — MODEL EVALUATION
# ════════════════════════════════════════════════════════════════════════════
# PropertyFile lets the ConditionStep read the JSON output of this step
evaluation_report = PropertyFile(
    name="VenueSignalEvaluationReport",
    output_name="evaluation",
    path="evaluation_report.json"
)

step_evaluation = ProcessingStep(
    name="VenueSignal-ModelEvaluation",
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(
            # Pull model artifacts from the training step output
            source=step_training.properties.ModelArtifacts.S3ModelArtifacts,
            destination='/opt/ml/processing/input/model'
        ),
        ProcessingInput(
            source=XGB_TEST_S3,
            destination='/opt/ml/processing/input/test'
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name='evaluation',
            source='/opt/ml/processing/output',
            destination=f"{CICD_EVAL_S3}model-eval/"
        )
    ],
    code='/tmp/cicd_scripts/evaluate.py',
    property_files=[evaluation_report]
)
print("✓ Step 3 defined: VenueSignal-ModelEvaluation")


# ════════════════════════════════════════════════════════════════════════════
#  STEP 4a — REGISTER MODEL (success branch)
# ════════════════════════════════════════════════════════════════════════════
from sagemaker.workflow.step_collections import RegisterModel

step_register = RegisterModel(
    name="VenueSignal-RegisterModel",
    estimator=xgb_estimator,
    model_data=step_training.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.m5.large", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=MODEL_PACKAGE_GROUP_NAME,
    approval_status="PendingManualApproval",
    model_metrics=sagemaker.model_metrics.ModelMetrics(
        model_statistics=sagemaker.model_metrics.MetricsSource(
            s3_uri=f"{CICD_EVAL_S3}model-eval/evaluation_report.json",
            content_type="application/json"
        )
    )
)
print("✓ Step 4a defined: VenueSignal-RegisterModel")


# ════════════════════════════════════════════════════════════════════════════
#  STEP 4b — FAIL STEP (failure branch)
# ════════════════════════════════════════════════════════════════════════════
step_fail = FailStep(
    name="VenueSignal-QualityGateFailed",
    error_message=JsonGet(
        step_name=step_evaluation.name,
        property_file=evaluation_report,
        json_path="f1"        # Surfaces the actual F1 value in the error
    )
)
print("✓ Step 4b defined: VenueSignal-QualityGateFailed")


# ════════════════════════════════════════════════════════════════════════════
#  STEP 4 — CONDITION (quality gate)
# ════════════════════════════════════════════════════════════════════════════
# Reads f1 from the evaluation report JSON and compares to the pipeline parameter
condition_f1 = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=step_evaluation.name,
        property_file=evaluation_report,
        json_path="f1"
    ),
    right=f1_threshold_param
)

step_condition = ConditionStep(
    name="VenueSignal-QualityGate",
    conditions=[condition_f1],
    if_steps=[step_register],      # F1 >= threshold → register
    else_steps=[step_fail]         # F1 <  threshold → fail
)
print("✓ Step 4 defined:  VenueSignal-QualityGate")
print(f"                   Gate: F1 >= {{F1Threshold}} (default: {DEFAULT_F1_THRESHOLD})")

In [ ]:
# ════════════════════════════════════════════════════════════════════════════
#  ASSEMBLE & REGISTER THE PIPELINE
# ════════════════════════════════════════════════════════════════════════════
pipeline = Pipeline(
    name=PIPELINE_NAME,
    parameters=[f1_threshold_param],
    steps=[
        step_validation,   # Step 1
        step_training,     # Step 2  (depends_on step_validation)
        step_evaluation,   # Step 3
        step_condition     # Step 4  (branches to register or fail)
    ],
    sagemaker_session=sagemaker_session
)

# Upsert: creates the pipeline if it doesn't exist, updates it if it does
pipeline_arn = pipeline.upsert(role_arn=role)
print(f"✓ Pipeline registered")
print(f"  Name: {PIPELINE_NAME}")
print(f"  ARN:  {pipeline_arn['PipelineArn']}")
print()
print("DAG structure:")
print("  [1] DataValidation")
print("       └─► [2] ModelTraining")
print("                └─► [3] ModelEvaluation")
print("                         └─► [4] QualityGate (F1 >= threshold?)")
print("                                  ├─ YES ─► [4a] RegisterModel")
print("                                  └─ NO  ─► [4b] QualityGateFailed")

### 9.5 Execute Pipeline — Success Case

Run with the default threshold (`0.70`). The XGBoost model trained in Section 6 should exceed this, so the pipeline will proceed to Model Registration.

In [ ]:
# ── Reconnect to existing registered pipeline ─────────────────────────────────
# Use this whenever 'pipeline' is not defined in the current kernel session
# (e.g. after a kernel restart, or jumping directly to section 9.5)

from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.parameters import ParameterFloat

PIPELINE_NAME        = "venuesignal-cicd-pipeline"
DEFAULT_F1_THRESHOLD = 0.70

# Verify the pipeline exists in AWS before reconnecting
try:
    desc = sm_client.describe_pipeline(PipelineName=PIPELINE_NAME)
    print(f"✓ Pipeline found in AWS")
    print(f"  Name:    {desc['PipelineName']}")
    print(f"  Status:  {desc['PipelineStatus']}")
    print(f"  Updated: {desc['LastModifiedTime'].strftime('%Y-%m-%d %H:%M UTC')}")
except sm_client.exceptions.ResourceNotFound:
    print("✗ Pipeline not found in AWS.")
    print("  Section 9.4 has not been run yet in any session.")
    print("  Please run section 9.4 fully before proceeding.")
    raise

# Reconnect — Pipeline() with just a name and session is enough to call .start()
pipeline = Pipeline(
    name=PIPELINE_NAME,
    sagemaker_session=sagemaker_session
)

# Also rebuild the threshold parameter reference (used in section 9.6 failure demo)
f1_threshold_param = ParameterFloat(
    name="F1Threshold",
    default_value=DEFAULT_F1_THRESHOLD
)

print()
print(f"✓ pipeline object reconnected — ready to call pipeline.start()")
print(f"✓ f1_threshold_param rebuilt (default: {DEFAULT_F1_THRESHOLD})")

In [ ]:
# ── Start a SUCCESS execution ─────────────────────────────────────────────────
# Uses the default F1Threshold = 0.70 (model should pass)

success_execution = pipeline.start(
    execution_display_name="cicd-success-demo",
    execution_description="Success run: F1 threshold set below expected model performance",
    parameters={"F1Threshold": 0.70}
)

print(f"✓ Pipeline execution started")
print(f"  Execution ARN: {success_execution.arn}")
print(f"  Execution name: cicd-success-demo")
print(f"  F1 Threshold: 0.70")
print()
print("View in SageMaker Studio:")
print(f"  Pipelines → {PIPELINE_NAME} → cicd-success-demo")
print()
print("Running wait() to track execution. This takes ~15-25 minutes.")
print("You can also open SageMaker Studio to watch the DAG update live.")
print("To skip waiting: comment out the next cell and track manually.")

In [ ]:
# ── Wait for success execution to complete ────────────────────────────────────
# Comment this out if you want to proceed without waiting

print("Waiting for pipeline execution to complete...")
print("(Each step status will print as they finish)")
print()

import time
from datetime import datetime

def get_steps(execution):
    """Safely extract step list regardless of SDK response shape."""
    raw = execution.list_steps()
    if isinstance(raw, list):
        return raw
    if isinstance(raw, dict):
        return raw.get('PipelineExecutionSteps', [])
    return []


print("Waiting for pipeline execution to complete...")
print("(Each step status will print as they finish)")
print()

seen_steps = set()
while True:
    for step in get_steps(success_execution):
        key = (step['StepName'], step['StepStatus'])
        if key not in seen_steps:
            seen_steps.add(key)
            ts   = datetime.now().strftime('%H:%M:%S')
            icon = "✓" if step['StepStatus'] == 'Succeeded' \
                   else "→" if step['StepStatus'] == 'Executing' \
                   else "✗"
            print(f"  [{ts}] {icon} {step['StepName']}: {step['StepStatus']}")

    desc   = sm_client.describe_pipeline_execution(
                 PipelineExecutionArn=success_execution.arn
             )
    status = desc['PipelineExecutionStatus']
    if status in ('Succeeded', 'Failed', 'Stopped'):
        print()
        print(f"Pipeline execution complete: {status}")
        break
    time.sleep(30)

success_execution_arn = success_execution.arn
%store success_execution_arn

In [ ]:
# ── Print execution summary ───────────────────────────────────────────────────

def get_steps(execution):
    """Safely extract step list regardless of SDK response shape."""
    raw = execution.list_steps()
    if isinstance(raw, list):
        return raw
    if isinstance(raw, dict):
        return raw.get('PipelineExecutionSteps', [])
    return []


# ── Overall execution status ──────────────────────────────────────────────────
desc   = sm_client.describe_pipeline_execution(
             PipelineExecutionArn=success_execution.arn
         )
status = desc['PipelineExecutionStatus']

print("=" * 60)
print("EXECUTION SUMMARY")
print("=" * 60)
print(f"  Status:      {status}")
print(f"  Started:     {desc['CreationTime']}")
print(f"  Last update: {desc['LastModifiedTime']}")
print()

# ── Per-step results ──────────────────────────────────────────────────────────
steps = get_steps(success_execution)
print("  Step results:")
failed_steps = []
for step in steps:
    icon = "✓" if step['StepStatus'] == 'Succeeded' else "✗"
    print(f"    {icon} {step['StepName']}: {step['StepStatus']}")
    if step['StepStatus'] in ('Failed', 'Stopped'):
        failed_steps.append(step)

# ── Failure diagnosis ─────────────────────────────────────────────────────────
if failed_steps:
    print()
    print("=" * 60)
    print("FAILURE DIAGNOSIS")
    print("=" * 60)
    for step in failed_steps:
        print(f"\n  Failed step: {step['StepName']}")

        # Failure reason from the step itself
        reason = step.get('FailureReason', '')
        if reason:
            print(f"  Reason:      {reason}")

        # For ProcessingJob steps — get the actual CloudWatch log
        metadata = step.get('Metadata', {})

        if 'ProcessingJob' in metadata:
            job_arn  = metadata['ProcessingJob'].get('Arn', '')
            job_name = job_arn.split('/')[-1] if job_arn else ''
            if job_name:
                print(f"  Job name:    {job_name}")
                try:
                    job_desc = sm_client.describe_processing_job(
                        ProcessingJobName=job_name
                    )
                    print(f"  Exit msg:    {job_desc.get('ExitMessage', 'N/A')}")
                    print(f"  Failure:     {job_desc.get('FailureReason', 'N/A')}")
                    print()
                    print("  To view full logs run:")
                    print(f"    CloudWatch → Log groups → /aws/sagemaker/ProcessingJobs")
                    print(f"    Log stream prefix: {job_name}")
                except Exception as e:
                    print(f"  Could not retrieve job details: {e}")

        if 'TrainingJob' in metadata:
            job_arn  = metadata['TrainingJob'].get('Arn', '')
            job_name = job_arn.split('/')[-1] if job_arn else ''
            if job_name:
                print(f"  Job name:    {job_name}")
                try:
                    job_desc = sm_client.describe_training_job(
                        TrainingJobName=job_name
                    )
                    print(f"  Exit msg:    {job_desc.get('SecondaryStatus', 'N/A')}")
                    print(f"  Failure:     {job_desc.get('FailureReason', 'N/A')}")
                except Exception as e:
                    print(f"  Could not retrieve job details: {e}")

    print()
    print("  Most likely causes:")
    print("    1. Training data not found at XGB_TRAIN_S3 path (check Section 5 ran)")
    print("    2. Evaluation script failed to find model.tar.gz (check training completed)")
    print("    3. IAM/LabRole permission on S3 path")
    print()
    print("  Quick path checks:")
    s3 = boto3.client('s3', region_name=REGION)
    paths_to_check = [
        (BASE_BUCKET_NAME, 'feature-store/training-data/train.csv',      'Training data'),
        (BASE_BUCKET_NAME, 'feature-store/training-data/validation.csv', 'Validation data'),
        (BASE_BUCKET_NAME, 'feature-store/training-data/test.csv',       'Test data'),
    ]
    for bucket, key, label in paths_to_check:
        try:
            s3.head_object(Bucket=bucket, Key=key)
            print(f"    ✓ {label}: s3://{bucket}/{key}")
        except Exception:
            print(f"    ✗ {label} NOT FOUND: s3://{bucket}/{key}")
            print(f"      → Re-run Section 5 to regenerate training splits")

elif status == 'Succeeded':
    print()
    # Retrieve evaluation metrics from S3
    try:
        s3_client = boto3.client('s3', region_name=REGION)
        eval_obj  = s3_client.get_object(
            Bucket=BASE_BUCKET_NAME,
            Key=f"{CICD_EVAL_PREFIX}model-eval/evaluation_report.json"
        )
        eval_metrics = json.loads(eval_obj['Body'].read())
        print("  Evaluation metrics:")
        print(f"    F1 Score:  {eval_metrics['f1']:.4f}")
        print(f"    Accuracy:  {eval_metrics['classification']['accuracy']:.4f}")
        print(f"    ROC-AUC:   {eval_metrics['classification']['roc_auc']:.4f}")
        if eval_metrics['regression']['mae']:
            print(f"    MAE:       {eval_metrics['regression']['mae']:.4f}")
    except Exception as e:
        print(f"  (Could not retrieve evaluation report: {e})")

### 9.6 Execute Pipeline — Failure Case (Video Demo)

We re-run the pipeline with `F1Threshold = 0.99` so the ConditionStep routes to the `FailStep`. This demonstrates the quality gate working as intended.

In [ ]:
# ── Start a FAILURE execution ─────────────────────────────────────────────────
# Override F1Threshold to 0.99 — model cannot reach this, pipeline will fail

failure_execution = pipeline.start(
    execution_display_name="cicd-failure-demo",
    execution_description="Failure run: F1 threshold set above model capability to demo quality gate",
    parameters={"F1Threshold": 0.99}
)

print(f"✓ Failure demo execution started")
print(f"  Execution ARN: {failure_execution.arn}")
print(f"  F1 Threshold: 0.99  ← model cannot reach this")
print()
print("Expected flow:")
print("  DataValidation  → Succeeded")
print("  ModelTraining   → Succeeded")
print("  ModelEvaluation → Succeeded")
print("  QualityGate     → Condition evaluates False (F1 < 0.99)")
print("  QualityGateFailed → Failed  ← this is what we want to see")

failure_execution_arn = failure_execution.arn
%store failure_execution_arn

In [ ]:
# ── Wait for failure execution ────────────────────────────────────────────────
# Fixed: handles list_steps() returning a list OR a dict

import time
from datetime import datetime

def get_steps(execution):
    """Safely extract step list regardless of SDK response shape."""
    raw = execution.list_steps()
    if isinstance(raw, list):
        return raw
    if isinstance(raw, dict):
        return raw.get('PipelineExecutionSteps', [])
    return []


print("Waiting for failure demo execution to complete...")
print()

seen_steps = set()
while True:
    for step in get_steps(failure_execution):
        key = (step['StepName'], step['StepStatus'])
        if key not in seen_steps:
            seen_steps.add(key)
            ts   = datetime.now().strftime('%H:%M:%S')
            icon = "✓" if step['StepStatus'] == 'Succeeded' \
                   else "→" if step['StepStatus'] == 'Executing' \
                   else "✗"
            print(f"  [{ts}] {icon} {step['StepName']}: {step['StepStatus']}")

    desc   = sm_client.describe_pipeline_execution(
                 PipelineExecutionArn=failure_execution.arn
             )
    status = desc['PipelineExecutionStatus']
    if status in ('Succeeded', 'Failed', 'Stopped'):
        print()
        print(f"Pipeline execution complete: {status}")
        break
    time.sleep(30)

In [ ]:
# ── Failure execution summary ─────────────────────────────────────────────────
# Fixed: handles list_steps() returning a list OR a dict

desc   = sm_client.describe_pipeline_execution(
             PipelineExecutionArn=failure_execution.arn
         )
status = desc['PipelineExecutionStatus']

print("=" * 60)
print("FAILURE EXECUTION SUMMARY")
print("=" * 60)
print(f"  Status:      {status}")
print(f"  Started:     {desc['CreationTime']}")
print(f"  Last update: {desc['LastModifiedTime']}")
print()

steps = get_steps(failure_execution)
print("  Step results:")
for step in steps:
    icon = "✓" if step['StepStatus'] == 'Succeeded' else "✗"
    print(f"    {icon} {step['StepName']}: {step['StepStatus']}")

print()

# ── Distinguish expected failure (QualityGate) from unexpected failure ─────────
step_names   = {s['StepName']: s['StepStatus'] for s in steps}
gate_failed  = step_names.get('VenueSignal-QualityGateFailed') == 'Failed'
infra_failed = any(
    step_names.get(s) == 'Failed'
    for s in ['VenueSignal-DataValidation',
               'VenueSignal-ModelTraining',
               'VenueSignal-ModelEvaluation']
)

if gate_failed and not infra_failed:
    print("✓ Quality gate worked as intended.")
    print("  All pipeline steps succeeded, but F1 did not reach 0.99.")
    print("  The FailStep triggered correctly — model was blocked from registration.")
    print()
    print("  This is the state to record for your video demonstration.")
    print("  In SageMaker Studio the DAG will show:")
    print("    ✓ DataValidation")
    print("    ✓ ModelTraining")
    print("    ✓ ModelEvaluation")
    print("    ✓ QualityGate  (condition evaluated — routed to else branch)")
    print("    ✗ QualityGateFailed  (red — expected)")
elif infra_failed:
    print("⚠  An infrastructure step failed before reaching the quality gate.")
    print("   This is NOT the intended failure — check the step details above.")
    print("   Resolve the underlying issue in the success run first,")
    print("   then re-run this failure demo.")
else:
    print(f"  Pipeline status: {status}")
    print("  Check SageMaker Studio for the full DAG view.")

### 9.7 View Registered Models in Model Registry

Only the **success execution** should have registered a model. This cell confirms the registry state and shows how to approve a model for deployment.

In [ ]:
# ── List all model versions in the package group ──────────────────────────────
print("=" * 60)
print(f"MODEL REGISTRY: {MODEL_PACKAGE_GROUP_NAME}")
print("=" * 60)

try:
    model_packages = sm_client.list_model_packages(
        ModelPackageGroupName=MODEL_PACKAGE_GROUP_NAME,
        SortBy='CreationTime',
        SortOrder='Descending'
    )

    packages = model_packages['ModelPackageSummaryList']
    if not packages:
        print("No model versions registered yet.")
    else:
        print(f"  {len(packages)} model version(s) found:\n")
        for pkg in packages:
            print(f"  Version:   {pkg['ModelPackageVersion']}")
            print(f"  Status:    {pkg['ModelApprovalStatus']}")
            print(f"  Created:   {pkg['CreationTime'].strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"  ARN:       {pkg['ModelPackageArn']}")
            print()

except Exception as e:
    print(f"Error accessing Model Registry: {e}")

In [ ]:
# ── Approve the latest registered model ───────────────────────────────────────
# In production this would be a manual review step.
# For the course demo we approve programmatically.

try:
    model_packages = sm_client.list_model_packages(
        ModelPackageGroupName=MODEL_PACKAGE_GROUP_NAME,
        ModelApprovalStatus='PendingManualApproval',
        SortBy='CreationTime',
        SortOrder='Descending'
    )

    latest = model_packages['ModelPackageSummaryList']
    if latest:
        model_pkg_arn = latest[0]['ModelPackageArn']
        sm_client.update_model_package(
            ModelPackageArn=model_pkg_arn,
            ModelApprovalStatus='Approved'
        )
        print(f"✓ Model approved for deployment")
        print(f"  ARN:     {model_pkg_arn}")
        print(f"  Status:  Approved")
        approved_model_arn = model_pkg_arn
        %store approved_model_arn
    else:
        print("No models pending approval.")

except Exception as e:
    print(f"Could not approve model: {e}")

### 9.8 Pipeline Execution History

Print a summary of all executions — useful for confirming the success/failure pair in your video and design document.

In [ ]:
# ── Full execution history for the pipeline ───────────────────────────────────
print("=" * 70)
print(f"EXECUTION HISTORY: {PIPELINE_NAME}")
print("=" * 70)

history = sm_client.list_pipeline_executions(
    PipelineName=PIPELINE_NAME,
    SortBy='CreationTime',
    SortOrder='Descending'
)

for exec_summary in history['PipelineExecutionSummaries']:
    status = exec_summary['PipelineExecutionStatus']
    icon = "✅" if status == 'Succeeded' else "❌" if status == 'Failed' else "⏳"
    name = exec_summary.get('PipelineExecutionDisplayName', 'N/A')
    created = exec_summary['StartTime'].strftime('%Y-%m-%d %H:%M:%S')
    print(f"  {icon} [{created}] {name}: {status}")

print()
print("View DAG visually in SageMaker Studio:")
print(f"  Home → Pipelines → {PIPELINE_NAME} → [select execution] → Graph")

### 9.9 CI/CD Section Summary

Print a complete reference summary of all CI/CD resources

In [ ]:
print("=" * 70)
print("CI/CD SECTION — COMPLETE RESOURCE SUMMARY")
print("VenueSignal | AAI-540 Group 6")
print("=" * 70)

print("\nS3 Resources (bucket: yelp-aai540-group6-{account_id}):")
print(f"  Scripts:    s3://{BASE_BUCKET_NAME}/{CICD_SCRIPTS_PREFIX}")
print(f"  Validation: s3://{BASE_BUCKET_NAME}/{CICD_EVAL_PREFIX}validation/")
print(f"  Evaluation: s3://{BASE_BUCKET_NAME}/{CICD_EVAL_PREFIX}model-eval/")
print(f"  Training:   s3://{BASE_BUCKET_NAME}/{MODEL_PREFIX}cicd-runs/")

print("\nSageMaker Pipeline:")
print(f"  Name:   {PIPELINE_NAME}")
print(f"  Steps:  DataValidation → ModelTraining → ModelEvaluation → QualityGate")
print(f"  Gate:   F1 >= F1Threshold parameter (default: {DEFAULT_F1_THRESHOLD})")

print("\nModel Registry:")
print(f"  Group:  {MODEL_PACKAGE_GROUP_NAME}")
print(f"  Only models passing the quality gate are registered")

print("\nExecution History:")
try:
    history = sm_client.list_pipeline_executions(
        PipelineName=PIPELINE_NAME, SortBy='CreationTime', SortOrder='Descending'
    )
    for e in history['PipelineExecutionSummaries']:
        icon = "✅" if e['PipelineExecutionStatus'] == 'Succeeded' else "❌"
        name = e.get('PipelineExecutionDisplayName', 'N/A')
        print(f"  {icon} {name}: {e['PipelineExecutionStatus']}")
except Exception:
    print("  (Run pipeline executions first)")

print("=" * 70)

## References

- Amazon Web Services. (n.d.). *Amazon SageMaker developer guide*. https://docs.aws.amazon.com/sagemaker/
- Amazon Web Services. (n.d.). *AWS SDK for Python (Boto3) documentation*. https://boto3.amazonaws.com/v1/documentation/api/latest/index.html
- Anthropic. (2024). *Claude* (Version 4.5 Sonnet) [Large language model]. https://www.anthropic.com/claude
- Chen, T., & Guestrin, C. (2016). XGBoost: A scalable tree boosting system. *Proceedings of the 22nd ACM SIGKDD* (pp. 785–794). https://doi.org/10.1145/2939672.2939785
- Harris, C. R., et al. (2020). Array programming with NumPy. *Nature, 585*(7825), 357–362. https://doi.org/10.1038/s41586-020-2649-2
- Hunter, J. D. (2007). Matplotlib: A 2D graphics environment. *Computing in Science & Engineering, 9*(3), 90–95. https://doi.org/10.1109/MCSE.2007.55
- Huyen, C. (2022). *Designing machine learning systems: An iterative process for production-ready applications*. O'Reilly Media.
- McKinney, W. (2010). Data structures for statistical computing in Python. *Proceedings of the 9th Python in Science Conference* (pp. 51–56).
- Pedregosa, F., et al. (2011). Scikit-learn: Machine learning in Python. *JMLR, 12*, 2825–2830.
- Waskom, M. L. (2021). seaborn: Statistical data visualization. *JOSS, 6*(60), 3021. https://doi.org/10.21105/joss.03021
- Yelp. (n.d.). *Yelp Open Dataset*. https://www.yelp.com/dataset

This project utilised Claude (Anthropic) and ChatGPT (OpenAI) for code debugging, documentation assistance, and technical guidance.
